In [ ]:
import re
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import math
from datetime import datetime
df = pd.read_csv('../input/train.tsv', delimiter='\t',low_memory=True, header = 0)
#df = pd.read_csv('/home/abhishek_jha/mercari/train.tsv', delimiter='\t',low_memory=True, header = 0)
#df = pd.read_csv('../input/train-test-data-with-id/mercari_train_with_id.csv000000000000', header = 0)
#df_train.head()
category_name_dict = {1:"Beauty/Bath & Body/Bath", 2:"Beauty/Bath & Body/Bathing Accessories", 3:"Beauty/Bath & Body/Cleansers", 4:"Beauty/Bath & Body/Other", 5:"Beauty/Bath & Body/Scrubs & Body Treatments", 6:"Beauty/Bath & Body/Sets", 7:"Beauty/Fragrance/Candles & Home Scents", 8:"Beauty/Fragrance/Kids", 9:"Beauty/Fragrance/Men", 10:"Beauty/Fragrance/Other", 11:"Beauty/Fragrance/Sets", 12:"Beauty/Fragrance/Women", 13:"Beauty/Hair Care/Conditioners", 14:"Beauty/Hair Care/Hair & Scalp Treatments", 15:"Beauty/Hair Care/Hair Color", 16:"Beauty/Hair Care/Hair Loss Products", 17:"Beauty/Hair Care/Hair Perms & Texturizers", 18:"Beauty/Hair Care/Hair Relaxers", 19:"Beauty/Hair Care/Other", 20:"Beauty/Hair Care/Shampoo & Conditioner Sets", 21:"Beauty/Hair Care/Shampoo Plus Conditioner", 22:"Beauty/Hair Care/Shampoos", 23:"Beauty/Hair Care/Styling Products", 24:"Beauty/Hair Care/Styling Tools", 25:"Beauty/Makeup/Body", 26:"Beauty/Makeup/Brushes & Applicators", 27:"Beauty/Makeup/Eyes", 28:"Beauty/Makeup/Face", 29:"Beauty/Makeup/Lips", 30:"Beauty/Makeup/Makeup Palettes", 31:"Beauty/Makeup/Makeup Remover", 32:"Beauty/Makeup/Makeup Sets", 33:"Beauty/Makeup/Nails", 34:"Beauty/Makeup/Other", 35:"Beauty/Other/Other", 36:"Beauty/Skin Care/Body", 37:"Beauty/Skin Care/Eyes", 38:"Beauty/Skin Care/Face", 39:"Beauty/Skin Care/Feet", 40:"Beauty/Skin Care/Hands & Nails", 41:"Beauty/Skin Care/Lips", 42:"Beauty/Skin Care/Maternity", 43:"Beauty/Skin Care/Other", 44:"Beauty/Skin Care/Sets & Kits", 45:"Beauty/Skin Care/Sun", 46:"Beauty/Tools & Accessories/Bags & Cases", 47:"Beauty/Tools & Accessories/Bathing Accessories", 48:"Beauty/Tools & Accessories/Cotton & Swabs", 49:"Beauty/Tools & Accessories/Epilators", 50:"Beauty/Tools & Accessories/Hair Coloring Tools", 51:"Beauty/Tools & Accessories/Hair Styling Tools", 52:"Beauty/Tools & Accessories/Makeup Brushes & Tools", 53:"Beauty/Tools & Accessories/Mirrors", 54:"Beauty/Tools & Accessories/Nail Tools", 55:"Beauty/Tools & Accessories/Other", 56:"Beauty/Tools & Accessories/Toiletry Kits", 57:"Beauty/Tools & Accessories/Tweezers", 58:"Beauty/Tools & Accessories/Waxing", 59:"Electronics/Cameras & Photography/Binoculars & Telescopes", 60:"Electronics/Cameras & Photography/Camcorders", 61:"Electronics/Cameras & Photography/Camera & Photo Accessories", 62:"Electronics/Cameras & Photography/Digital Cameras", 63:"Electronics/Cameras & Photography/Film Photography", 64:"Electronics/Cameras & Photography/Flashes & Flash Accessories", 65:"Electronics/Cameras & Photography/Lenses & Filters", 66:"Electronics/Cameras & Photography/Lighting & Studio", 67:"Electronics/Cameras & Photography/Other", 68:"Electronics/Cameras & Photography/Tripods & Supports", 69:"Electronics/Car Audio, Video & GPS/Car Security & Convenience", 70:"Electronics/Car Audio, Video & GPS/Car Speakers & Systems", 71:"Electronics/Car Audio, Video & GPS/Car Stereos & Components", 72:"Electronics/Car Audio, Video & GPS/Car Subwoofers", 73:"Electronics/Car Audio, Video & GPS/Car Video", 74:"Electronics/Car Audio, Video & GPS/GPS Accessories & Mounts", 75:"Electronics/Car Audio, Video & GPS/GPS Units & Equipment", 76:"Electronics/Car Audio, Video & GPS/Other", 77:"Electronics/Car Audio, Video & GPS/Other Accessories", 78:"Electronics/Cell Phones & Accessories/Batteries", 79:"Electronics/Cell Phones & Accessories/Cables & Adapters", 80:"Electronics/Cell Phones & Accessories/Cases, Covers & Skins", 81:"Electronics/Cell Phones & Accessories/Cell Phone Accessories", 82:"Electronics/Cell Phones & Accessories/Cell Phones & Smartphones", 83:"Electronics/Cell Phones & Accessories/Chargers & Cradles", 84:"Electronics/Cell Phones & Accessories/Headsets", 85:"Electronics/Cell Phones & Accessories/Other", 86:"Electronics/Cell Phones & Accessories/Screen Protectors", 87:"Electronics/Computers & Tablets/Components & Parts", 88:"Electronics/Computers & Tablets/Desktops & All-In-Ones", 89:"Electronics/Computers & Tablets/Drives, Storage & Media", 90:"Electronics/Computers & Tablets/Laptops & Netbooks", 91:"Electronics/Computers & Tablets/Networking & Connectivity", 92:"Electronics/Computers & Tablets/Other", 93:"Electronics/Computers & Tablets/Printers, Scanners & Supplies", 94:"Electronics/Computers & Tablets/iPad/Tablet/eBook Access", 95:"Electronics/Computers & Tablets/iPad/Tablet/eBook Readers", 96:"Electronics/Media/Blu-Ray", 97:"Electronics/Media/CD", 98:"Electronics/Media/DVD", 99:"Electronics/Media/Other", 100:"Electronics/Media/VHS", 101:"Electronics/Other/Other", 102:"Electronics/TV, Audio & Surveillance/DVD & Blu-ray Players", 103:"Electronics/TV, Audio & Surveillance/Gadgets", 104:"Electronics/TV, Audio & Surveillance/Headphones", 105:"Electronics/TV, Audio & Surveillance/Home Audio", 106:"Electronics/TV, Audio & Surveillance/Home Speakers & Subwoofers", 107:"Electronics/TV, Audio & Surveillance/Home Surveillance", 108:"Electronics/TV, Audio & Surveillance/Other", 109:"Electronics/TV, Audio & Surveillance/Portable Audio & Accessories", 110:"Electronics/TV, Audio & Surveillance/Televisions", 111:"Electronics/Video Games & Consoles/Accessories", 112:"Electronics/Video Games & Consoles/Consoles", 113:"Electronics/Video Games & Consoles/Games", 114:"Electronics/Video Games & Consoles/Other", 115:"Electronics/Video Games & Consoles/Replacement Parts & Tools", 116:"Electronics/Video Games & Consoles/Strategy Guides", 117:"Electronics/Video Games & Consoles/Video Gaming Merchandise", 118:"Handmade/Accessories/Apron", 119:"Handmade/Accessories/Belt", 120:"Handmade/Accessories/Case", 121:"Handmade/Accessories/Charm", 122:"Handmade/Accessories/Cozy", 123:"Handmade/Accessories/Cuff", 124:"Handmade/Accessories/Eyewear", 125:"Handmade/Accessories/Gloves", 126:"Handmade/Accessories/Hair", 127:"Handmade/Accessories/Hat", 128:"Handmade/Accessories/Keychain", 129:"Handmade/Accessories/Lanyard", 130:"Handmade/Accessories/Leg Warmers", 131:"Handmade/Accessories/Men", 132:"Handmade/Accessories/Mirror", 133:"Handmade/Accessories/Necktie", 134:"Handmade/Accessories/Other", 135:"Handmade/Accessories/Patch", 136:"Handmade/Accessories/Pin", 137:"Handmade/Accessories/Pinback Button", 138:"Handmade/Accessories/Scarf", 139:"Handmade/Accessories/Wallet", 140:"Handmade/Accessories/Watch", 141:"Handmade/Accessories/Women", 142:"Handmade/Art/Aceo", 143:"Handmade/Art/Collages", 144:"Handmade/Art/Drawings", 145:"Handmade/Art/Illustration", 146:"Handmade/Art/Mixed Media", 147:"Handmade/Art/Other", 148:"Handmade/Art/Painting", 149:"Handmade/Art/Photography", 150:"Handmade/Art/Print", 151:"Handmade/Art/Printmaking", 152:"Handmade/Art/Sculptures", 153:"Handmade/Bags and Purses/Backpack", 154:"Handmade/Bags and Purses/Clutch", 155:"Handmade/Bags and Purses/Diaper Bag", 156:"Handmade/Bags and Purses/Hip Bag", 157:"Handmade/Bags and Purses/Laptop", 158:"Handmade/Bags and Purses/Novelty", 159:"Handmade/Bags and Purses/Other", 160:"Handmade/Bags and Purses/Pouch", 161:"Handmade/Bags and Purses/Purse", 162:"Handmade/Bags and Purses/Tote", 163:"Handmade/Bags and Purses/Wallet", 164:"Handmade/Bags and Purses/Wristlet", 165:"Handmade/Books and Zines/Book", 166:"Handmade/Books and Zines/Bookmark", 167:"Handmade/Books and Zines/Comic", 168:"Handmade/Books and Zines/Journal", 169:"Handmade/Books and Zines/Other", 170:"Handmade/Candles/Other", 171:"Handmade/Candles/Scrapbooking", 172:"Handmade/Candles/Stationery", 173:"Handmade/Candles/Sticker", 174:"Handmade/Ceramics and Pottery/Bowls", 175:"Handmade/Ceramics and Pottery/Coasters", 176:"Handmade/Ceramics and Pottery/Home Decor", 177:"Handmade/Ceramics and Pottery/Jewelry", 178:"Handmade/Ceramics and Pottery/Kitchen", 179:"Handmade/Ceramics and Pottery/Other", 180:"Handmade/Ceramics and Pottery/Planters", 181:"Handmade/Ceramics and Pottery/Sculptures", 182:"Handmade/Ceramics and Pottery/Supplies", 183:"Handmade/Ceramics and Pottery/Tiles", 184:"Handmade/Ceramics and Pottery/Vases", 185:"Handmade/Children/Accessories", 186:"Handmade/Children/Art", 187:"Handmade/Children/Baby", 188:"Handmade/Children/Clothing", 189:"Handmade/Children/Furniture", 190:"Handmade/Children/Jewelry", 191:"Handmade/Children/Other", 192:"Handmade/Children/Toddler", 193:"Handmade/Children/Toy", 194:"Handmade/Clothing/Children", 195:"Handmade/Clothing/Corset", 196:"Handmade/Clothing/Costume", 197:"Handmade/Clothing/Dress", 198:"Handmade/Clothing/Lingerie", 199:"Handmade/Clothing/Other", 200:"Handmade/Clothing/Shirt", 201:"Handmade/Clothing/Shoes", 202:"Handmade/Clothing/Tshirt", 203:"Handmade/Clothing/Women", 204:"Handmade/Crochet/Accessories", 205:"Handmade/Crochet/Afghan", 206:"Handmade/Crochet/Bags and Purses", 207:"Handmade/Crochet/Clothing", 208:"Handmade/Crochet/Doll", 209:"Handmade/Crochet/Hat", 210:"Handmade/Crochet/Housewares", 211:"Handmade/Crochet/Other", 212:"Handmade/Crochet/Scarf", 213:"Handmade/Crochet/Supplies", 214:"Handmade/Dolls and Miniatures/Animals", 215:"Handmade/Dolls and Miniatures/Art Doll", 216:"Handmade/Dolls and Miniatures/Artist Bears", 217:"Handmade/Dolls and Miniatures/Child Friendly", 218:"Handmade/Dolls and Miniatures/Fantasy", 219:"Handmade/Dolls and Miniatures/Fashion Dolls Apparel", 220:"Handmade/Dolls and Miniatures/Figurines", 221:"Handmade/Dolls and Miniatures/Human Figure Doll", 222:"Handmade/Dolls and Miniatures/Miniature", 223:"Handmade/Dolls and Miniatures/Other", 224:"Handmade/Dolls and Miniatures/Plush", 225:"Handmade/Dolls and Miniatures/Primitive", 226:"Handmade/Dolls and Miniatures/Scale Dollhouse Miniature", 227:"Handmade/Dolls and Miniatures/Scale Models", 228:"Handmade/Furniture/Fixture", 229:"Handmade/Furniture/Other", 230:"Handmade/Furniture/Shelf", 231:"Handmade/Geekery/Accessory", 232:"Handmade/Geekery/Fantasy", 233:"Handmade/Geekery/Gadget", 234:"Handmade/Geekery/Horror", 235:"Handmade/Geekery/Housewares", 236:"Handmade/Geekery/Humor", 237:"Handmade/Geekery/Jewelry", 238:"Handmade/Geekery/Magic", 239:"Handmade/Geekery/Other", 240:"Handmade/Geekery/Science", 241:"Handmade/Geekery/Toy", 242:"Handmade/Geekery/Videogame", 243:"Handmade/Glass/Beads", 244:"Handmade/Glass/Bottles", 245:"Handmade/Glass/Bowls", 246:"Handmade/Glass/Dishes", 247:"Handmade/Glass/Glassware", 248:"Handmade/Glass/Home Decor", 249:"Handmade/Glass/Jewelry", 250:"Handmade/Glass/Mirrors", 251:"Handmade/Glass/Ornaments", 252:"Handmade/Glass/Other", 253:"Handmade/Glass/Paperweights", 254:"Handmade/Glass/Sculptures", 255:"Handmade/Glass/Stained Glass", 256:"Handmade/Glass/Supplies", 257:"Handmade/Glass/Vases", 258:"Handmade/Holidays/Birthday", 259:"Handmade/Holidays/Christmas", 260:"Handmade/Holidays/Easter", 261:"Handmade/Holidays/Halloween", 262:"Handmade/Holidays/New Years", 263:"Handmade/Holidays/Other", 264:"Handmade/Holidays/Patriotic", 265:"Handmade/Holidays/St Patricks", 266:"Handmade/Holidays/Thanksgiving", 267:"Handmade/Holidays/Valentine", 268:"Handmade/Housewares/Bathroom", 269:"Handmade/Housewares/Cleaning", 270:"Handmade/Housewares/Entertaining/Serving", 271:"Handmade/Housewares/Home Decor", 272:"Handmade/Housewares/Kitchen", 273:"Handmade/Housewares/Lighting", 274:"Handmade/Housewares/Magnets", 275:"Handmade/Housewares/Office", 276:"Handmade/Housewares/Other", 277:"Handmade/Housewares/Pillows", 278:"Handmade/Housewares/Storage Solutions", 279:"Handmade/Housewares/Wall Decor", 280:"Handmade/Jewelry/Books and Zines", 281:"Handmade/Jewelry/Clothing", 282:"Handmade/Jewelry/Jewelry", 283:"Handmade/Jewelry/Music", 284:"Handmade/Jewelry/Other", 285:"Handmade/Knitting/Accessories", 286:"Handmade/Knitting/Bags and Purses", 287:"Handmade/Knitting/Blanket", 288:"Handmade/Knitting/Cozy", 289:"Handmade/Knitting/Doll", 290:"Handmade/Knitting/Hat", 291:"Handmade/Knitting/Knitting Supplies", 292:"Handmade/Knitting/Other", 293:"Handmade/Knitting/Scarf", 294:"Handmade/Knitting/Women", 295:"Handmade/Music/Equipment", 296:"Handmade/Music/Instrument", 297:"Handmade/Music/Other", 298:"Handmade/Music/Poster", 299:"Handmade/Music/Tape", 300:"Handmade/Music/Vinyl", 301:"Handmade/Needlecraft/Accessories", 302:"Handmade/Needlecraft/Clothing", 303:"Handmade/Needlecraft/Cross Stitch", 304:"Handmade/Needlecraft/Doll", 305:"Handmade/Needlecraft/Embroidery", 306:"Handmade/Needlecraft/Felted", 307:"Handmade/Needlecraft/Needlepoint", 308:"Handmade/Needlecraft/Other", 309:"Handmade/Needlecraft/Pattern", 310:"Handmade/Needlecraft/Pillow", 311:"Handmade/Needlecraft/Supplies", 312:"Handmade/Other/Other", 313:"Handmade/Others/Custom", 314:"Handmade/Others/Educational", 315:"Handmade/Others/Graphic Design", 316:"Handmade/Others/Magic", 317:"Handmade/Others/Other", 318:"Handmade/Others/Personalized", 319:"Handmade/Others/Religious", 320:"Handmade/Paper Goods/Bookmark", 321:"Handmade/Paper Goods/Calendar", 322:"Handmade/Paper Goods/Cards", 323:"Handmade/Paper Goods/Gift Wrap", 324:"Handmade/Paper Goods/Journal", 325:"Handmade/Paper Goods/Notebook", 326:"Handmade/Paper Goods/Origami", 327:"Handmade/Paper Goods/Other", 328:"Handmade/Paper Goods/Pad", 329:"Handmade/Paper Goods/Papermaking", 330:"Handmade/Paper Goods/Scrapbooking", 331:"Handmade/Paper Goods/Stationery", 332:"Handmade/Paper Goods/Sticker", 333:"Handmade/Paper Goods/Tag", 334:"Handmade/Patterns/Accessories", 335:"Handmade/Patterns/Baby", 336:"Handmade/Patterns/Beading", 337:"Handmade/Patterns/Crochet", 338:"Handmade/Patterns/Cross Stitch", 339:"Handmade/Patterns/Doll Clothing", 340:"Handmade/Patterns/Embroidery", 341:"Handmade/Patterns/Handmade", 342:"Handmade/Patterns/Knitting", 343:"Handmade/Patterns/Other", 344:"Handmade/Patterns/Painting", 345:"Handmade/Patterns/Quilt", 346:"Handmade/Patterns/Sewing", 347:"Handmade/Pets/Accessories", 348:"Handmade/Pets/Bed", 349:"Handmade/Pets/Blanket", 350:"Handmade/Pets/Bowl", 351:"Handmade/Pets/Clothing", 352:"Handmade/Pets/Collar", 353:"Handmade/Pets/Feeding", 354:"Handmade/Pets/Grooming", 355:"Handmade/Pets/House", 356:"Handmade/Pets/Leash", 357:"Handmade/Pets/Other", 358:"Handmade/Pets/Pet Lover", 359:"Handmade/Pets/Small Animal", 360:"Handmade/Pets/Tag", 361:"Handmade/Pets/Toy", 362:"Handmade/Quilts/Baby", 363:"Handmade/Quilts/Bed", 364:"Handmade/Quilts/Fabric Postcard", 365:"Handmade/Quilts/Other", 366:"Handmade/Quilts/Patch", 367:"Handmade/Quilts/Patchwork", 368:"Handmade/Quilts/Pillow", 369:"Handmade/Quilts/Trim", 370:"Handmade/Quilts/Wall Hanging", 371:"Handmade/Toys/Bear", 372:"Handmade/Toys/Children", 373:"Handmade/Toys/Doll Clothes", 374:"Handmade/Toys/Food", 375:"Handmade/Toys/Game", 376:"Handmade/Toys/Other", 377:"Handmade/Toys/Plush", 378:"Handmade/Toys/Sports", 379:"Handmade/Weddings/Accessories", 380:"Handmade/Weddings/Bags and Purses", 381:"Handmade/Weddings/Bouquets", 382:"Handmade/Weddings/Cake Toppers", 383:"Handmade/Weddings/Candles", 384:"Handmade/Weddings/Cards", 385:"Handmade/Weddings/Clothing", 386:"Handmade/Weddings/Decorations", 387:"Handmade/Weddings/Favors", 388:"Handmade/Weddings/Frames", 389:"Handmade/Weddings/Guest Books", 390:"Handmade/Weddings/Invitations", 391:"Handmade/Weddings/Jewelry", 392:"Handmade/Weddings/Just Married", 393:"Handmade/Weddings/Men", 394:"Handmade/Weddings/Other", 395:"Handmade/Weddings/Pillows", 396:"Handmade/Weddings/Something Blue", 397:"Handmade/Woodworking/Accessories", 398:"Handmade/Woodworking/Boxes", 399:"Handmade/Woodworking/Burning", 400:"Handmade/Woodworking/Carving", 401:"Handmade/Woodworking/Home Decor", 402:"Handmade/Woodworking/Jewelry", 403:"Handmade/Woodworking/Other", 404:"Handmade/Woodworking/Outdoor", 405:"Handmade/Woodworking/Sculptures", 406:"Handmade/Woodworking/Seasonal", 407:"Handmade/Woodworking/Signs", 408:"Handmade/Woodworking/Sports", 409:"Handmade/Woodworking/Supplies", 410:"Handmade/Woodworking/Toys", 411:"Home/Artwork/Drawings", 412:"Home/Artwork/Lithographs, Etchings & Woodcuts", 413:"Home/Artwork/Other", 414:"Home/Artwork/Paintings", 415:"Home/Artwork/Photographs", 416:"Home/Artwork/Posters & Prints", 417:"Home/Bath/Bath Linen Sets", 418:"Home/Bath/Bath Rugs", 419:"Home/Bath/Bathroom Accessories", 420:"Home/Bath/Bathroom Furniture Sets", 421:"Home/Bath/Bathroom Shelves", 422:"Home/Bath/Other", 423:"Home/Bath/Towels", 424:"Home/Bedding/Bed Pillows", 425:"Home/Bedding/Bed in a Bag", 426:"Home/Bedding/Bedspreads & Coverlets", 427:"Home/Bedding/Blankets & Throws", 428:"Home/Bedding/Comforters & Sets", 429:"Home/Bedding/Decorative Pillows, Inserts & Covers", 430:"Home/Bedding/Duvet Covers & Sets", 431:"Home/Bedding/Feather Beds", 432:"Home/Bedding/Inflatable Beds", 433:"Home/Bedding/Mattress Pads", 434:"Home/Bedding/Other", 435:"Home/Bedding/Quilts", 436:"Home/Bedding/Shams, Bed Skirts & Bed Frame Draperies", 437:"Home/Bedding/Sheets & Pillowcases", 438:"Home/Cleaning Supplies/Air Fresheners", 439:"Home/Cleaning Supplies/Brushes", 440:"Home/Cleaning Supplies/Dusting", 441:"Home/Cleaning Supplies/Gloves", 442:"Home/Cleaning Supplies/Household Cleaners", 443:"Home/Cleaning Supplies/Jewelry Boxes & Organizers", 444:"Home/Cleaning Supplies/Mopping", 445:"Home/Cleaning Supplies/Other", 446:"Home/Cleaning Supplies/Paper Towels", 447:"Home/Cleaning Supplies/Sponges", 448:"Home/Cleaning Supplies/Sweeping", 449:"Home/Cleaning Supplies/Trash Bags", 450:"Home/Furniture/Bathroom Furniture", 451:"Home/Furniture/Bedroom Furniture", 452:"Home/Furniture/Home Bar Furniture", 453:"Home/Furniture/Home Entertainment Furniture", 454:"Home/Furniture/Home Office Furniture", 455:"Home/Furniture/Living Room Furniture", 456:"Home/Furniture/Other", 457:"Home/Furniture/Other Furniture", 458:"Home/Furniture/Replacement Parts", 459:"Home/Home Appliances/Air Conditioners", 460:"Home/Home Appliances/Air Purifiers", 461:"Home/Home Appliances/Dehumidifiers", 462:"Home/Home Appliances/Dishwashers", 463:"Home/Home Appliances/Fans", 464:"Home/Home Appliances/Freezers & Ice Makers", 465:"Home/Home Appliances/Garbage Disposals", 466:"Home/Home Appliances/Garment Steamers", 467:"Home/Home Appliances/Humidifiers", 468:"Home/Home Appliances/Irons & Ironing Boards", 469:"Home/Home Appliances/Kitchen Appliances", 470:"Home/Home Appliances/Microwaves", 471:"Home/Home Appliances/Refrigerators", 472:"Home/Home Appliances/Space Heaters", 473:"Home/Home Appliances/Vacuums & Floor Care", 474:"Home/Home Appliances/Washers & Dryers", 475:"Home/Home Appliances/Water Coolers & Filters", 476:"Home/Home Appliances/Wine, Beer & Beverage Coolers", 477:"Home/Home Décor/Area Rugs & Pads", 478:"Home/Home Décor/Baskets", 479:"Home/Home Décor/Candles & Holders", 480:"Home/Home Décor/Clocks", 481:"Home/Home Décor/Decorative Pillows", 482:"Home/Home Décor/Doormats", 483:"Home/Home Décor/Doorstops", 484:"Home/Home Décor/Draft Stoppers", 485:"Home/Home Décor/Fireplaces & Accessories", 486:"Home/Home Décor/Home Décor Accents", 487:"Home/Home Décor/Home Fragrance", 488:"Home/Home Décor/Lamps& Accessories", 489:"Home/Home Décor/Mirrors", 490:"Home/Home Décor/Other", 491:"Home/Home Décor/Photo Albums & Frames", 492:"Home/Home Décor/Slipcovers", 493:"Home/Home Décor/Tapestries", 494:"Home/Home Décor/Vases", 495:"Home/Home Décor/Window Treatments", 496:"Home/Kids' Home Store/Kids' Bath", 497:"Home/Kids' Home Store/Kids' Bedding", 498:"Home/Kids' Home Store/Kids' Flatware", 499:"Home/Kids' Home Store/Kids' Furniture", 500:"Home/Kids' Home Store/Kids' Room Décor", 501:"Home/Kids' Home Store/Nursery Bedding", 502:"Home/Kids' Home Store/Nursery Décor", 503:"Home/Kids' Home Store/Nursery Furniture", 504:"Home/Kids' Home Store/Other", 505:"Home/Kitchen & Dining/Bakeware", 506:"Home/Kitchen & Dining/Coffee & Tea Accessories", 507:"Home/Kitchen & Dining/Cookware", 508:"Home/Kitchen & Dining/Dining & Entertaining", 509:"Home/Kitchen & Dining/Food Service Equipment & Supplies", 510:"Home/Kitchen & Dining/Home Brewing & Wine Making", 511:"Home/Kitchen & Dining/Kitchen & Table Linens", 512:"Home/Kitchen & Dining/Kitchen Knives & Cutlery Accessories", 513:"Home/Kitchen & Dining/Kitchen Utensils & Gadgets", 514:"Home/Kitchen & Dining/Other", 515:"Home/Kitchen & Dining/Small Appliances", 516:"Home/Kitchen & Dining/Storage & Organization", 517:"Home/Kitchen & Dining/Water Coolers & Filters", 518:"Home/Kitchen & Dining/Wine Accessories", 519:"Home/Other/Other", 520:"Home/Seasonal Décor/Birthday", 521:"Home/Seasonal Décor/Christmas", 522:"Home/Seasonal Décor/Day of the Dead", 523:"Home/Seasonal Décor/Easter", 524:"Home/Seasonal Décor/Halloween", 525:"Home/Seasonal Décor/Hanukkah", 526:"Home/Seasonal Décor/New Year's", 527:"Home/Seasonal Décor/Other", 528:"Home/Seasonal Décor/Patriotic", 529:"Home/Seasonal Décor/St Patrick's", 530:"Home/Seasonal Décor/Thanksgiving", 531:"Home/Seasonal Décor/Valentine", 532:"Home/Storage & Organization/Baskets & Bins", 533:"Home/Storage & Organization/Bathroom Storage & Organization", 534:"Home/Storage & Organization/Clothing & Closet Storage", 535:"Home/Storage & Organization/Garage Storage & Organization", 536:"Home/Storage & Organization/Holiday Décor Storage", 537:"Home/Storage & Organization/Jewelry Boxes & Organizers", 538:"Home/Storage & Organization/Kitchen Storage & Organization", 539:"Home/Storage & Organization/Laundry Storage & Organization", 540:"Home/Storage & Organization/Other", 541:"Home/Storage & Organization/Racks, Shelves & Drawers", 542:"Home/Storage & Organization/Storage Cabinets", 543:"Home/Storage & Organization/Trash & Recycling", 544:"Kids/Bathing & Skin Care/Bathing Tubs & Seats", 545:"Kids/Bathing & Skin Care/Bathroom Safety", 546:"Kids/Bathing & Skin Care/Bubble Bath", 547:"Kids/Bathing & Skin Care/Conditioners", 548:"Kids/Bathing & Skin Care/Grooming & Healthcare Kits", 549:"Kids/Bathing & Skin Care/Non-Slip Bath Mats", 550:"Kids/Bathing & Skin Care/Other", 551:"Kids/Bathing & Skin Care/Shampoo", 552:"Kids/Bathing & Skin Care/Skin Care", 553:"Kids/Bathing & Skin Care/Soaps & Cleansers", 554:"Kids/Bathing & Skin Care/Travel Bathing Kits", 555:"Kids/Bathing & Skin Care/Washcloths & Towels", 556:"Kids/Boys (4+)/Accessories", 557:"Kids/Boys (4+)/Bottoms", 558:"Kids/Boys (4+)/Coats & Jackets", 559:"Kids/Boys (4+)/Other", 560:"Kids/Boys (4+)/Shoes", 561:"Kids/Boys (4+)/Swimwear", 562:"Kids/Boys (4+)/Top & T-shirts", 563:"Kids/Boys 0-24 Mos/Accessories", 564:"Kids/Boys 0-24 Mos/Bottoms", 565:"Kids/Boys 0-24 Mos/Coats & Jackets", 566:"Kids/Boys 0-24 Mos/One-Pieces", 567:"Kids/Boys 0-24 Mos/Other", 568:"Kids/Boys 0-24 Mos/Shoes", 569:"Kids/Boys 0-24 Mos/Swimwear", 570:"Kids/Boys 0-24 Mos/Tops & T-Shirts", 571:"Kids/Boys 2T-5T/Accessories", 572:"Kids/Boys 2T-5T/Bottoms", 573:"Kids/Boys 2T-5T/Coats & Jackets", 574:"Kids/Boys 2T-5T/One-Pieces", 575:"Kids/Boys 2T-5T/Other", 576:"Kids/Boys 2T-5T/Shoes", 577:"Kids/Boys 2T-5T/Swimwear", 578:"Kids/Boys 2T-5T/Tops & T-Shirts", 579:"Kids/Car Seats & Accessories/Accessories", 580:"Kids/Car Seats & Accessories/Car Seats", 581:"Kids/Car Seats & Accessories/Other", 582:"Kids/Diapering/Changing Kits", 583:"Kids/Diapering/Changing Pads & Covers", 584:"Kids/Diapering/Cloth Diapers", 585:"Kids/Diapering/Diaper Bags", 586:"Kids/Diapering/Diaper Pails & Refills", 587:"Kids/Diapering/Diaper Stackers & Caddies", 588:"Kids/Diapering/Disposable Diapers", 589:"Kids/Diapering/Other", 590:"Kids/Diapering/Powders & Lotions", 591:"Kids/Diapering/Washcloths & Towels", 592:"Kids/Diapering/Wipes & Holders", 593:"Kids/Feeding/Bottle-Feeding", 594:"Kids/Feeding/Breastfeeding", 595:"Kids/Feeding/Gift Sets", 596:"Kids/Feeding/Highchairs & Booster Seats", 597:"Kids/Feeding/Other", 598:"Kids/Feeding/Pacifiers & Accessories", 599:"Kids/Feeding/Pillows & Stools", 600:"Kids/Feeding/Storage & Containers", 601:"Kids/Gear/Activity Centers & Entertainers", 602:"Kids/Gear/Baby Gyms & Playmats", 603:"Kids/Gear/Baby Seats", 604:"Kids/Gear/Backpacks & Carriers", 605:"Kids/Gear/Bicycle Child Seats & Trailers", 606:"Kids/Gear/Harnesses & Leashes", 607:"Kids/Gear/Other", 608:"Kids/Gear/Playard Bedding", 609:"Kids/Gear/Playards", 610:"Kids/Gear/Shopping Cart Covers", 611:"Kids/Gear/Swings, Jumpers & Bouncers", 612:"Kids/Gear/Travel Beds", 613:"Kids/Gear/Walkers", 614:"Kids/Girls (4+)/Accessories", 615:"Kids/Girls (4+)/Coats & Jackets", 616:"Kids/Girls (4+)/Dresses", 617:"Kids/Girls (4+)/Jeans", 618:"Kids/Girls (4+)/Other", 619:"Kids/Girls (4+)/Shoes", 620:"Kids/Girls (4+)/Swimwear", 621:"Kids/Girls (4+)/Tops & T-Shirts", 622:"Kids/Girls 0-24 Mos/Accessories", 623:"Kids/Girls 0-24 Mos/Bottoms", 624:"Kids/Girls 0-24 Mos/Coats & Jackets", 625:"Kids/Girls 0-24 Mos/Dresses", 626:"Kids/Girls 0-24 Mos/One-Pieces", 627:"Kids/Girls 0-24 Mos/Other", 628:"Kids/Girls 0-24 Mos/Shoes", 629:"Kids/Girls 0-24 Mos/Swimwear", 630:"Kids/Girls 0-24 Mos/Tops & T-Shirts", 631:"Kids/Girls 2T-5T/Accessories", 632:"Kids/Girls 2T-5T/Bottoms", 633:"Kids/Girls 2T-5T/Coats & Jackets", 634:"Kids/Girls 2T-5T/Dresses", 635:"Kids/Girls 2T-5T/One-Pieces", 636:"Kids/Girls 2T-5T/Other", 637:"Kids/Girls 2T-5T/Shoes", 638:"Kids/Girls 2T-5T/Swimwear", 639:"Kids/Girls 2T-5T/Tops & T-Shirts", 640:"Kids/Health & Baby Care/Humidifiers & Vaporizers", 641:"Kids/Health & Baby Care/Nail Care", 642:"Kids/Health & Baby Care/Nasal Aspirators", 643:"Kids/Health & Baby Care/Other", 644:"Kids/Health & Baby Care/Sun Protection", 645:"Kids/Health & Baby Care/Teethers", 646:"Kids/Health & Baby Care/Teething Relief", 647:"Kids/Health & Baby Care/Thermometers", 648:"Kids/Health & Baby Care/Toothbrushes", 649:"Kids/Nursery/Bedding", 650:"Kids/Nursery/Furniture", 651:"Kids/Nursery/Nursery Décor", 652:"Kids/Nursery/Other", 653:"Kids/Other/Other", 654:"Kids/Potty Training/Other", 655:"Kids/Potty Training/Potties & Seats", 656:"Kids/Potty Training/Seat Covers", 657:"Kids/Potty Training/Step Stools", 658:"Kids/Potty Training/Training Pants", 659:"Kids/Pregnancy & Maternity/Breastfeeding Pillows & Stools", 660:"Kids/Pregnancy & Maternity/Family Planning Tests", 661:"Kids/Pregnancy & Maternity/Maternity Pillows", 662:"Kids/Pregnancy & Maternity/Other", 663:"Kids/Pregnancy & Maternity/Prenatal Monitoring Devices", 664:"Kids/Safety/Bathroom Safety", 665:"Kids/Safety/Cabinet Locks & Straps", 666:"Kids/Safety/Crib Netting", 667:"Kids/Safety/Edge & Corner Guards", 668:"Kids/Safety/Electrical Safety", 669:"Kids/Safety/Gates & Doorways", 670:"Kids/Safety/Harnesses & Leashes", 671:"Kids/Safety/Kitchen Safety", 672:"Kids/Safety/Monitors", 673:"Kids/Safety/Other", 674:"Kids/Safety/Outdoor Safety", 675:"Kids/Safety/Safety Caps", 676:"Kids/Safety/Sleep Positioners", 677:"Kids/Strollers/Accessories", 678:"Kids/Strollers/Joggers", 679:"Kids/Strollers/Lightweight", 680:"Kids/Strollers/Other", 681:"Kids/Strollers/Prams", 682:"Kids/Strollers/Standard", 683:"Kids/Strollers/Travel Systems", 684:"Kids/Toys/Action Figures & Statues", 685:"Kids/Toys/Arts & Crafts", 686:"Kids/Toys/Baby & Toddler Toys", 687:"Kids/Toys/Building Toys", 688:"Kids/Toys/Dolls & Accessories", 689:"Kids/Toys/Dress Up & Pretend Play", 690:"Kids/Toys/Electronics for Kids", 691:"Kids/Toys/Games", 692:"Kids/Toys/Hobbies", 693:"Kids/Toys/Kids' Furniture, Décor & Storage", 694:"Kids/Toys/Learning & Education", 695:"Kids/Toys/Novelty & Gag Toys", 696:"Kids/Toys/Other", 697:"Kids/Toys/Party Supplies", 698:"Kids/Toys/Puzzles", 699:"Kids/Toys/Sports & Outdoor Play", 700:"Kids/Toys/Stuffed Animals & Plush", 701:"Kids/Toys/Toy Remote Control & Play Vehicles", 702:"Kids/Toys/Tricycles, Scooters & Wagons", 703:"Men/Athletic Apparel/Competitive Swimwear", 704:"Men/Athletic Apparel/Jackets", 705:"Men/Athletic Apparel/Jerseys", 706:"Men/Athletic Apparel/Other", 707:"Men/Athletic Apparel/Pants", 708:"Men/Athletic Apparel/Shirts & Tops", 709:"Men/Athletic Apparel/Shorts", 710:"Men/Athletic Apparel/Snowsuits & Bibs", 711:"Men/Athletic Apparel/Socks", 712:"Men/Athletic Apparel/Tracksuits & Sweats", 713:"Men/Athletic Apparel/Vests", 714:"Men/Blazers & Sport Coats/Double Breasted", 715:"Men/Blazers & Sport Coats/One Button", 716:"Men/Blazers & Sport Coats/Other", 717:"Men/Blazers & Sport Coats/Three Button", 718:"Men/Blazers & Sport Coats/Two Button", 719:"Men/Coats & Jackets/Fleece Jacket", 720:"Men/Coats & Jackets/Flight/Bomber", 721:"Men/Coats & Jackets/Jean Jacket", 722:"Men/Coats & Jackets/Military", 723:"Men/Coats & Jackets/Motorcycle", 724:"Men/Coats & Jackets/Other", 725:"Men/Coats & Jackets/Parka", 726:"Men/Coats & Jackets/Peacoat", 727:"Men/Coats & Jackets/Poncho", 728:"Men/Coats & Jackets/Puffer", 729:"Men/Coats & Jackets/Rainwear", 730:"Men/Coats & Jackets/Trench", 731:"Men/Coats & Jackets/Varsity/Baseball", 732:"Men/Coats & Jackets/Vest", 733:"Men/Coats & Jackets/Windbreaker", 734:"Men/Coats & Jackets/Wool", 735:"Men/Jeans/Baggy, Loose", 736:"Men/Jeans/Boot Cut", 737:"Men/Jeans/Cargo", 738:"Men/Jeans/Carpenter", 739:"Men/Jeans/Classic, Straight Leg", 740:"Men/Jeans/Other", 741:"Men/Jeans/Overalls", 742:"Men/Jeans/Relaxed", 743:"Men/Jeans/Slim, Skinny", 744:"Men/Men's Accessories/Backpacks, Bags & Briefcases", 745:"Men/Men's Accessories/Belts", 746:"Men/Men's Accessories/Hats", 747:"Men/Men's Accessories/Other", 748:"Men/Men's Accessories/Sunglasses", 749:"Men/Men's Accessories/Ties", 750:"Men/Men's Accessories/Wallets", 751:"Men/Men's Accessories/Watches", 752:"Men/Other/Other", 753:"Men/Pants/Cargo", 754:"Men/Pants/Carpenter", 755:"Men/Pants/Casual Pants", 756:"Men/Pants/Corduroys", 757:"Men/Pants/Dress - Flat Front", 758:"Men/Pants/Dress - Pleat", 759:"Men/Pants/Khakis, Chinos", 760:"Men/Pants/Other", 761:"Men/Shoes/Athletic", 762:"Men/Shoes/Boots", 763:"Men/Shoes/Fashion Sneakers", 764:"Men/Shoes/Loafers & Slip-Ons", 765:"Men/Shoes/Mules & Clogs", 766:"Men/Shoes/Outdoor", 767:"Men/Shoes/Oxfords", 768:"Men/Shoes/Sandals", 769:"Men/Shoes/Slippers", 770:"Men/Shoes/Work & Safety", 771:"Men/Shorts/Athletic", 772:"Men/Shorts/Board, Surf", 773:"Men/Shorts/Cargo", 774:"Men/Shorts/Carpenter, Utility", 775:"Men/Shorts/Casual Shorts", 776:"Men/Shorts/Denim", 777:"Men/Shorts/Dress Shorts", 778:"Men/Shorts/Khakis, Chinos", 779:"Men/Shorts/Other", 780:"Men/Suits/Four Button", 781:"Men/Suits/One Button", 782:"Men/Suits/Other", 783:"Men/Suits/Three Button", 784:"Men/Suits/Tuxedo", 785:"Men/Suits/Two Button", 786:"Men/Sweaters/Cardigan", 787:"Men/Sweaters/Crewneck", 788:"Men/Sweaters/Full Zip", 789:"Men/Sweaters/Other", 790:"Men/Sweaters/Polo", 791:"Men/Sweaters/Turtleneck", 792:"Men/Sweaters/V-Neck", 793:"Men/Sweaters/Vest", 794:"Men/Sweats & Hoodies/Hoodie", 795:"Men/Sweats & Hoodies/Other", 796:"Men/Sweats & Hoodies/Sweatshirt, Pullover", 797:"Men/Sweats & Hoodies/Track & Sweat Pants", 798:"Men/Sweats & Hoodies/Track & Sweat Suits", 799:"Men/Sweats & Hoodies/Track Jacket", 800:"Men/Swimwear/Board Shorts", 801:"Men/Swimwear/Swim Briefs", 802:"Men/Swimwear/Swim Trunks", 803:"Men/Tops/Button-Front", 804:"Men/Tops/Dress Shirts", 805:"Men/Tops/Hawaiian", 806:"Men/Tops/Henley", 807:"Men/Tops/Other", 808:"Men/Tops/Polo, Rugby", 809:"Men/Tops/T-shirts", 810:"Men/Tops/Tank", 811:"Men/Tops/Turtleneck", 812:"Other/Automotive/Automotive Enthusiast Merchandise", 813:"Other/Automotive/Car Care", 814:"Other/Automotive/Car Electronics & Accessories", 815:"Other/Automotive/Exterior Accessories", 816:"Other/Automotive/Interior Accessories", 817:"Other/Automotive/Lights & Lighting Accessories", 818:"Other/Automotive/Motorcycle & Powersports", 819:"Other/Automotive/Oils & Fluids", 820:"Other/Automotive/Other", 821:"Other/Automotive/Paint, Body & Trim", 822:"Other/Automotive/Performance Parts & Accessories", 823:"Other/Automotive/RV Parts & Accessories", 824:"Other/Automotive/Replacement Parts", 825:"Other/Automotive/Tires & Wheels", 826:"Other/Automotive/Tools & Equipment", 827:"Other/Books/Arts & Photography", 828:"Other/Books/Biographies & Memoirs", 829:"Other/Books/Business & Money", 830:"Other/Books/Children's Books", 831:"Other/Books/Christian Books & Bibles", 832:"Other/Books/Education & Teaching", 833:"Other/Books/Engineering & Transportation", 834:"Other/Books/History", 835:"Other/Books/Literature & Fiction", 836:"Other/Books/Medical Books", 837:"Other/Books/Other", 838:"Other/Books/Politics & Social Sciences", 839:"Other/Books/Reference", 840:"Other/Books/Religion & Spirituality", 841:"Other/Books/Science & Math", 842:"Other/Daily & Travel items/Baby & Child Care", 843:"Other/Daily & Travel items/Health Care", 844:"Other/Daily & Travel items/Household Supplies", 845:"Other/Daily & Travel items/Medical Supplies & Equipment", 846:"Other/Daily & Travel items/Other", 847:"Other/Daily & Travel items/Personal Care", 848:"Other/Daily & Travel items/Sports Nutrition", 849:"Other/Daily & Travel items/Stationery & Party Supplies", 850:"Other/Magazines/International Publications", 851:"Other/Magazines/Lifestyle & Cultures", 852:"Other/Magazines/Other", 853:"Other/Musical instruments/Amplifiers & Effects", 854:"Other/Musical instruments/Band & Orchestra", 855:"Other/Musical instruments/Bass Guitars", 856:"Other/Musical instruments/Brass Instruments", 857:"Other/Musical instruments/DJ, Electronic Music & Karaoke", 858:"Other/Musical instruments/Drums & Percussion", 859:"Other/Musical instruments/Guitars", 860:"Other/Musical instruments/Instrument Accessories", 861:"Other/Musical instruments/Keyboards", 862:"Other/Musical instruments/Live Sound & Stage", 863:"Other/Musical instruments/Microphones & Accessories", 864:"Other/Musical instruments/Other", 865:"Other/Musical instruments/Stringed Instruments", 866:"Other/Musical instruments/Studio Recording Equipment", 867:"Other/Musical instruments/Wind & Woodwind Instruments", 868:"Other/Office supplies/Basic Supplies", 869:"Other/Office supplies/Calendars", 870:"Other/Office supplies/Electronics", 871:"Other/Office supplies/Furniture", 872:"Other/Office supplies/Ink & Toner", 873:"Other/Office supplies/Organization", 874:"Other/Office supplies/Other", 875:"Other/Office supplies/Paper", 876:"Other/Office supplies/Presentation", 877:"Other/Office supplies/School Supplies", 878:"Other/Office supplies/Shipping Supplies", 879:"Other/Office supplies/Writing", 880:"Other/Other/Other", 881:"Other/Pet Supplies/Cage", 882:"Other/Pet Supplies/Dogs", 883:"Other/Pet Supplies/Others", 884:"Other/Pet Supplies/Pet Food", 885:"Sports & Outdoors/Apparel/Accessories", 886:"Sports & Outdoors/Apparel/Boys", 887:"Sports & Outdoors/Apparel/Girls", 888:"Sports & Outdoors/Apparel/Men", 889:"Sports & Outdoors/Apparel/Other", 890:"Sports & Outdoors/Apparel/Women", 891:"Sports & Outdoors/Exercise/Athletic Training", 892:"Sports & Outdoors/Exercise/Bowling", 893:"Sports & Outdoors/Exercise/Boxing & MMA", 894:"Sports & Outdoors/Exercise/Dance/Ballet", 895:"Sports & Outdoors/Exercise/Fitness accessories", 896:"Sports & Outdoors/Exercise/Fitness technology", 897:"Sports & Outdoors/Exercise/Other", 898:"Sports & Outdoors/Exercise/Strength training", 899:"Sports & Outdoors/Exercise/Track & Field", 900:"Sports & Outdoors/Exercise/Yoga & Pilates", 901:"Sports & Outdoors/Fan Shop/MLB", 902:"Sports & Outdoors/Fan Shop/NBA", 903:"Sports & Outdoors/Fan Shop/NCAA", 904:"Sports & Outdoors/Fan Shop/NFL", 905:"Sports & Outdoors/Fan Shop/NHL", 906:"Sports & Outdoors/Fan Shop/Other", 907:"Sports & Outdoors/Footwear/Cleats", 908:"Sports & Outdoors/Footwear/Kids", 909:"Sports & Outdoors/Footwear/Men", 910:"Sports & Outdoors/Footwear/Other", 911:"Sports & Outdoors/Footwear/Women", 912:"Sports & Outdoors/Golf/Electronics", 913:"Sports & Outdoors/Golf/Golf Apparel", 914:"Sports & Outdoors/Golf/Golf Bags", 915:"Sports & Outdoors/Golf/Golf Balls", 916:"Sports & Outdoors/Golf/Golf Shoes", 917:"Sports & Outdoors/Golf/Men's Golf Clubs", 918:"Sports & Outdoors/Golf/Other", 919:"Sports & Outdoors/Golf/Women's Golf Clubs", 920:"Sports & Outdoors/Other/Other", 921:"Sports & Outdoors/Outdoors/Bike & Skate", 922:"Sports & Outdoors/Outdoors/Boating", 923:"Sports & Outdoors/Outdoors/Fishing", 924:"Sports & Outdoors/Outdoors/Hiking & Camping", 925:"Sports & Outdoors/Outdoors/Indoor/Outdoor Games", 926:"Sports & Outdoors/Outdoors/Other", 927:"Sports & Outdoors/Outdoors/Skateboard", 928:"Sports & Outdoors/Outdoors/Snowboard", 929:"Sports & Outdoors/Outdoors/Water Sports", 930:"Sports & Outdoors/Team Sports/All Other Sports", 931:"Sports & Outdoors/Team Sports/Badminton", 932:"Sports & Outdoors/Team Sports/Baseball & Softball", 933:"Sports & Outdoors/Team Sports/Basketball", 934:"Sports & Outdoors/Team Sports/Football", 935:"Sports & Outdoors/Team Sports/Hockey", 936:"Sports & Outdoors/Team Sports/Lacrosse", 937:"Sports & Outdoors/Team Sports/Soccer", 938:"Sports & Outdoors/Team Sports/Tennis & Racquets", 939:"Sports & Outdoors/Team Sports/Volleyball", 940:"Vintage & Collectibles/Accessories/Apron", 941:"Vintage & Collectibles/Accessories/Belt", 942:"Vintage & Collectibles/Accessories/Brooch", 943:"Vintage & Collectibles/Accessories/Buckle", 944:"Vintage & Collectibles/Accessories/Compact", 945:"Vintage & Collectibles/Accessories/Cuff Links", 946:"Vintage & Collectibles/Accessories/Eyewear", 947:"Vintage & Collectibles/Accessories/Gloves", 948:"Vintage & Collectibles/Accessories/Handkerchief", 949:"Vintage & Collectibles/Accessories/Hat", 950:"Vintage & Collectibles/Accessories/Keychain", 951:"Vintage & Collectibles/Accessories/Necktie", 952:"Vintage & Collectibles/Accessories/Other", 953:"Vintage & Collectibles/Accessories/Scarf", 954:"Vintage & Collectibles/Accessories/Shoes", 955:"Vintage & Collectibles/Accessories/Wallet", 956:"Vintage & Collectibles/Antique/100 Years or Older", 957:"Vintage & Collectibles/Antique/50 To 75 Years", 958:"Vintage & Collectibles/Antique/75 To 100 Years", 959:"Vintage & Collectibles/Antique/Accessories", 960:"Vintage & Collectibles/Antique/Bags and Purses", 961:"Vintage & Collectibles/Antique/Book", 962:"Vintage & Collectibles/Antique/Collectibles", 963:"Vintage & Collectibles/Antique/Electronics", 964:"Vintage & Collectibles/Antique/Furniture", 965:"Vintage & Collectibles/Antique/Home Decor", 966:"Vintage & Collectibles/Antique/Housewares", 967:"Vintage & Collectibles/Antique/Jewelry", 968:"Vintage & Collectibles/Antique/Other", 969:"Vintage & Collectibles/Antique/Paper Ephemera", 970:"Vintage & Collectibles/Antique/Serving", 971:"Vintage & Collectibles/Antique/Supplies", 972:"Vintage & Collectibles/Antique/Toy", 973:"Vintage & Collectibles/Bags and Purses/Case", 974:"Vintage & Collectibles/Bags and Purses/Change Purse", 975:"Vintage & Collectibles/Bags and Purses/Clutch", 976:"Vintage & Collectibles/Bags and Purses/Diaper Bag", 977:"Vintage & Collectibles/Bags and Purses/Formal", 978:"Vintage & Collectibles/Bags and Purses/Handbag", 979:"Vintage & Collectibles/Bags and Purses/Leather", 980:"Vintage & Collectibles/Bags and Purses/Luggage", 981:"Vintage & Collectibles/Bags and Purses/Other", 982:"Vintage & Collectibles/Bags and Purses/Pouch", 983:"Vintage & Collectibles/Bags and Purses/Purse", 984:"Vintage & Collectibles/Bags and Purses/Tote", 985:"Vintage & Collectibles/Bags and Purses/Wallet", 986:"Vintage & Collectibles/Book/Art", 987:"Vintage & Collectibles/Book/Biography", 988:"Vintage & Collectibles/Book/Children", 989:"Vintage & Collectibles/Book/Comics", 990:"Vintage & Collectibles/Book/Cookbook", 991:"Vintage & Collectibles/Book/Crafting", 992:"Vintage & Collectibles/Book/Decorating", 993:"Vintage & Collectibles/Book/Fiction", 994:"Vintage & Collectibles/Book/Health", 995:"Vintage & Collectibles/Book/History", 996:"Vintage & Collectibles/Book/How to", 997:"Vintage & Collectibles/Book/Illustrated", 998:"Vintage & Collectibles/Book/Instructional", 999:"Vintage & Collectibles/Book/Nonfiction", 1000:"Vintage & Collectibles/Book/Other", 1001:"Vintage & Collectibles/Book/Poetry", 1002:"Vintage & Collectibles/Book/Reference", 1003:"Vintage & Collectibles/Book/Religion", 1004:"Vintage & Collectibles/Book/Science", 1005:"Vintage & Collectibles/Book/Scifi", 1006:"Vintage & Collectibles/Clothing/Baby", 1007:"Vintage & Collectibles/Clothing/Blouse", 1008:"Vintage & Collectibles/Clothing/Children", 1009:"Vintage & Collectibles/Clothing/Corset", 1010:"Vintage & Collectibles/Clothing/Dress", 1011:"Vintage & Collectibles/Clothing/Jacket", 1012:"Vintage & Collectibles/Clothing/Other", 1013:"Vintage & Collectibles/Clothing/Outerwear", 1014:"Vintage & Collectibles/Clothing/Pants", 1015:"Vintage & Collectibles/Clothing/Shawl", 1016:"Vintage & Collectibles/Clothing/Shirt", 1017:"Vintage & Collectibles/Clothing/Shorts", 1018:"Vintage & Collectibles/Clothing/Skirt", 1019:"Vintage & Collectibles/Clothing/Sweater", 1020:"Vintage & Collectibles/Clothing/Swimwear", 1021:"Vintage & Collectibles/Clothing/Tank", 1022:"Vintage & Collectibles/Clothing/Tshirt", 1023:"Vintage & Collectibles/Collectibles/Doll", 1024:"Vintage & Collectibles/Collectibles/Figurine", 1025:"Vintage & Collectibles/Collectibles/Glass", 1026:"Vintage & Collectibles/Collectibles/Other", 1027:"Vintage & Collectibles/Collectibles/Porcelain", 1028:"Vintage & Collectibles/Collectibles/Souvenir", 1029:"Vintage & Collectibles/Electronics/Camera", 1030:"Vintage & Collectibles/Electronics/Clock", 1031:"Vintage & Collectibles/Electronics/Game", 1032:"Vintage & Collectibles/Electronics/Other", 1033:"Vintage & Collectibles/Electronics/Radio", 1034:"Vintage & Collectibles/Electronics/Telephone", 1035:"Vintage & Collectibles/Electronics/Television", 1036:"Vintage & Collectibles/Electronics/Video Game", 1037:"Vintage & Collectibles/Furniture/Chair", 1038:"Vintage & Collectibles/Furniture/Entertainment", 1039:"Vintage & Collectibles/Furniture/Fixture", 1040:"Vintage & Collectibles/Furniture/Other", 1041:"Vintage & Collectibles/Furniture/Shelf", 1042:"Vintage & Collectibles/Furniture/Storage", 1043:"Vintage & Collectibles/Home Decor/Basket", 1044:"Vintage & Collectibles/Home Decor/Bedding", 1045:"Vintage & Collectibles/Home Decor/Box", 1046:"Vintage & Collectibles/Home Decor/Candle Holder", 1047:"Vintage & Collectibles/Home Decor/Frame", 1048:"Vintage & Collectibles/Home Decor/Lighting", 1049:"Vintage & Collectibles/Home Decor/Linens", 1050:"Vintage & Collectibles/Home Decor/Other", 1051:"Vintage & Collectibles/Home Decor/Pillow", 1052:"Vintage & Collectibles/Home Decor/Planter", 1053:"Vintage & Collectibles/Home Decor/Tray", 1054:"Vintage & Collectibles/Home Decor/Vase", 1055:"Vintage & Collectibles/Home Decor/Wall Hanging", 1056:"Vintage & Collectibles/Housewares/Bowl", 1057:"Vintage & Collectibles/Housewares/Box", 1058:"Vintage & Collectibles/Housewares/Ceramic", 1059:"Vintage & Collectibles/Housewares/Coaster", 1060:"Vintage & Collectibles/Housewares/Cup", 1061:"Vintage & Collectibles/Housewares/Glass", 1062:"Vintage & Collectibles/Housewares/Light", 1063:"Vintage & Collectibles/Housewares/Magnet", 1064:"Vintage & Collectibles/Housewares/Other", 1065:"Vintage & Collectibles/Housewares/Plate", 1066:"Vintage & Collectibles/Housewares/Pot Holder", 1067:"Vintage & Collectibles/Housewares/Table", 1068:"Vintage & Collectibles/Housewares/Towel", 1069:"Vintage & Collectibles/Housewares/Tray", 1070:"Vintage & Collectibles/Jewelry/Bracelet", 1071:"Vintage & Collectibles/Jewelry/Brooch", 1072:"Vintage & Collectibles/Jewelry/Earrings", 1073:"Vintage & Collectibles/Jewelry/Necklace", 1074:"Vintage & Collectibles/Jewelry/Other", 1075:"Vintage & Collectibles/Jewelry/Pendant", 1076:"Vintage & Collectibles/Jewelry/Ring", 1077:"Vintage & Collectibles/Jewelry/Watch", 1078:"Vintage & Collectibles/Other/Other", 1079:"Vintage & Collectibles/Paper Ephemera/Advertisement", 1080:"Vintage & Collectibles/Paper Ephemera/Game", 1081:"Vintage & Collectibles/Paper Ephemera/Map", 1082:"Vintage & Collectibles/Paper Ephemera/Other", 1083:"Vintage & Collectibles/Paper Ephemera/Postcard", 1084:"Vintage & Collectibles/Paper Ephemera/Stamps", 1085:"Vintage & Collectibles/Serving/Bowl", 1086:"Vintage & Collectibles/Serving/Butter Dish", 1087:"Vintage & Collectibles/Serving/Casserole", 1088:"Vintage & Collectibles/Serving/Cream and Sugar Set", 1089:"Vintage & Collectibles/Serving/Dinnerware Set", 1090:"Vintage & Collectibles/Serving/Flatware", 1091:"Vintage & Collectibles/Serving/Glassware", 1092:"Vintage & Collectibles/Serving/Mug", 1093:"Vintage & Collectibles/Serving/Other", 1094:"Vintage & Collectibles/Serving/Pitcher", 1095:"Vintage & Collectibles/Serving/Plate", 1096:"Vintage & Collectibles/Serving/Platter", 1097:"Vintage & Collectibles/Serving/Salt and Pepper Shakers", 1098:"Vintage & Collectibles/Serving/Teacup", 1099:"Vintage & Collectibles/Serving/Teapot", 1100:"Vintage & Collectibles/Serving/Tray", 1101:"Vintage & Collectibles/Serving/Tumbler", 1102:"Vintage & Collectibles/Supplies/Bead", 1103:"Vintage & Collectibles/Supplies/Button", 1104:"Vintage & Collectibles/Supplies/Cabochon", 1105:"Vintage & Collectibles/Supplies/Chain", 1106:"Vintage & Collectibles/Supplies/Charm", 1107:"Vintage & Collectibles/Supplies/Ephemera", 1108:"Vintage & Collectibles/Supplies/Fabric", 1109:"Vintage & Collectibles/Supplies/Finding", 1110:"Vintage & Collectibles/Supplies/Other", 1111:"Vintage & Collectibles/Supplies/Pattern", 1112:"Vintage & Collectibles/Supplies/Trim", 1113:"Vintage & Collectibles/Supplies/Yarn", 1114:"Vintage & Collectibles/Supplies/Zipper", 1115:"Vintage & Collectibles/Toy/Action Figure", 1116:"Vintage & Collectibles/Toy/Animal", 1117:"Vintage & Collectibles/Toy/Block", 1118:"Vintage & Collectibles/Toy/Car", 1119:"Vintage & Collectibles/Toy/Children", 1120:"Vintage & Collectibles/Toy/Doll", 1121:"Vintage & Collectibles/Toy/Electronic", 1122:"Vintage & Collectibles/Toy/Game", 1123:"Vintage & Collectibles/Toy/Other", 1124:"Vintage & Collectibles/Toy/Puzzle", 1125:"Vintage & Collectibles/Toy/Sport", 1126:"Vintage & Collectibles/Trading Cards/Action, Adventure", 1127:"Vintage & Collectibles/Trading Cards/Animation", 1128:"Vintage & Collectibles/Trading Cards/Comic", 1129:"Vintage & Collectibles/Trading Cards/Historical, Military", 1130:"Vintage & Collectibles/Trading Cards/Other", 1131:"Vintage & Collectibles/Trading Cards/Price Guides & Publications", 1132:"Vintage & Collectibles/Trading Cards/Sci-Fi, Fantasy", 1133:"Vintage & Collectibles/Trading Cards/Sports", 1134:"Vintage & Collectibles/Trading Cards/Vintage", 1135:"Women/Athletic Apparel/Jackets", 1136:"Women/Athletic Apparel/Jerseys", 1137:"Women/Athletic Apparel/Other", 1138:"Women/Athletic Apparel/Pants, Tights, Leggings", 1139:"Women/Athletic Apparel/Shirts & Tops", 1140:"Women/Athletic Apparel/Shorts", 1141:"Women/Athletic Apparel/Skirts, Skorts & Dresses", 1142:"Women/Athletic Apparel/Snowsuits & Bibs", 1143:"Women/Athletic Apparel/Socks", 1144:"Women/Athletic Apparel/Sports Bras", 1145:"Women/Athletic Apparel/Tracksuits & Sweats", 1146:"Women/Athletic Apparel/Vests", 1147:"Women/Coats & Jackets/Cape", 1148:"Women/Coats & Jackets/Fleece Jacket", 1149:"Women/Coats & Jackets/Jean Jacket", 1150:"Women/Coats & Jackets/Military", 1151:"Women/Coats & Jackets/Motorcycle", 1152:"Women/Coats & Jackets/Other", 1153:"Women/Coats & Jackets/Parka", 1154:"Women/Coats & Jackets/Peacoat", 1155:"Women/Coats & Jackets/Poncho", 1156:"Women/Coats & Jackets/Puffer", 1157:"Women/Coats & Jackets/Raincoat", 1158:"Women/Coats & Jackets/Trench", 1159:"Women/Coats & Jackets/Vest", 1160:"Women/Coats & Jackets/Windbreaker", 1161:"Women/Coats & Jackets/Wool", 1162:"Women/Dresses/Above Knee, Mini", 1163:"Women/Dresses/Asymmetrical Hem", 1164:"Women/Dresses/Full-Length", 1165:"Women/Dresses/Knee-Length", 1166:"Women/Dresses/Mid-Calf", 1167:"Women/Dresses/Other", 1168:"Women/Jeans/Boot Cut", 1169:"Women/Jeans/Boyfriend", 1170:"Women/Jeans/Capri, Cropped", 1171:"Women/Jeans/Cargo", 1172:"Women/Jeans/Flare", 1173:"Women/Jeans/Leggings", 1174:"Women/Jeans/Other", 1175:"Women/Jeans/Overalls", 1176:"Women/Jeans/Relaxed", 1177:"Women/Jeans/Slim, Skinny", 1178:"Women/Jeans/Straight Leg", 1179:"Women/Jeans/Wide Leg", 1180:"Women/Jewelry/Bracelets", 1181:"Women/Jewelry/Earrings", 1182:"Women/Jewelry/Necklaces", 1183:"Women/Jewelry/Rings", 1184:"Women/Maternity/Athletic Apparel", 1185:"Women/Maternity/Coats & Jackets", 1186:"Women/Maternity/Dresses", 1187:"Women/Maternity/Jeans", 1188:"Women/Maternity/Other", 1189:"Women/Maternity/Pants", 1190:"Women/Maternity/Skirts", 1191:"Women/Maternity/Suits & Blazers", 1192:"Women/Maternity/Sweaters", 1193:"Women/Maternity/Tops & Blouses", 1194:"Women/Other/Other", 1195:"Women/Pants/Capris, Cropped", 1196:"Women/Pants/Cargo", 1197:"Women/Pants/Casual Pants", 1198:"Women/Pants/Corduroys", 1199:"Women/Pants/Dress Pants", 1200:"Women/Pants/Khakis, Chinos", 1201:"Women/Pants/Leather", 1202:"Women/Pants/Linen", 1203:"Women/Pants/Other", 1204:"Women/Shoes/Athletic", 1205:"Women/Shoes/Boots", 1206:"Women/Shoes/Fashion Sneakers", 1207:"Women/Shoes/Flats", 1208:"Women/Shoes/Loafers & Slip-Ons", 1209:"Women/Shoes/Mules & Clogs", 1210:"Women/Shoes/Other", 1211:"Women/Shoes/Outdoor", 1212:"Women/Shoes/Oxfords", 1213:"Women/Shoes/Pumps", 1214:"Women/Shoes/Sandals", 1215:"Women/Shoes/Slippers", 1216:"Women/Shoes/Work & Safety", 1217:"Women/Skirts/A-Line", 1218:"Women/Skirts/Asymmetrical", 1219:"Women/Skirts/Bubble", 1220:"Women/Skirts/Full Skirt", 1221:"Women/Skirts/Maxi", 1222:"Women/Skirts/Mini", 1223:"Women/Skirts/Other", 1224:"Women/Skirts/Peasant", 1225:"Women/Skirts/Pleated", 1226:"Women/Skirts/Straight, Pencil", 1227:"Women/Skirts/Tiered", 1228:"Women/Skirts/Wrap", 1229:"Women/Suits & Blazers/Blazer", 1230:"Women/Suits & Blazers/Dress Suit", 1231:"Women/Suits & Blazers/Other", 1232:"Women/Suits & Blazers/Pant Suit", 1233:"Women/Suits & Blazers/Skirt Suit", 1234:"Women/Sweaters/Cardigan", 1235:"Women/Sweaters/Collared", 1236:"Women/Sweaters/Cowl Neck", 1237:"Women/Sweaters/Crewneck", 1238:"Women/Sweaters/Full Zip", 1239:"Women/Sweaters/Henley", 1240:"Women/Sweaters/Hooded", 1241:"Women/Sweaters/Other", 1242:"Women/Sweaters/Poncho", 1243:"Women/Sweaters/Scoop Neck", 1244:"Women/Sweaters/Shrug", 1245:"Women/Sweaters/Sweatercoat", 1246:"Women/Sweaters/Tunic", 1247:"Women/Sweaters/Turtleneck, Mock", 1248:"Women/Sweaters/V-Neck", 1249:"Women/Sweaters/Vest, Sleeveless", 1250:"Women/Sweaters/Wrap", 1251:"Women/Swimwear/Beach Accessories", 1252:"Women/Swimwear/Cover-Ups", 1253:"Women/Swimwear/One-Piece", 1254:"Women/Swimwear/Two-Piece", 1255:"Women/Tops & Blouses/Blouse", 1256:"Women/Tops & Blouses/Button Down Shirt", 1257:"Women/Tops & Blouses/Halter", 1258:"Women/Tops & Blouses/Knit Top", 1259:"Women/Tops & Blouses/Other", 1260:"Women/Tops & Blouses/Polo Shirt", 1261:"Women/Tops & Blouses/T-Shirts", 1262:"Women/Tops & Blouses/Tank, Cami", 1263:"Women/Tops & Blouses/Tunic", 1264:"Women/Tops & Blouses/Turtleneck", 1265:"Women/Tops & Blouses/Wrap", 1266:"Women/Underwear/Bras", 1267:"Women/Underwear/G-Strings & Thongs", 1268:"Women/Underwear/Other", 1269:"Women/Underwear/Panties", 1270:"Women/Underwear/Thermal Underwear", 1271:"Women/Women's Accessories/Belts", 1272:"Women/Women's Accessories/Hair Accessories", 1273:"Women/Women's Accessories/Hats", 1274:"Women/Women's Accessories/Other", 1275:"Women/Women's Accessories/Scarves & Wraps", 1276:"Women/Women's Accessories/Sunglasses", 1277:"Women/Women's Accessories/Wallets", 1278:"Women/Women's Accessories/Watches", 1279:"Women/Women's Handbags/Backpack Style", 1280:"Women/Women's Handbags/Baguette", 1281:"Women/Women's Handbags/Cosmetic Bags", 1282:"Women/Women's Handbags/Hobo", 1283:"Women/Women's Handbags/Messenger & Crossbody", 1284:"Women/Women's Handbags/Other", 1285:"Women/Women's Handbags/Satchel", 1286:"Women/Women's Handbags/Shoulder Bag", 1287:"Women/Women's Handbags/Totes & Shoppers"}

df['category_name'].fillna(0, inplace=True)
df['brand_name'].fillna(0, inplace=True)

print ('i m here1')
a = datetime.now()
for i in range(len(category_name_dict)):
	df['category_name'] = np.where(df['category_name'] == category_name_dict[i + 1] , i + 1, df['category_name'])

b = datetime.now()
(b - a).total_seconds()
#df['category_name'] = df['category_name'].astype(dtype=float)
print ('i m here11')

def pattern_matches(name, pattern):
	if re.search(pattern, str(name)):
		#print 'i m here'
		return True
	else:
		return False
#df1 = pd.read_csv('../input/train.tsv', delimiter='\t',low_memory=True, header = 0)


print ('i m here2')
brand_name_dict = {1:"!iT Jeans", 2:"% Pure", 3:"10.Deep", 4:"191 Unlimited", 5:"1byone", 6:"21men", 7:"24/7 Comfort Apparel", 8:"2K Games", 9:"2XU", 10:"3.1 Phillip Lim", 11:"3M®", 12:"47", 13:"47 Brand", 14:"5.11 Tactical", 15:"525 America", 16:"5th & Ocean", 17:"7 Diamonds", 18:"7 For All Mankind®", 19:"8732 Apparel", 20:"90 Degree By Reflex", 21:"A Bathing Ape", 22:"A Pea In The Pod", 23:"A Plus Child Supply", 24:"A Wish Come True", 25:"A&A Optical", 26:"A&E", 27:"A&R Sports", 28:"A+D", 29:"A-Shirt", 30:"A. Byer", 31:"A.B.S. by Allen Schwartz", 32:"A.D. Sutton & Sons", 33:"A.K.A", 34:"A.L.C.", 35:"A.P.C.", 36:"A/X Armani Exchange", 37:"AA Aquarium", 38:"AB Studio", 39:"ABC Studios", 40:"ABS by Allen Schwartz", 41:"AC/DC", 42:"ACCEL", 43:"ACDelco", 44:"ADAM", 45:"AERIN", 46:"AG Adriano Goldschmied", 47:"AGB", 48:"AGUADECOCO", 49:"AKA New York", 50:"AKIRA", 51:"AKOO", 52:"ALDO", 53:"ALEX AND ANI", 54:"ALLOY", 55:"ALO Yoga", 56:"AMD", 57:"AMIA", 58:"ANAMA", 59:"AND", 60:"AND1", 61:"ANDREA FENZI", 62:"ANGL", 63:"API", 64:"ARCONA", 65:"ART", 66:"ASICS", 67:"ASOLO", 68:"ASOS", 69:"ASSETS by Sara Blakely", 70:"ASTR", 71:"ASUS", 72:"AT&T", 73:"AT-A-GLANCE", 74:"ATI Technologies", 75:"AX Paris", 76:"Abbott", 77:"Abercrombie & Fitch", 78:"Able provider", 79:"Abu Garcia", 80:"Acacia Swimwear", 81:"Academy", 82:"Accessory Collective", 83:"Accessory Innovations", 84:"Accessory Power", 85:"Accessory Workshop", 86:"Accutron", 87:"Acer", 88:"Acne Jeans", 89:"Acne Studios", 90:"Acorn", 91:"Acrobat", 92:"Act", 93:"Active", 94:"Activision", 95:"Actron®", 96:"Acure", 97:"Adagio", 98:"Adams", 99:"Adams Golf", 100:"Add Down", 101:"Addison", 102:"Adee Kaye", 103:"Adidas", 104:"Adolfo", 105:"Adonna", 106:"Adora", 107:"Adrianna Papell", 108:"Adrienne Landau", 109:"Adrienne Vittadini", 110:"Advanced Graphics", 111:"Advanced Healthcare Distributors", 112:"Advantage", 113:"Advent", 114:"Advil", 115:"Aero", 116:"AeroLatte", 117:"AeroPress", 118:"Aeropostale", 119:"Aerosoles", 120:"Affliction", 121:"Agent Provocateur", 122:"Agetec", 123:"Agfa", 124:"Agraria", 125:"Agua Bendita", 126:"Aidan Mattox", 127:"Aiptek", 128:"Air Force", 129:"Air Hogs", 130:"Air Jordan", 131:"Air Wick", 132:"Air Zone", 133:"Airwalk", 134:"Akademiks", 135:"Akai", 136:"Akribos XXIV", 137:"Akris", 138:"Akris punto", 139:"Alan Flusser", 140:"Alba", 141:"Albert Nipon", 142:"Alberto Makali", 143:"Alcon", 144:"Alegro", 145:"Alessi", 146:"Alex & Ava", 147:"Alex Evenings", 148:"Alex Marie", 149:"Alex Toys", 150:"Alexander Julian", 151:"Alexander Kalifano", 152:"Alexander McQueen", 153:"Alexander Wang", 154:"Alexandra de Markoff", 155:"Alexis Brittar", 156:"Alfani", 157:"Alfred Angelo", 158:"Alfred Dunner", 159:"Algenist", 160:"Ali & Kris", 161:"Alia", 162:"Alice & Trixie", 163:"Alice & You", 164:"Alice + Olivia", 165:"Alien Workshop", 166:"Alife", 167:"All", 168:"All American", 169:"All Four Paws", 170:"All Living Things", 171:"All Saints", 172:"All Sport", 173:"All-Clad", 174:"AllSaints", 175:"Allegra K", 176:"Allegri", 177:"Allen Allen", 178:"Allen B. by Allen Schwartz", 179:"Allen Edmonds", 180:"Allen Solly", 181:"Allison Brittney", 182:"Allison Daley", 183:"Allison Taylor", 184:"Allure", 185:"Almay", 186:"Almost Famous", 187:"Alo", 188:"Alpha Industries", 189:"Alpine", 190:"Alpine Design", 191:"Alpinestars", 192:"Alstyle Apparel", 193:"Altamont", 194:"Altar'd State", 195:"Altec Lansing", 196:"Alternative", 197:"Alternative Apparel", 198:"Altra", 199:"Always", 200:"Alyce Paris", 201:"Alyn Paige", 202:"Alythea", 203:"Amanda Smith", 204:"Amazon", 205:"Amazon Essentials", 206:"AmazonBasics", 207:"Ambi", 208:"Ambiance Apparel", 209:"Ambrielle", 210:"Ameda", 211:"American Apparel", 212:"American Boy & Girl", 213:"American Classics", 214:"American Eagle", 215:"American Fighter", 216:"American Girl ®", 217:"American Living", 218:"American Needle", 219:"American Rag", 220:"American Rebel Boot Company", 221:"American Tourister", 222:"American Vintage", 223:"American Weigh", 224:"Amoena", 225:"Amphibious Outfitters", 226:"Amsoil", 227:"Anastasia Beverly Hills", 228:"Anchor Blue", 229:"Anchor Hocking", 230:"Anderson Bean", 231:"Andis", 232:"Andrea by Sadek", 233:"Andrew Christian", 234:"Andrew Marc", 235:"Angelcare", 236:"Angelina", 237:"Angels", 238:"Angie", 239:"Angry Birds", 240:"Animal Adventure", 241:"Animal Alley", 242:"Animal Planet", 243:"Anita", 244:"Anker", 245:"Ann Marino", 246:"Ann Taylor", 247:"Ann Taylor LOFT", 248:"Anna Kostrova", 249:"Anna Sui", 250:"Annabelle", 251:"Anne Cole", 252:"Anne Fontaine", 253:"Anne Geddes", 254:"Anne Klein", 255:"Anne Michelle", 256:"Anolon", 257:"Anthony", 258:"Anthropologie", 259:"Antigua", 260:"Antik Batik", 261:"Antik Denim", 262:"Antilia Femme", 263:"Antique Rivet", 264:"Antonio Melani", 265:"Anvil", 266:"Anxiety", 267:"Apostrophe", 268:"Apple", 269:"Apple Bottoms", 270:"Apt.", 271:"Apt. 9", 272:"Aqua", 273:"Aqua Clear", 274:"Aqua Lung Sport", 275:"Aquafresh", 276:"Aquaphor", 277:"Aquarius", 278:"Aquatopia", 279:"Aqueon", 280:"Arbonne", 281:"Arc'teryx", 282:"Archaic", 283:"Archie Comics", 284:"Architect", 285:"Ardell", 286:"Arden B", 287:"Aria", 288:"Ariana Grande", 289:"Ariat", 290:"Aritzia", 291:"Ariya", 292:"Arizona", 293:"Arizona Jean Company", 294:"Ark & Co", 295:"Arm & Hammer", 296:"Arm's Reach Concepts", 297:"Armani", 298:"Armani Collezioni", 299:"Armani Exchange", 300:"Armani Jeans", 301:"Armor All®", 302:"Army", 303:"Aroma", 304:"Arrid", 305:"Arrow", 306:"Artful Dodger", 307:"Arturo Chiang", 308:"As Seen on TV", 309:"As U Wish", 310:"Ash", 311:"Ashley B", 312:"Ashley Graham", 313:"Ashley Stewart", 314:"Ashworth", 315:"Ask", 316:"Aspeed", 317:"Aspen Pet", 318:"Assets", 319:"Astroglide", 320:"Atari", 321:"Athena", 322:"Athleta", 323:"Athlete", 324:"Athletech", 325:"Athletic Works", 326:"Atlantis Toy and Hobby", 327:"Atmosphere", 328:"Attends", 329:"Attention", 330:"Audemars Piguet", 331:"Audi", 332:"Audio-Technica", 333:"AudioSource", 334:"Audiovox", 335:"Audrey Brooke", 336:"August Silk", 337:"Augusta Sportswear", 338:"Aura Cacia", 339:"Aurora World", 340:"Aurum", 341:"Aussie", 342:"Austin Clothing Co.", 343:"Austin Reed", 344:"Australian Gold", 345:"Authentic Original Vintage Style", 346:"Auto Expressions", 347:"Auto Meter", 348:"Auto Ventshade®", 349:"Autolite®", 350:"Automotive Innovations", 351:"Avanti", 352:"Aveda", 353:"Aveeno", 354:"Avent", 355:"Avenue", 356:"Avery", 357:"Avia", 358:"Avirex", 359:"Avon", 360:"Avon Cosmetics, Inc", 361:"Axcess", 362:"Axe", 363:"Azo", 364:"Azules", 365:"Azzaro", 366:"A|X Armani Exchange", 367:"Aéropostale", 368:"B Darlin", 369:"B O X Packaging", 370:"B is for Bear", 371:"B-Low the Belt", 372:"B. Darlin", 373:"B. Moss", 374:"B. Smart", 375:"B.swim", 376:"BAD BOY", 377:"BB", 378:"BB Dakota", 379:"BC Footwear", 380:"BCBG", 381:"BCBG Max Azria", 382:"BCBGMAXAZRIA", 383:"BCBGeneration", 384:"BCBGirls", 385:"BCX", 386:"BDG", 387:"BEACH RIOT", 388:"BEARPAW", 389:"BISOU BISOU", 390:"BKE", 391:"BLACKHAWK!", 392:"BLANKNYC", 393:"BLANQI", 394:"BLOCH", 395:"BLUE", 396:"BMW", 397:"BOB Strollers", 398:"BOGS", 399:"BOOMco.", 400:"BOSS Black", 401:"BOSS HUGO BOSS", 402:"BOYS + ARROWS", 403:"BUNN", 404:"BVD", 405:"Babies R Us", 406:"Babies R Us Infant", 407:"Babies R Us Plush", 408:"Babolat", 409:"Baby Aspen", 410:"Baby Boom", 411:"Baby Brezza", 412:"Baby Bullet", 413:"Baby Delight", 414:"Baby Einstein", 415:"Baby Essentials", 416:"Baby Gear", 417:"Baby Jogger", 418:"Baby Legs", 419:"Baby Magic", 420:"Baby Phat", 421:"Baby Trend", 422:"Baby connection", 423:"BabyBjorn", 424:"BabyGanics", 425:"Babyhome", 426:"Bacati", 427:"Bacco Bucci", 428:"Bachmann Trains", 429:"Bachrach", 430:"Back Buddy", 431:"Back to Basics", 432:"Badger", 433:"Badger Basket", 434:"Badgley Mischka", 435:"Baffin", 436:"Bagatelle", 437:"Baggallini", 438:"Bailey", 439:"Bailey 44", 440:"Baker & Taylor", 441:"Baker by Ted Baker", 442:"Bakers", 443:"Balboa Baby", 444:"Baleaf", 445:"Balenciaga", 446:"Ball", 447:"Bally", 448:"Balmain", 449:"Baltex", 450:"Bambo Nature", 451:"Bamboo", 452:"Bamboo Cay", 453:"Bamboobies", 454:"Ban", 455:"Banana Boat", 456:"Banana Republic", 457:"Bananafish", 458:"Band of Gypsies", 459:"Band-Aid", 460:"Bandai", 461:"Bandai Namco", 462:"Bandolino", 463:"Bang & Olufsen", 464:"Banzai", 465:"Bape", 466:"Bar Lll", 467:"Baratza", 468:"Barbasol", 469:"Barbie", 470:"Barbour", 471:"Bare Escentuals", 472:"Bare Minerals", 473:"Barefoot Dreams", 474:"Bari Jay", 475:"Barnett Crossbows", 476:"Barneys New York", 477:"Barron's", 478:"Barska", 479:"Basic Editions", 480:"Basic Fun", 481:"Bass", 482:"Bassett Furniture", 483:"Bates", 484:"Bath & Body Works", 485:"Batman", 486:"BaubleBar", 487:"Bauer", 488:"Bausch & Lomb", 489:"Bay Island", 490:"Bay Studio", 491:"Bayer Expert Care", 492:"Be Wicked!", 493:"BeBop", 494:"BeMaternity", 495:"Beaba", 496:"Beach Bunny", 497:"Beach Joy", 498:"Bear Archery", 499:"Beats", 500:"Beats by Dr. Dre", 501:"Beautees", 502:"BeautiControl", 503:"Beautifly", 504:"Beautycounter", 505:"Bebe", 506:"Bebe Au Lait", 507:"Bebe Stores", 508:"BebeSounds", 509:"Becca", 510:"Becca Cosmetics", 511:"Bechamel", 512:"Bed Stu", 513:"BedHead", 514:"Beechnut", 515:"Begin Again", 516:"Bejeweled", 517:"Believe", 518:"Belif", 519:"Belkin", 520:"Bella", 521:"Bella Cucina", 522:"Bella Dahl", 523:"Bellatrix", 524:"Belldini", 525:"Belle Du Jour", 526:"Belle by Sigerson Morrison", 527:"Belleek", 528:"Belltech", 529:"Belly Bandit®", 530:"Ben Davis", 531:"Ben Nye", 532:"Ben Sherman", 533:"BenBat", 534:"Bench", 535:"Benefit", 536:"Benefit Cosmetics", 537:"Benetton", 538:"Bepuzzled", 539:"Berek", 540:"Beretta", 541:"Bergan", 542:"Berkley", 543:"Berkshire", 544:"Bernardo", 545:"Berne Apparel", 546:"BernzOmatic", 547:"Bershka", 548:"Best Friends by Sheri", 549:"Bethesda Softworks", 550:"Betsey Johnson", 551:"Betsey Johnson Label", 552:"Betsy & Adam", 553:"Better Shea Butter", 554:"Bettie Page", 555:"Betty Boop", 556:"Betty Crocker", 557:"Beverly Drive", 558:"Beverly Hills MD", 559:"Beverly Hills Polo Club", 560:"Beyond Yoga", 561:"Bh", 562:"Bialetti", 563:"Bic", 564:"Big Bang Theory", 565:"Big Buddha", 566:"Big Dogs", 567:"Big Star", 568:"Big Time Toys", 569:"Bike", 570:"Bikerwear", 571:"Bill Blass", 572:"Billabong", 573:"Billionaire Boys Club", 574:"Billionaire Boys Club and Ice Cream", 575:"Billy Reid", 576:"Bio Oil", 577:"Bio Spot", 578:"Biore", 579:"Bioworld", 580:"Bioworld Merchandising", 581:"Birkenstock", 582:"Bissell", 583:"Bite Beauty", 584:"Bitten", 585:"Bjorn Borg", 586:"Blac Label", 587:"Black & Decker", 588:"Black Halo", 589:"Black Poppy", 590:"Black Rivet", 591:"Black Scale", 592:"Blackhawk", 593:"Blackmilk", 594:"Blair", 595:"Blanc Noir", 596:"Blank", 597:"Blank NYC", 598:"Blauer", 599:"Blendtec", 600:"Blinc", 601:"Blind", 602:"Blink", 603:"Blip Toys", 604:"Bliss", 605:"Blitz®", 606:"Blizzard", 607:"Blondie Nites", 608:"Bloomfield", 609:"Blooming Bath", 610:"Bloomingdale's", 611:"Blossom Bucket", 612:"Blowfish", 613:"Blu Pepper", 614:"Blue Asphalt", 615:"Blue Bird", 616:"Blue Box", 617:"Blue Buffalo", 618:"Blue Cult", 619:"Blue Life", 620:"Blue Microphones", 621:"Blue Nile", 622:"Blunt Power", 623:"Blush", 624:"Bob Mackie", 625:"Bobbi Brown", 626:"Bobbie Brooks", 627:"Bobeau", 628:"Boden", 629:"Bodum", 630:"Body & Hair", 631:"Body Central", 632:"Body Glove", 633:"Bogner", 634:"Boho Chic", 635:"Bolle", 636:"Bon Bebe", 637:"BonWorth", 638:"Bondi Band", 639:"Bongo", 640:"Bonne Bell", 641:"Bonnie Baby", 642:"Bonnie Jean", 643:"Bonobos", 644:"Boo!", 645:"Boogie Wipes", 646:"BooginHead", 647:"Boohoo", 648:"Boohoo Petite", 649:"Boohoo Plus", 650:"Boom Boom", 651:"Boon", 652:"Boots", 653:"Boppy", 654:"Borghese", 655:"Born", 656:"BornFree", 657:"Borrelli", 658:"Bosch", 659:"Boscia", 660:"Bose", 661:"Boston Proper", 662:"Bostonian", 663:"Botkier", 664:"Bottega Veneta", 665:"Boudreaux", 666:"Boudreauxs Butt Paste", 667:"Boulevard Boutique", 668:"Bourbon and Bowties", 669:"Bourjois", 670:"Boutique 9", 671:"Bowery Supply", 672:"Bowflex", 673:"Boxercraft", 674:"Boy + Girl", 675:"Braetan", 676:"Brahmin", 677:"Brake & Wheel Parts", 678:"Brandon Thomas", 679:"Brandy Melville", 680:"Braun", 681:"Bravado", 682:"Bravado Designs", 683:"Brave Soul", 684:"BrazilRoxx", 685:"BreathableBaby", 686:"Breckelles", 687:"Breckenridge", 688:"Breezies", 689:"Breitling", 690:"Brentwood", 691:"Bret Michaels", 692:"Breville", 693:"Breyer", 694:"Briarpatch", 695:"Brica", 696:"Bridgestone Golf", 697:"Briefly Stated", 698:"Briggs New York", 699:"Bright Starts", 700:"Brighton", 701:"Brighton Collectibles", 702:"Brine", 703:"Briogeo", 704:"Brioni", 705:"Brita", 706:"Britax", 707:"Brittany Black", 708:"Brixton", 709:"Brockum", 710:"Brooklyn Express", 711:"Brooks", 712:"Brooks Brothers", 713:"Brookstone", 714:"Brother", 715:"Browning", 716:"Bruder Toys America", 717:"Bruno Magli", 718:"Brut", 719:"Brute", 720:"Bubblegum", 721:"Bucket Feet", 722:"Buckle", 723:"Buckley Supply Co.", 724:"Budge", 725:"Buff", 726:"Buffalo", 727:"Buffalo David Bitton", 728:"Buffalo Games", 729:"Buffalo Jeans", 730:"Bugaboo", 731:"Bugle Boy", 732:"Bullhead", 733:"Bully Dog", 734:"Bulova", 735:"Bulwark", 736:"Bumble Collection", 737:"Bumble and bumble", 738:"Bumbleride", 739:"Bumbo", 740:"Bumkins", 741:"BumpStart", 742:"Burberry", 743:"Burberry Brit", 744:"Burberry London", 745:"Burnside", 746:"Burt's Bees", 747:"Burt's Bees Baby", 748:"Burton", 749:"Bushnell", 750:"Butterfly", 751:"Buxom", 752:"Buxton", 753:"Buxton Leather Goods", 754:"Bvlgari", 755:"By Terry", 756:"Byer Too", 757:"Bésame Cosmetics", 758:"C by Champion", 759:"C&A", 760:"C&C California", 761:"C. O. Bigelow", 762:"CA by vitamin A", 763:"CALIA by Carrie Underwood", 764:"CAMPER", 765:"CAT", 766:"CAbi", 767:"CB Sports", 768:"CBK", 769:"CC Skye", 770:"CCB Collection", 771:"CCM", 772:"CERRUTI", 773:"CHI", 774:"CIOR", 775:"CJ Banks", 776:"CK Jeans", 777:"CLEAN", 778:"CM Style Fashion", 779:"COLLECTIVE CONCEPTS", 780:"COOGI", 781:"CW-X", 782:"Cabales", 783:"Cabela's", 784:"Cabernet", 785:"Cabin Creek", 786:"Cable & Gauge", 787:"Cache", 788:"Cachet", 789:"Cacique", 790:"Cactus", 791:"Cake", 792:"Calgon", 793:"Cali", 794:"Calico Critters", 795:"California Baby", 796:"California Costume Collection", 797:"California Scents", 798:"California Waves", 799:"Call It Spring", 800:"Callaway", 801:"Calphalon", 802:"Calvin Klein", 803:"Calypso St. Barth", 804:"Cambio", 805:"Cambridge Classics", 806:"Cambridge Satchel", 807:"CamelBak", 808:"Camilia", 809:"Camilla", 810:"Campia Moda", 811:"Canada Goose", 812:"Canali", 813:"Candie's", 814:"Candle-Lite", 815:"Cannon", 816:"Canon", 817:"Canterbury", 818:"Canvas", 819:"Canyon River Blues", 820:"Capcom", 821:"Capelli", 822:"Capelli of New York", 823:"Capezio", 824:"Capresso", 825:"Caravelli", 826:"Carbon", 827:"Cardinal Industries", 828:"Cardinal of Canada", 829:"Cards Against Humanity", 830:"Carefree", 831:"Caress Brand Shop", 832:"Cargo", 833:"Carhartt", 834:"Caribbean Joe", 835:"Carlos Santana", 836:"Carlson Pet Products", 837:"Carmen Marc Valvo", 838:"Carmex", 839:"Carnival", 840:"Carol Anderson", 841:"Carol's Daughter", 842:"Carole Hochman", 843:"Carole Little", 844:"Carolina Blues", 845:"Carolina Colours", 846:"Carolina Herrera", 847:"Carolyn Taylor", 848:"Carrera", 849:"Carrhart", 850:"Carson Home Accents", 851:"Carter's", 852:"Cartier", 853:"Cartoon Network", 854:"Caruso", 855:"Carven", 856:"Cascade", 857:"Case Logic", 858:"Casetify", 859:"Casio", 860:"Caslon", 861:"Cassandra Stone", 862:"Castle", 863:"Casual", 864:"Casual Corner", 865:"Casual Corner Annex", 866:"Casual Industrees", 867:"Catalina", 868:"Caterpillar", 869:"Catherine Catherine Malandrino", 870:"Catherine Malandrino", 871:"Catherine Popesco", 872:"Catherines", 873:"Cathy Daniels", 874:"Cathy Jean", 875:"Catit", 876:"Cato", 877:"Caudalie", 878:"Cdc Clothing", 879:"Ceaco", 880:"Cecico", 881:"Celebrity Pink", 882:"Celine", 883:"Cello Jeans", 884:"Central Park West", 885:"Century", 886:"Cepia", 887:"CeraVe", 888:"Cesar", 889:"Cetaphil", 890:"Cezani", 891:"Cha Cha Vente", 892:"Chaco", 893:"Chadwicks", 894:"Chalk Line", 895:"Chamilia", 896:"Champion", 897:"Champion Sports", 898:"Champro", 899:"Champs", 900:"Chan Luu", 901:"Chanel", 902:"Changes", 903:"Chann Luu", 904:"Channel", 905:"Chantelle", 906:"Chaps", 907:"Chapstick", 908:"Character", 909:"Charles David", 910:"Charles Klein", 911:"Charles River Apparel", 912:"Charles Tyrwhitt", 913:"Charlie Banana", 914:"Charlie by Mattew Zink", 915:"Charlotte Olympia", 916:"Charlotte Russe", 917:"Charlotte Tilbury", 918:"CharmLeaks", 919:"Charmin", 920:"Charming Charlie", 921:"Charter Club", 922:"Chaser", 923:"Chaus", 924:"Cheap Monday", 925:"Cheetah", 926:"Chef's Choice", 927:"Chef's Secret", 928:"Chelsea & Violet", 929:"Chemex", 930:"Chemistry", 931:"Chequer", 932:"Cherokee", 933:"Chesley", 934:"Chevrolet", 935:"Chevron®", 936:"Chic", 937:"Chicago", 938:"Chicco", 939:"Chico's", 940:"Children's Place", 941:"Chilton", 942:"China Glaze Co. Ltd.", 943:"Chinese Laundry", 944:"Chip & Pepper", 945:"Chloe", 946:"Chloe + Isabel", 947:"Chloe K", 948:"Chloé", 949:"Chocolove", 950:"Chopard", 951:"Choppers", 952:"Christian Audigier", 953:"Christian Dior", 954:"Christian Lacroix", 955:"Christian Louboutin", 956:"Christian Siriano", 957:"Christina", 958:"Christine Alexander", 959:"Christophe Robin", 960:"Christopher & Banks", 961:"Christopher Blue", 962:"Chroma™", 963:"Chrome Hearts", 964:"Chromo Inc", 965:"Chronicle Books", 966:"Chuckit", 967:"Ciaté London", 968:"Cinch", 969:"Cinema Secrets", 970:"Cipo & Baxx", 971:"Circa", 972:"Circo", 973:"Circulon", 974:"Circus by Sam Edelman", 975:"Citizen", 976:"Citizens of Humanity", 977:"Citizens of Humanity Jeans", 978:"Citron Santa Monica", 979:"City Chic", 980:"City Streets", 981:"City Studio", 982:"City Triangles", 983:"Claiborne", 984:"Claire's", 985:"Clairol", 986:"Clarins", 987:"Clarisonic", 988:"Clarisse", 989:"Clark's Botanicals", 990:"Clarks", 991:"Classic Accessories", 992:"Classic Elements", 993:"Classic Media", 994:"Classiques Entier", 995:"Claudia Richard", 996:"Clean & Clear", 997:"Clear Care", 998:"Clearasil", 999:"Cleobella", 1000:"Cleveland Golf", 1001:"Cliff Keen", 1002:"Clinique", 1003:"Clinique Du Rond", 1004:"Clio", 1005:"Cloud B", 1006:"Clover Canyon", 1007:"Club Monaco", 1008:"Club Room", 1009:"CoCaLo", 1010:"Coach", 1011:"Coast", 1012:"Cobra", 1013:"Cobra Electronics", 1014:"Coby Electronics", 1015:"Coca-Cola", 1016:"Coco Rave", 1017:"Coco Reef", 1018:"Cocomo", 1019:"Code Bleu", 1020:"Coed Sportwear", 1021:"Coldwater Creek", 1022:"Cole Haan", 1023:"Colebrook", 1024:"Coleman", 1025:"Colgate", 1026:"Colin Stuart", 1027:"Collector's Gallery", 1028:"College Edition", 1029:"Colonial Candle", 1030:"Color Oops", 1031:"Colorado Clothing", 1032:"Colorbok", 1033:"Colosseum", 1034:"ColourPop Cosmetics", 1035:"Colourpop", 1036:"Columbia", 1037:"Com", 1038:"Combi", 1039:"Comfort & Harmony", 1040:"Comfort Choice", 1041:"Comfort Colors", 1042:"Comfort Zone", 1043:"Comic Images", 1044:"Comme des Garcons", 1045:"Commonwealth Toys", 1046:"Comotomo", 1047:"Compaq", 1048:"Complete", 1049:"ComplyRight", 1050:"Comptoir Sud Pacifique", 1051:"Conair", 1052:"Concept One", 1053:"Connected", 1054:"Context", 1055:"Contigo", 1056:"Contours", 1057:"Converse", 1058:"Converse Shoes", 1059:"Coobie", 1060:"Coogi Fashions", 1061:"Cookie Lee", 1062:"Cooks Essentials", 1063:"Coola", 1064:"Cooler Master", 1065:"Cooper", 1066:"Cooperative", 1067:"Copco", 1068:"Copper Key", 1069:"Coral Bay", 1070:"Cordani Shoes", 1071:"Corelle", 1072:"CorningWare", 1073:"Corolle", 1074:"Corona", 1075:"Corral", 1076:"Corsair", 1077:"Corso Como", 1078:"Cos", 1079:"Cosabella", 1080:"Cosco", 1081:"Costa Del Mar", 1082:"Costume Super Center", 1083:"Cotton Express", 1084:"Cotton Heritage", 1085:"Cotton On", 1086:"Cottonelle", 1087:"Coty", 1088:"Coty, Inc.", 1089:"Coulbourne", 1090:"Counterparts", 1091:"Courtenay", 1092:"Cover Fx", 1093:"Cover Girl", 1094:"CoverGirl", 1095:"Covington", 1096:"Cowgirl Tuff", 1097:"Cra-Z-Art", 1098:"Crabtree & Evelyn", 1099:"Craftsman", 1100:"Crane", 1101:"Crayola", 1102:"Crazy Horse", 1103:"Creative Designs", 1104:"Creative Labs", 1105:"Creative Recreation", 1106:"Creativity for Kids", 1107:"Creature", 1108:"Creed", 1109:"Cremieux", 1110:"Crest", 1111:"CritterWare", 1112:"Crocs", 1113:"Croft & Barrow", 1114:"Crofton", 1115:"Crooks & Castles", 1116:"Croota", 1117:"Crosley Radio", 1118:"Crosman", 1119:"Cruel Girl", 1120:"Cruiser Accessories", 1121:"Crystal Doll", 1122:"Crystal Rock", 1123:"Cubavera", 1124:"Cuddl Duds", 1125:"Cuisinart", 1126:"Cumberland Bay", 1127:"Curel", 1128:"Current/Elliott", 1129:"Curtis Computer Products", 1130:"Curvation", 1131:"Curve", 1132:"Curves", 1133:"Curvy Couture", 1134:"Custo Barcelona", 1135:"Custom Accessories", 1136:"Customized & Personalized", 1137:"Cutie Pie Baby", 1138:"Cutter & Buck", 1139:"Cutters", 1140:"Cygnus", 1141:"Cynthia Howie", 1142:"Cynthia Rowley", 1143:"Cypress Links", 1144:"Cyrus", 1145:"Céline", 1146:"D&G", 1147:"D'Addario", 1148:"D-Link", 1149:"DAKS", 1150:"DBX", 1151:"DC Comics", 1152:"DC Direct", 1153:"DC Shoes", 1154:"DC Sports", 1155:"DCC", 1156:"DDF", 1157:"DGK", 1158:"DIESEL", 1159:"DIRECTV", 1160:"DJI", 1161:"DKNY", 1162:"DKNY Jeans", 1163:"DKNYC", 1164:"DL", 1165:"DL1961", 1166:"DP", 1167:"DRI DUCK", 1168:"DUO", 1169:"DV by Dolce Vita", 1170:"DVS", 1171:"DYMO", 1172:"Da-Nang", 1173:"Dainese", 1174:"Daisy Fuentes", 1175:"Dakine", 1176:"Dale Of Norway", 1177:"Dalia", 1178:"Dan Post", 1179:"Dana Buchman", 1180:"Daniel Cremieux", 1181:"Daniel Rainn", 1182:"Daniel Wellington", 1183:"Danielle Nicole", 1184:"Danielson Designs", 1185:"Danier", 1186:"Danner Inc", 1187:"Danny & Nicole", 1188:"Dansk", 1189:"Danskin", 1190:"Danskin Now", 1191:"Dansko", 1192:"Dark Horse", 1193:"Dave & Johnny", 1194:"David & Goliath", 1195:"David Brooks", 1196:"David Carey", 1197:"David Kahn", 1198:"David Taylor", 1199:"David Yurman", 1200:"David's Bridal", 1201:"Dawgs", 1202:"Daytrip", 1203:"Dazzle", 1204:"Dazzle Multimedia", 1205:"De La Mer", 1206:"DeLONG", 1207:"DeLonghi", 1208:"DeMarini", 1209:"DeWALT®", 1210:"Deb", 1211:"Debbie Morgan", 1212:"Deco Mesh", 1213:"DecoLav", 1214:"Decree", 1215:"Deena & Ozzy", 1216:"Degree", 1217:"Deletta", 1218:"Delia's", 1219:"Dell", 1220:"Delta", 1221:"Delta Burke", 1222:"Deltana", 1223:"Demdaco", 1224:"DenTek", 1225:"Denali", 1226:"Denim & Co.", 1227:"Denim & Supply Ralph Lauren", 1228:"Denim Riders", 1229:"Dennis Basso", 1230:"Denon", 1231:"Denso", 1232:"Derek Heart", 1233:"Derek Lam", 1234:"Dereon", 1235:"Dermablend", 1236:"Dermal", 1237:"Descente", 1238:"Design History", 1239:"Desigual", 1240:"Desitin", 1241:"Dessy Collection", 1242:"Destination Maternity", 1243:"Deux Lux", 1244:"DevaCurl", 1245:"Devon & Jones", 1246:"Dex", 1247:"Dex Baby", 1248:"Dexter", 1249:"Diadora", 1250:"Dial", 1251:"Diamond", 1252:"Diamond Plate", 1253:"Diamond Select", 1254:"Diamond Supply Co.", 1255:"Diane Gilman", 1256:"Diane von Furstenberg", 1257:"Diaper Dude", 1258:"Diaper Genie", 1259:"Diba", 1260:"Dickies", 1261:"Dingo", 1262:"Diono", 1263:"Dior", 1264:"Discovery Kids", 1265:"Disguise", 1266:"Disney", 1267:"Disney Baby", 1268:"Disney Collection", 1269:"Disney Interactive", 1270:"Disney Jr.", 1271:"Disney Pixar", 1272:"Disney Pixar Cars", 1273:"Disney Pixar Planes", 1274:"Disney Press", 1275:"Disney Princess", 1276:"Disney Tsum Tsum", 1277:"District", 1278:"District Threads", 1279:"Dittos", 1280:"Diva", 1281:"Divided", 1282:"Dockers", 1283:"Doctor Who", 1284:"Doe", 1285:"Dog MD", 1286:"Dogeared", 1287:"Dogswell", 1288:"Dolce & Gabbana", 1289:"Dolce Vita", 1290:"Dollhouse", 1291:"Dollie & Me", 1292:"Donald J. Pliner", 1293:"Donald Trump", 1294:"Doncaster", 1295:"Donna Bella", 1296:"Donna Karen", 1297:"Donna Morgan", 1298:"Donna Ricco", 1299:"Donnybrook", 1300:"Dooney & Bourke", 1301:"Dope", 1302:"Dora the Explorer", 1303:"Dorel", 1304:"Doris", 1305:"Dorman®", 1306:"Dots", 1307:"Double Zero", 1308:"Douglas", 1309:"Dove", 1310:"Downy", 1311:"Dr. Brandt", 1312:"Dr. Brown's", 1313:"Dr. Dennis Gross Skincare", 1314:"Dr. Hauschka", 1315:"Dr. Martens", 1316:"Dr. Miracle's", 1317:"Dr. Scholl's", 1318:"Dr. Seuss", 1319:"Dr. Teal's", 1320:"Dragon", 1321:"Dragon Alliance", 1322:"Dragon Ball Z", 1323:"Dragonfly", 1324:"Drake", 1325:"Dream Lites", 1326:"Dream On Me", 1327:"Dreamgirl", 1328:"Dreamworks", 1329:"Dreamworks Animated", 1330:"Dremel", 1331:"Dress Barn", 1332:"Drinkwell", 1333:"Drunk Elephant", 1334:"Drybar", 1335:"Dscp", 1336:"Dsquared", 1337:"DuWop", 1338:"Dubble Bubble", 1339:"Duck Commander", 1340:"Duck Head", 1341:"Ducks Unlimited", 1342:"Duluth Trading Company", 1343:"Dunbrooke", 1344:"Duncan Toys", 1345:"Dunlop", 1346:"Duo Maternity", 1347:"Duracell", 1348:"Durango", 1349:"Dynasty", 1350:"Dyson", 1351:"EA Sports", 1352:"EACH X OTHER", 1353:"ECI", 1354:"ECKO", 1355:"EK Success", 1356:"ELF", 1357:"ELLE", 1358:"ENERGIE", 1359:"EO", 1360:"EO Essential Oils", 1361:"ESPN", 1362:"EVGA", 1363:"EVISU", 1364:"EXP Computer", 1365:"Eagle USA", 1366:"Earl Jean", 1367:"Earl Jeans", 1368:"Earnest Sewn", 1369:"Earth Mama Angel Baby", 1370:"Earth's Best", 1371:"Earth’s Nectar", 1372:"Easel", 1373:"East 5th", 1374:"Eastern Mountain Sports", 1375:"Eastessence", 1376:"Eastland", 1377:"Easton", 1378:"Easy Spirit", 1379:"Ecco Bella", 1380:"Ecko Red", 1381:"Ecko Unltd.", 1382:"Eclectic Products", 1383:"Ecote", 1384:"Ecotools", 1385:"Ed Hardy", 1386:"Eddie Bauer", 1387:"Eddie Bauer Clothing", 1388:"Edelbrock", 1389:"Eden", 1390:"Edge", 1391:"Edu Science", 1392:"Educational Insights", 1393:"Eight Eight Eight", 1394:"Eighty Eight", 1395:"Eileen Fisher", 1396:"Ektelon", 1397:"Elan", 1398:"Electric Family", 1399:"Electric Visual", 1400:"Electronic Arts", 1401:"Elegant Moments", 1402:"Element", 1403:"Elements", 1404:"Elementz", 1405:"Elemis Ltd", 1406:"Elevenses", 1407:"Elf on the Shelf", 1408:"Elie Saab", 1409:"Elie Tahari", 1410:"Elite", 1411:"Elixir", 1412:"Eliza J", 1413:"Elizabeth Arden", 1414:"Elizabeth Hurley", 1415:"Elizabeth and James", 1416:"Elizavecca", 1417:"Ella Moss", 1418:"Ellemenno", 1419:"Ellen Tracy", 1420:"Elliott Lucca", 1421:"Ellison", 1422:"Elmers", 1423:"Elmo", 1424:"Elomi", 1425:"Elwood", 1426:"Ely Cattleman", 1427:"Emachines", 1428:"Emanuel Ungaro", 1429:"Emerald Sundae", 1430:"Emerica", 1431:"Emerson", 1432:"Emile et Rose", 1433:"Emilio Pucci", 1434:"Emily Ray", 1435:"Eminence", 1436:"Emma Fox", 1437:"Emma James", 1438:"Empire", 1439:"Emporio Armani", 1440:"Emson", 1441:"Emu", 1442:"En Focus Studio", 1443:"Endless Rose", 1444:"Energizer®", 1445:"Enesco", 1446:"Enfagrow", 1447:"Enfamil", 1448:"English Factory", 1449:"Enjoi", 1450:"Enyce", 1451:"Enyce Clothing", 1452:"Enzo Angiolini", 1453:"Epic Threads", 1454:"Epson", 1455:"Equipment", 1456:"Ergodyne", 1457:"Eric Carle", 1458:"Eric Javits", 1459:"Erika", 1460:"Ermenegildo Zegna", 1461:"Ernie Ball", 1462:"Escada", 1463:"Escante", 1464:"Escort Radar", 1465:"Esley", 1466:"Especially For Kids", 1467:"Especially for Baby", 1468:"Esprit", 1469:"Essence of Beauty", 1470:"Essentials", 1471:"Essentials by Milano", 1472:"Essie", 1473:"Estee Lauder", 1474:"Estes", 1475:"Etcetera", 1476:"Etch A Sketch", 1477:"Etienne Aigner", 1478:"Etonic", 1479:"Etro", 1480:"Eucerin", 1481:"Euro Cuisine", 1482:"Euro-Pro", 1483:"Evan Picone", 1484:"Evenflo", 1485:"Ever-Pretty", 1486:"Everlane", 1487:"Everlast", 1488:"Everly", 1489:"Everly Grey", 1490:"Evie", 1491:"EvoShield", 1492:"Evolution", 1493:"Evriholder", 1494:"ExOfficio", 1495:"Excalibur", 1496:"Excell", 1497:"Excelled", 1498:"ExerSaucer", 1499:"Exergen", 1500:"Exo-Terra", 1501:"Express", 1502:"Express, Inc.", 1503:"Expressions", 1504:"Exuviance", 1505:"Eyeshadow", 1506:"Ezekiel", 1507:"FABRIZIO GIANNI", 1508:"FAO Schwarz", 1509:"FELICITY & COCO", 1510:"FIONI Clothing", 1511:"FOREVER 21", 1512:"FORM Athletics", 1513:"FUBU", 1514:"FUBU Apparel", 1515:"FULL TILT", 1516:"Faber-Castell", 1517:"Fabletics", 1518:"Fabulous Frannie", 1519:"Faded Glory", 1520:"Fagor", 1521:"Fairway", 1522:"Faithfull the Brand", 1523:"Falls Creek", 1524:"Family Guy", 1525:"Famosa America", 1526:"Famous Catalog", 1527:"Famous Stars & Straps", 1528:"Fanatics", 1529:"Fang", 1530:"Fantasie", 1531:"Farberware", 1532:"Farmacy", 1533:"Farsali", 1534:"Fascinations", 1535:"Fashion Angels", 1536:"Fashion Bug", 1537:"Fashion Fair", 1538:"Fashion Forms", 1539:"Fashion Junkee", 1540:"Fashion Nova", 1541:"Fashion to Figure", 1542:"Fashionomics", 1543:"Fast Lane", 1544:"Fatal Clothing", 1545:"Fathead", 1546:"Faviana", 1547:"Febreze", 1548:"Feetures!", 1549:"Fellowes", 1550:"Fender", 1551:"Fendi", 1552:"Fenn Wright Manson", 1553:"Ferasali", 1554:"Fergalicious", 1555:"Fergie", 1556:"Ferragamo", 1557:"Ferrari", 1558:"Ferrecci", 1559:"Fetco", 1560:"Fever", 1561:"Field & Stream", 1562:"Field Gear", 1563:"Fiesta", 1564:"Fifth Sun", 1565:"Fila", 1566:"Fila Sport", 1567:"Filson", 1568:"Filter", 1569:"Filthy Dripped", 1570:"Finders Keepers", 1571:"Finish Line", 1572:"Fiore", 1573:"Fiorentini + Baker", 1574:"Fire Los Angeles", 1575:"First Act", 1576:"First Aid Beauty", 1577:"First Impressions", 1578:"First Wave", 1579:"Fisher-Price", 1580:"Fisher-Price Baby", 1581:"Fitbit", 1582:"Fitness Gear", 1583:"Five Four", 1584:"Fjallraven", 1585:"Flamingo", 1586:"Flavia", 1587:"Flax", 1588:"Flexees", 1589:"Flexi", 1590:"Flip", 1591:"Flirtitude", 1592:"Floam", 1593:"Floppy Products", 1594:"Floracraft", 1595:"Florsheim", 1596:"Flow Society", 1597:"Fluval", 1598:"Fluxus", 1599:"Fly London", 1600:"Fly Society", 1601:"Flying Monkey", 1602:"Flying Tomato", 1603:"Flynn Skye", 1604:"Focus", 1605:"Foley + Corinna", 1606:"Folkmanis", 1607:"Foot Locker", 1608:"Foot Traffic", 1609:"FootJoy", 1610:"For Bare Feet", 1611:"For Joseph", 1612:"For Love and Lemons", 1613:"Ford", 1614:"Ford Motorsport", 1615:"Forecaster of Boston", 1616:"Foreign Exchange", 1617:"Foreo", 1618:"Forever Unique", 1619:"Forplay", 1620:"Fossil", 1621:"Foster Grant", 1622:"Foundry", 1623:"Fourstar Clothing", 1624:"Fox", 1625:"Fox Racing", 1626:"Foxcroft", 1627:"Frame Denim", 1628:"Francesca's Collections", 1629:"Francesco Biasia", 1630:"Franco", 1631:"Franco Sarto", 1632:"Frank & Oak", 1633:"Frankie B", 1634:"Frankie's Bikinis", 1635:"Franklin", 1636:"Franklin Electronics", 1637:"Franklin Learning Systems", 1638:"Fred Perry", 1639:"Freddy", 1640:"Frederick's of Hollywood", 1641:"Free Country", 1642:"Free People", 1643:"Freestyle", 1644:"Freeze", 1645:"French Connection", 1646:"French Laundry", 1647:"French Toast", 1648:"Fresh", 1649:"Fresh Produce", 1650:"Freshly Picked", 1651:"Freshman", 1652:"Freya", 1653:"Frigidaire", 1654:"Friskies", 1655:"Frontline", 1656:"Frost Cutlery", 1657:"Fruit of the Loom", 1658:"Frye", 1659:"Frédérique Constant", 1660:"Fuda", 1661:"Fuji", 1662:"Fujifilm", 1663:"Fun World", 1664:"Funimation Production", 1665:"Funko", 1666:"Funrise", 1667:"Funtasma", 1668:"Furla", 1669:"Furminator", 1670:"G by Guess", 1671:"G-III", 1672:"G-Shock", 1673:"G-Star Raw", 1674:"G-Unit", 1675:"G.H. Bass & Co.", 1676:"G.I. JOE", 1677:"G.M. Specialties", 1678:"GAIAM", 1679:"GAIN", 1680:"GAME", 1681:"GBC", 1682:"GBX", 1683:"GE", 1684:"GEAR FOR SPORTS", 1685:"GEIGER", 1686:"GH", 1687:"GM", 1688:"GM Performance Parts", 1689:"GOLD TOE", 1690:"GPX", 1691:"GUESS", 1692:"GUM", 1693:"GX by Gwen Stefani", 1694:"Gabriel", 1695:"Gaggia", 1696:"Galaxy", 1697:"Gallery", 1698:"Galt", 1699:"Game Gear", 1700:"Gamo", 1701:"Ganz", 1702:"Gap", 1703:"Gap Inc.", 1704:"Garage", 1705:"Garfield & Marks", 1706:"Garmin", 1707:"Garnet Hill", 1708:"Garnier", 1709:"Gateway", 1710:"Gatorade", 1711:"Gear", 1712:"Gearbox", 1713:"GenOne", 1714:"General Electric", 1715:"Generic", 1716:"Generra", 1717:"Genetic Denim", 1718:"Geneva", 1719:"Geneva Platinum", 1720:"Genica", 1721:"Gentle Giant", 1722:"Gentle Souls", 1723:"GeoDiscovery", 1724:"GeoTrax", 1725:"Geoffrey Beene", 1726:"George", 1727:"George Foreman", 1728:"George Simonton", 1729:"Georgia Boot", 1730:"Georgio Armani", 1731:"Georgiou", 1732:"Gerber", 1733:"Gerber Childrenswear", 1734:"Gerber Foods", 1735:"Gerber Good Start", 1736:"Gevalia", 1737:"Ghast", 1738:"Ghost", 1739:"GiGi New York", 1740:"Giacca", 1741:"Gianfranco Ferre", 1742:"Giani Bernini", 1743:"Gianni Bini", 1744:"Giant", 1745:"Gibson", 1746:"Giftcraft", 1747:"Gigabyte", 1748:"Gigatent", 1749:"Gildan", 1750:"Gillette", 1751:"Gilligan & O'Malley", 1752:"Gilly Hicks", 1753:"Gimmick", 1754:"Ginsey", 1755:"Giordano", 1756:"Giorgio Armani", 1757:"Giorgio Brutini", 1758:"Giovanni Navarre", 1759:"Girbaud", 1760:"Giro", 1761:"Gitano", 1762:"Giuseppe Zanotti", 1763:"Givenchy", 1764:"Glade", 1765:"Glamglow", 1766:"Glamorise", 1767:"Glide Bikes", 1768:"Glo", 1769:"Glo Clothing", 1770:"Glo Jeans", 1771:"Glo-Brite", 1772:"GloFish", 1773:"GloMinerals", 1774:"Global", 1775:"Global Holdings", 1776:"Globe", 1777:"Gloria Vanderbilt", 1778:"Go Gear", 1779:"GoGroove Pal", 1780:"GoPro", 1781:"Goddess", 1782:"Godinger", 1783:"Gold Bond", 1784:"Gold Bug", 1785:"Gold's Gym", 1786:"Goldberger Co", 1787:"GoldieBlox", 1788:"GolfBuddy", 1789:"Goliath Games", 1790:"Good Cook", 1791:"Good Technology", 1792:"Goodie Two Sleeves", 1793:"Goody", 1794:"Goodyear", 1795:"Gossard", 1796:"Gossip Girl", 1797:"Got 2B", 1798:"Gotta Flurt", 1799:"Gottex", 1800:"Goyard", 1801:"Grace", 1802:"Grace Elements", 1803:"Grace In LA", 1804:"Grace Karin", 1805:"Gracia", 1806:"Graco", 1807:"Graham & Spencer", 1808:"Grand Slam", 1809:"Grane", 1810:"Grannick's", 1811:"Grant Thomas", 1812:"Grasslands Road", 1813:"Great Expectations", 1814:"Great Northwest", 1815:"GreatNeck®", 1816:"Green Apple", 1817:"Green Toys", 1818:"Greenies", 1819:"Greenleaf Dollhouses", 1820:"Greenlight Collectibles", 1821:"Greg Norman", 1822:"Gregory", 1823:"Griffin", 1824:"Groovy Toys", 1825:"Grreat Choice", 1826:"Guard Master", 1827:"Gucci", 1828:"Guerlain", 1829:"Guess by Marciano", 1830:"Guidecraft", 1831:"Guinness", 1832:"Gund", 1833:"Gunne Sax", 1834:"Guy Harvey", 1835:"Gymboree", 1836:"Gymshark", 1837:"Gypsy", 1838:"Gypsy 05", 1839:"H&M", 1840:"HDE", 1841:"HEAD", 1842:"HERMES", 1843:"HOBO INTERNATIONAL", 1844:"HOT MILK", 1845:"HP", 1846:"HTC", 1847:"HUF", 1848:"HUGO", 1849:"HYDRAULIC", 1850:"Haband", 1851:"Haggar", 1852:"Haier", 1853:"Hailey Logan", 1854:"Hale Bob", 1855:"Hallmark", 1856:"Halo", 1857:"Halogen", 1858:"Halston", 1859:"Halston Heritage", 1860:"Hamilton", 1861:"Hamilton Beach", 1862:"Hamilton Collection", 1863:"Hamilton Electronics", 1864:"Hammer", 1865:"Hampshire Studio", 1866:"Hanes", 1867:"Hang Ten", 1868:"Hanky Panky", 1869:"Hanna Anderson", 1870:"Hanna Andersson", 1871:"Hannspree", 1872:"Hape", 1873:"Happy Chic Baby by Jonathan Adler", 1874:"Happy Socks", 1875:"Harajuku Lovers", 1876:"Harbor Bay", 1877:"Hard Candy", 1878:"Hard Candy Cosmetics", 1879:"Hard Rock Cafe", 1880:"Hard Tail", 1881:"Hardwood Classics", 1882:"Harley-Davidson", 1883:"Harlowe and Graham", 1884:"Harman Kardon", 1885:"Harmont & Blaine", 1886:"Harold's", 1887:"HarperCollins Publishers", 1888:"Harrington", 1889:"Harriton", 1890:"Hart & Huntington", 1891:"Hartstrings", 1892:"Hartz", 1893:"Harvard Square", 1894:"Harve Benard", 1895:"Harvey's Seatbelt Bag", 1896:"Hasbro", 1897:"Hasbro Games", 1898:"Hauck", 1899:"Haute Hippie", 1900:"Havaianas", 1901:"Hawk", 1902:"Hawke & Co", 1903:"Hayabusa", 1904:"Haynes", 1905:"Hazel", 1906:"Head & Shoulders", 1907:"Heart Soul", 1908:"HeartSoul", 1909:"Hearthside Collection", 1910:"Hedstrom", 1911:"Heelys", 1912:"Hein Werner", 1913:"Heinz", 1914:"Heirloom Collectibles", 1915:"Helix", 1916:"Hello Kitty", 1917:"Helly Hansen", 1918:"Helmut Lang", 1919:"Henckels", 1920:"Henri Bendel", 1921:"Henry Grethel", 1922:"Herbal Essences", 1923:"Heritage", 1924:"Heritage 1981", 1925:"Hermès", 1926:"Herschel Supply Company", 1927:"Hershey", 1928:"Herve Leger", 1929:"Hi-Tec", 1930:"Hickey Freeman", 1931:"High Sierra", 1932:"Highlander", 1933:"Highway Jeans", 1934:"Hikari", 1935:"Hilfiger Denim", 1936:"Hill's Ideal Balance", 1937:"Hill's Science Diet", 1938:"Hillard & Hanson", 1939:"Himalayan", 1940:"Hind", 1941:"Hint", 1942:"Hipanema", 1943:"Hippie", 1944:"Hippie Rose", 1945:"Hitachi", 1946:"HoMedics", 1947:"Hobie", 1948:"Hohner Kids", 1949:"Hoka One", 1950:"Holden", 1951:"Hollister", 1952:"Hollister Co.", 1953:"Holloway", 1954:"Home Depot", 1955:"Home Interiors", 1956:"Home-it", 1957:"Honda", 1958:"Honey Punch", 1959:"Honeydew Intimates", 1960:"Honeywell", 1961:"Honolua", 1962:"Honora", 1963:"Hoover", 1964:"Horizon Organic", 1965:"Horny Toad", 1966:"Hot Chillys", 1967:"Hot Kiss", 1968:"Hot Sox", 1969:"Hot Topic", 1970:"Hot Wheels", 1971:"Hourglass", 1972:"Hourglass Cosmetics", 1973:"House of Harlow 1960", 1974:"Hoverboard", 1975:"Howard Miller", 1976:"Howe", 1977:"Hublot", 1978:"Huda Beauty", 1979:"Hudson", 1980:"Hudson Baby", 1981:"Hudson Jeans", 1982:"Hue", 1983:"Huffy", 1984:"Hugga Bebe", 1985:"Huggies", 1986:"Huggies Goodnites", 1987:"Huggies Little Movers", 1988:"Huggies Little Snugglers", 1989:"Huggies Little Swimmers", 1990:"Huggies One & Done", 1991:"Huggies Pull-Ups", 1992:"Huggies Snug & Dry", 1993:"Hugo Boss", 1994:"Hula Honey", 1995:"Hunt Club", 1996:"Hunter", 1997:"Hunter Boots", 1998:"Hurley", 1999:"Hush Puppies", 2000:"Hustler", 2001:"Hybrid", 2002:"Hyland's", 2003:"Hype", 2004:"Hypertech®", 2005:"I Heart Ronson", 2006:"I LOVE NEW YORK by Bond No. 9", 2007:"I-Jam", 2008:"I-Play", 2009:"IBM", 2010:"IKEA", 2011:"IMAN", 2012:"INC International Concepts", 2013:"INHABIT", 2014:"ION Audio", 2015:"IRO", 2016:"ISIS", 2017:"IT Cosmetics", 2018:"IZOD", 2019:"Ice", 2020:"Ice Clothing", 2021:"Icebreaker", 2022:"Icon", 2023:"Ideology", 2024:"Igloo", 2025:"Illamasqua", 2026:"Illesteva", 2027:"Illuminations", 2028:"Imaginarium", 2029:"Imagine", 2030:"Imagine Baby", 2031:"Imaginext", 2032:"Imperial", 2033:"Imperial Motion", 2034:"Impossible", 2035:"Impressions", 2036:"In Bloom by Jonquil", 2037:"InCharacter Costumes", 2038:"Inc", 2039:"Incase Designs", 2040:"Incipio", 2041:"Indah", 2042:"Independent", 2043:"Indigo Brands", 2044:"Indigo Moon", 2045:"Indigo Palms", 2046:"Industrial Cotton", 2047:"Infantino", 2048:"Infinity", 2049:"Infinity Raine", 2050:"Ingear", 2051:"Ingenuity", 2052:"Ingersoll-Rand", 2053:"Inglesina", 2054:"Inglot", 2055:"Ingrid & Isabel", 2056:"Inner Beauty", 2057:"Innovative Designs", 2058:"Inov-", 2059:"Insignia", 2060:"Inspire", 2061:"Instantly Ageless", 2062:"Intel", 2063:"InterDesign", 2064:"Intex", 2065:"Intrigue", 2066:"Investments", 2067:"Invicta", 2068:"Invicta Watch Group", 2069:"Iolani", 2070:"Iris", 2071:"Irish Setter", 2072:"Irish Spring", 2073:"Iron Fist", 2074:"Iron Maiden", 2075:"Iron Man", 2076:"Ironman", 2077:"Isaac Mizrahi", 2078:"Isabel Marant", 2079:"Isabella Oliver", 2080:"Isabella Rodriguez", 2081:"Island Escape", 2082:"Island Shores", 2083:"Issa de' mar", 2084:"Issey Miyake", 2085:"It Figures", 2086:"It's Our Time", 2087:"Itzy Ritzy", 2088:"Ivanka Trump", 2089:"Ivivva", 2090:"Ivory Ella", 2091:"Ivy & Blu", 2092:"Ivy Crew", 2093:"Iz Byer", 2094:"J & Company", 2095:"J Brand", 2096:"J G Hook", 2097:"J Kara", 2098:"J Workshop", 2099:"J&L", 2100:"J. America", 2101:"J. Crew", 2102:"J. Ferrar", 2103:"J. Jill", 2104:"J. McLaughlin", 2105:"J.Crew", 2106:"J.Lo by Jennifer Lopez", 2107:"JACO", 2108:"JAG", 2109:"JBL", 2110:"JC Toys", 2111:"JCPenney", 2112:"JERZEES", 2113:"JET Performance", 2114:"JF J. Ferrar", 2115:"JJ Cole Collections", 2116:"JM Collection", 2117:"JNCO", 2118:"JS Boutique", 2119:"JS Collections", 2120:"JTB", 2121:"JUMP", 2122:"JVC", 2123:"Jac Vanek", 2124:"Jack Daniels", 2125:"Jack LaLanne", 2126:"Jack Rogers", 2127:"Jack Spade", 2128:"Jack by BB Dakota", 2129:"Jaclyn Smith", 2130:"Jacqueline Ferrar", 2131:"Jacques Moret", 2132:"Jada Toys", 2133:"Jade", 2134:"Jafra", 2135:"Jag Jeans", 2136:"Jagermeister", 2137:"Jakks Pacific", 2138:"Jamaica Bay", 2139:"Jamaica Jaxx", 2140:"Jamberry", 2141:"Jambu", 2142:"James Avery", 2143:"James Jeans", 2144:"James Perse", 2145:"Jamie Sadock", 2146:"Jams World", 2147:"JanSport", 2148:"Jane Ashley", 2149:"Jane Iredale", 2150:"Janie and Jack", 2151:"Jantzen", 2152:"Jason Maxwell", 2153:"Jason Wu", 2154:"Jay at Play", 2155:"Jazwares", 2156:"Jean Paul Gaultier", 2157:"Jeanne Pierre", 2158:"Jeep", 2159:"Jeffery Campbell", 2160:"Jeffree Star Cosmetics", 2161:"Jeffrey Campbell", 2162:"Jelly Belly", 2163:"Jen's Pirate Booty", 2164:"Jenni by Jennifer Moore", 2165:"Jennifer Lopez", 2166:"Jennifer Moore", 2167:"Jenny Yoo", 2168:"Jensen", 2169:"Jeremy Scott x Adidas", 2170:"Jergens", 2171:"Jessica Howard", 2172:"Jessica McClintock", 2173:"Jessica Simpson", 2174:"Jezebel", 2175:"Jil Sander", 2176:"Jim Dunlop", 2177:"Jim Shore", 2178:"Jimmy Choo", 2179:"Jinx", 2180:"Jo Malone", 2181:"Joan & David", 2182:"Joan Rivers", 2183:"Joby", 2184:"Jockey", 2185:"Jodi Kristopher", 2186:"Joe & Elle", 2187:"Joe Benbasset", 2188:"Joe Fresh", 2189:"Joe Rocket", 2190:"Joe's Jeans", 2191:"John Ashford", 2192:"John Blair", 2193:"John Deere", 2194:"John Frieda", 2195:"John Hardy", 2196:"John Henry", 2197:"John Meyer", 2198:"John Paul Pet", 2199:"John Paul Richard", 2200:"John Roberts", 2201:"John Varvatos", 2202:"John Varvatos Collection", 2203:"John Varvatos Star USA", 2204:"Johnny Martin", 2205:"Johnny Was", 2206:"Johnson", 2207:"Johnson & Johnson", 2208:"Johnston & Murphy", 2209:"Joico", 2210:"Joie", 2211:"Jollychic.com", 2212:"Jolt", 2213:"Jolyn Clothing", 2214:"Jonathan Adler", 2215:"Jonathan Martin", 2216:"Jones Jeans", 2217:"Jones New York", 2218:"Jones Sport", 2219:"Jonesport", 2220:"Joovy", 2221:"Jordache", 2222:"Jordan", 2223:"Jordan Craig", 2224:"Jordans", 2225:"Jos. A. Bank", 2226:"Josef Seibel", 2227:"Joseph A", 2228:"Joseph Abboud", 2229:"Joseph Ribkoff", 2230:"Josephine Chaus", 2231:"Josie", 2232:"Josie Maran", 2233:"Josie Natori", 2234:"Jostar", 2235:"Jou Jou", 2236:"Joujou", 2237:"Joules", 2238:"Journee Collection", 2239:"Journey Girls", 2240:"Jovani", 2241:"Joy Mangano", 2242:"Joyce Chen", 2243:"Joyce Leslie", 2244:"Joyrich", 2245:"JuJu", 2246:"Judith March", 2247:"Juiceman", 2248:"Juicy Couture", 2249:"Julep", 2250:"Julie Brown", 2251:"Juliette Has a Gun", 2252:"Jump Apparel", 2253:"Jumping Beans", 2254:"Jun Planning", 2255:"Junk Food", 2256:"Junk Food Clothing", 2257:"Just Born", 2258:"Just Cavalli", 2259:"Just Like Home", 2260:"Just Love", 2261:"Just My Size", 2262:"Just Play", 2263:"Just USA", 2264:"JustFab", 2265:"Justice", 2266:"Justin", 2267:"Justin Boots", 2268:"K & W", 2269:"K Advantix", 2270:"K&N Engineering", 2271:"K'NEX", 2272:"K-SWISS", 2273:"K. Bell", 2274:"KAPLAN MD", 2275:"KDLINKS", 2276:"KEEN", 2277:"KIINI", 2278:"KIKO MILANO", 2279:"KMS California", 2280:"KONG", 2281:"KR3W", 2282:"KRE-O", 2283:"KRW", 2284:"KUT from the Kloth", 2285:"KY", 2286:"Kae Argatherapie", 2287:"Kaii", 2288:"Kailijumei Lipsticks", 2289:"Kala", 2290:"Kalencom", 2291:"Kalorik", 2292:"Kamik", 2293:"Kanebo", 2294:"Kanga Care", 2295:"Kangol", 2296:"Kappa", 2297:"Kardashian Kollection", 2298:"Karen Kane", 2299:"Karen Millen", 2300:"Karen Neuburger", 2301:"Karen Scott", 2302:"Karl Lagerfeld", 2303:"Karma", 2304:"Kasper", 2305:"Kat Von D", 2306:"Kat Von D Beauty", 2307:"Kate Aspen", 2308:"Kate Hill", 2309:"Kate Landry", 2310:"Kate Somerville", 2311:"Kate Spade", 2312:"Kathie Lee", 2313:"Kathy Lee", 2314:"Kathy Van Zeeland", 2315:"Katia", 2316:"Katie B.", 2317:"Katydid", 2318:"Kawasaki", 2319:"Kay Jewelers", 2320:"Kay Unger", 2321:"Kaytee", 2322:"Kc Collections", 2323:"Keds", 2324:"Keekaroo", 2325:"Keepsake the Label", 2326:"Kelly & Katie", 2327:"Kelsi Dagger", 2328:"Kelty", 2329:"Kenar", 2330:"Kendall & Kylie", 2331:"Kendra Scott", 2332:"Kenmore", 2333:"Kenneth Cole", 2334:"Kenneth Cole New York", 2335:"Kenneth Cole Reaction", 2336:"Kenneth Jay Lane", 2337:"Kensie", 2338:"Kensington", 2339:"Kenwood", 2340:"Kenzo", 2341:"Keranique", 2342:"Kerusso", 2343:"Keurig", 2344:"Key", 2345:"Keys", 2346:"Khombu", 2347:"Kibbles 'n Bits", 2348:"Kickee Pants", 2349:"Kid Galaxy", 2350:"KidCo", 2351:"KidKraft", 2352:"Kids Headquarters", 2353:"Kids Only", 2354:"Kids Preferred", 2355:"Kidsline", 2356:"Kiehl's", 2357:"Kiinde", 2358:"Kikit", 2359:"Kikkerland", 2360:"Kim Rogers", 2361:"Kimchi Blue", 2362:"KinderGlo", 2363:"King Baby Studio", 2364:"Kingston", 2365:"Kipling", 2366:"Kiplling", 2367:"Kirkland Signature", 2368:"Kirra", 2369:"Kische", 2370:"Kiss Me", 2371:"Kiss My Face", 2372:"KitchenAid", 2373:"Kiton", 2374:"Kiyonna", 2375:"Kleenex", 2376:"Klipsch", 2377:"Klorane", 2378:"Klutz", 2379:"Knights Sportswear", 2380:"Knightsbridge", 2381:"Knocker", 2382:"Koala Baby", 2383:"Koala Baby Boutique", 2384:"Koala Kids", 2385:"Kodak", 2386:"Koh Koh", 2387:"Kolcraft", 2388:"Koman", 2389:"Konami", 2390:"Kooba", 2391:"Koolatron", 2392:"Kopari", 2393:"Koret", 2394:"Kork-Ease", 2395:"Korres", 2396:"Koss", 2397:"Kotex", 2398:"Krazy Kat", 2399:"Krew", 2400:"Krista Lee", 2401:"Kristen Blake", 2402:"Krups", 2403:"Krush", 2404:"Kryolan", 2405:"Kryptonics", 2406:"Kuhl", 2407:"Kurgo", 2408:"Kurio", 2409:"Kyjen", 2410:"Kylie Cosmetics", 2411:"Kyocera", 2412:"Kyodan", 2413:"L'Oreal", 2414:"L'eggs", 2415:"L*SPACE", 2416:"L.A. Blues", 2417:"L.A. Gear", 2418:"L.A.M.B.", 2419:"L.L. Bean", 2420:"LA Hearts", 2421:"LA Idol", 2422:"LA Splash", 2423:"LAT", 2424:"LAmade", 2425:"LC Lauren Conrad", 2426:"LEGO", 2427:"LF", 2428:"LG", 2429:"LK Bennett", 2430:"LOFT", 2431:"LRG", 2432:"La Blanca", 2433:"La Femme", 2434:"La Girl", 2435:"La Perla", 2436:"La Prairie", 2437:"La Redoute", 2438:"La Roche-Posay", 2439:"La Senza", 2440:"La Sportiv", 2441:"LaCie", 2442:"LaMarque", 2443:"LaROK", 2444:"Lacoste", 2445:"Lady Hagen", 2446:"Lady Noiz", 2447:"Lafayette  New York", 2448:"Lafayette 148 New York", 2449:"Laguna Beach", 2450:"Laguna Beach Jeans", 2451:"Lakai", 2452:"Lalaloopsy", 2453:"Lamaze", 2454:"Lambs & Ivy", 2455:"Lampe Berger", 2456:"Lancome", 2457:"Lancôme", 2458:"Landau", 2459:"Lands' End", 2460:"Lane Bryant", 2461:"Lansinoh", 2462:"Lanvin", 2463:"Lapis", 2464:"Laredo", 2465:"Larry Levine", 2466:"Lasko", 2467:"Last Kiss", 2468:"Latique", 2469:"Laundry by Design", 2470:"Laundry by Shelli Segal", 2471:"Laura Ashley", 2472:"Laura Geller", 2473:"Laura Mercier", 2474:"Laura Scott", 2475:"Laurel Burch", 2476:"Lauren Conrad", 2477:"Lauren Moshi", 2478:"Lauren Ralph Lauren", 2479:"Lavazza", 2480:"LawMan", 2481:"Lazy Oaf", 2482:"Le Creuset", 2483:"Le Mystere", 2484:"Le Suit", 2485:"LeSportsac", 2486:"Leachco", 2487:"Leading Lady", 2488:"Leap Frog", 2489:"Learning Curve", 2490:"Learning Resources", 2491:"Leather Gallery", 2492:"Lee", 2493:"Leith", 2494:"Lela Rose", 2495:"Lennie for Nina Leonard", 2496:"Lenovo", 2497:"Lenox", 2498:"Leota", 2499:"Leslie Fay", 2500:"Letarte", 2501:"Levana", 2502:"Level", 2503:"Levi Strauss & Co.", 2504:"Levi's®", 2505:"Lexar", 2506:"Lexmark", 2507:"Lia Sophia", 2508:"Liberty Apparel", 2509:"Liberty of London", 2510:"Liebeskind", 2511:"Life Is Good", 2512:"Life Stride", 2513:"LifeProof", 2514:"Lil Critters", 2515:"Lil' Joey", 2516:"Liliana", 2517:"Lilli Ann", 2518:"Lillie Rubin", 2519:"Lilly Pulitzer", 2520:"Lilly Pulitzer for Target", 2521:"Lilu", 2522:"Lily Rose", 2523:"Lily White", 2524:"Lilyette", 2525:"Lime Crime", 2526:"Lime Crime Cosmetics", 2527:"Limited Too", 2528:"Lincoln Logs", 2529:"Linea Donatella", 2530:"Linea Pelle", 2531:"Linksys", 2532:"Lionel", 2533:"Lions Gate", 2534:"Lip Service", 2535:"LipSense", 2536:"Lipstick Bandit", 2537:"Lipstick Boutique", 2538:"Lipstick Queen", 2539:"Lipsy", 2540:"Liquid Blue", 2541:"Lisa Maree", 2542:"Listerine", 2543:"Lite Brix", 2544:"Litter Genie", 2545:"Little Gifts", 2546:"Little Lass", 2547:"Little Me", 2548:"Little Me Childrens Wear", 2549:"Little Mistress", 2550:"Little Pim", 2551:"Little Tikes", 2552:"Little Trees", 2553:"Littlest Pet Shop", 2554:"Live a Little", 2555:"Liverpool Jeans Company", 2556:"Living Dead Souls", 2557:"Livinglibations", 2558:"Lixit", 2559:"Liz & Co.", 2560:"Liz Baker", 2561:"Liz Claiborne", 2562:"Liz Lange", 2563:"Liz Lange for Target", 2564:"Lladro", 2565:"Lodge", 2566:"Lodis", 2567:"Loeffler Randall", 2568:"Loewe", 2569:"Logitech", 2570:"Lojas Riachuelo", 2571:"Lokai", 2572:"Lola", 2573:"Lole", 2574:"Lolita", 2575:"Lolli swim", 2576:"Lolly Wolly Doodle", 2577:"London Fog", 2578:"London Jeans", 2579:"London Times", 2580:"Longaberger", 2581:"Longchamp", 2582:"Longchamp Paris", 2583:"Longines", 2584:"Longitude", 2585:"Looney Tunes", 2586:"Lorac", 2587:"Lord & Taylor", 2588:"Lorna Jane", 2589:"Lot", 2590:"Lou & Grey", 2591:"Loudmouth Golf", 2592:"Louis Garneau", 2593:"Louis Raphael", 2594:"Louis Vuitton", 2595:"Louisville Slugger", 2596:"Loungefly", 2597:"Love By Design", 2598:"Love Culture", 2599:"Love Moschino", 2600:"Love Riche", 2601:"Love Stitch", 2602:"Loveappella", 2603:"Lovers + Friends", 2604:"Lowa", 2605:"LuLaRoe", 2606:"Luca Luca", 2607:"LucasArts Entertainment", 2608:"Lucca", 2609:"Lucca Couture", 2610:"Lucchese", 2611:"Lucien Piccard", 2612:"Lucky Brand", 2613:"Lucky Brand Jeans", 2614:"Lucky Dog", 2615:"Lucy Activewear", 2616:"Lucy Brand", 2617:"Lucy Love", 2618:"Lugz", 2619:"Luichiny", 2620:"Luli Fama", 2621:"Lullaby Earth", 2622:"Lulu", 2623:"Lulu Frost", 2624:"Lulu Guinness", 2625:"Lulu Townsend", 2626:"Lulu's", 2627:"Lululemon", 2628:"LulyBoo", 2629:"Lumisource", 2630:"Lunaire", 2631:"Lush", 2632:"Lush Clothing", 2633:"Luvable Friends", 2634:"Luvs", 2635:"Lux", 2636:"Lyle & Scott", 2637:"Lyons / Hit Ent.", 2638:"Lyons Hit Entertainment", 2639:"Lysol", 2640:"L’Occitane", 2641:"M", 2642:"M Missoni", 2643:"M by Marc Bouwer", 2644:"M&M", 2645:"M&O KNITS", 2646:"M.I.A.", 2647:"MAC", 2648:"MAC Cosmetics", 2649:"MAKE UP FOR EVER", 2650:"MAM", 2651:"MAM Baby", 2652:"MANDALYNN", 2653:"MARC BY MARC JACOBS", 2654:"MARC JACOBS", 2655:"MATRIX", 2656:"MCM", 2657:"MCM Worldwide", 2658:"MEK", 2659:"MGA Entertainment", 2660:"MGM", 2661:"MIA", 2662:"MICHELE", 2663:"MICHI by Michelle Watson", 2664:"MILK MAKEUP", 2665:"MINIMALE ANIMALE", 2666:"MINKPINK", 2667:"MISS SIXTY", 2668:"MIU MIU", 2669:"MIkoh", 2670:"ML Monique Lhuillier", 2671:"MLB", 2672:"MM Couture", 2673:"MMA Elite", 2674:"MOTHER", 2675:"MSI", 2676:"MSK", 2677:"MV Sport", 2678:"MZ Wallace", 2679:"Maaji", 2680:"Mac & Jac", 2681:"Mac Cosmetic", 2682:"Mac Duggal", 2683:"Machine", 2684:"Mackage", 2685:"Mackintosh", 2686:"Maclaren", 2687:"Macy's", 2688:"Mad Engine", 2689:"Madam C.J. Walker Beauty Culture", 2690:"Madame Alexander", 2691:"Madden Girl", 2692:"Madeline Stuart", 2693:"Madewell", 2694:"Madison", 2695:"Madison Leigh", 2696:"Maeve", 2697:"Mag 1", 2698:"Mag-Float", 2699:"Magaschoni", 2700:"Magazine", 2701:"Magellan®", 2702:"Magformers", 2703:"Maggie Barnes", 2704:"Maggie Sottero", 2705:"Maggy London", 2706:"Magic", 2707:"Magic Bullet", 2708:"Magic Sports", 2709:"Magicsuit", 2710:"Magnavox", 2711:"Magnolia", 2712:"Magnum", 2713:"Maidenform", 2714:"Mainstreet Blues", 2715:"Maison Goyard", 2716:"Maison Jules", 2717:"Maison Margiela", 2718:"Maison Martin Margiela", 2719:"Maisto", 2720:"Majestic", 2721:"Makeup Eraser", 2722:"Makeup Forever", 2723:"Makeup Geek", 2724:"Malibu", 2725:"Mally Beauty", 2726:"Mamas & Papas", 2727:"Mamia", 2728:"Mammut", 2729:"Mandalay", 2730:"Manduka", 2731:"Manga", 2732:"Mango", 2733:"Manhattan Toy", 2734:"Manic Panic", 2735:"Manolo Blahnik", 2736:"Mansur Gavriel", 2737:"Mar Y Sol", 2738:"Mara Hoffman", 2739:"Marc Ecko", 2740:"Marc Fisher", 2741:"Marc New York", 2742:"Marcelo Burlon County of Milan", 2743:"Marciano", 2744:"Marco Santi", 2745:"Margarita", 2746:"Margaritaville", 2747:"Marika", 2748:"Marimekko", 2749:"Marina", 2750:"Marineland", 2751:"Mario Badescu Skin Care", 2752:"Marisa Christina", 2753:"Marks and Spencer", 2754:"Marmot", 2755:"Marni", 2756:"Marshall", 2757:"Martha Stewart", 2758:"Martin Margiela", 2759:"Marucci", 2760:"Marvel", 2761:"Marvel Universe", 2762:"Mary Grace Swim", 2763:"Mary Kay", 2764:"Marysia Swim", 2765:"Mason's", 2766:"Masquerade", 2767:"Masters", 2768:"Matchbox", 2769:"Material Girl", 2770:"Maternal America", 2771:"Matix", 2772:"Matt Bernson", 2773:"Mattel®", 2774:"Matty M", 2775:"Maui & Sons", 2776:"Maui Jim", 2777:"Maurices", 2778:"Maverik", 2779:"Mavi Jeans", 2780:"Max & Mia", 2781:"Max Azria", 2782:"Max Edition", 2783:"Max Factor", 2784:"Max Mara", 2785:"Max Studio", 2786:"Max and Cleo", 2787:"Maxell", 2788:"Maxi-Cosi", 2789:"Maxima", 2790:"Maxine of Hollywood", 2791:"Maxtor", 2792:"May Queen", 2793:"Maya", 2794:"Maybelline", 2795:"Maybelline New York", 2796:"Mayoral", 2797:"McDavid", 2798:"McFarlane Sports", 2799:"McFarlane Toys", 2800:"McGuire Denim", 2801:"McKenzie", 2802:"MeDusa", 2803:"Mecca", 2804:"Mechanix Wear®", 2805:"Medela", 2806:"Mederma", 2807:"Mediasonic", 2808:"Mega Bloks", 2809:"Mega Brands", 2810:"Mega Hobby", 2811:"Mehron", 2812:"Mel by Melissa", 2813:"Melannco", 2814:"Melissa", 2815:"Melissa & Doug", 2816:"Melissa McCarthy", 2817:"Melissa Odabash", 2818:"Melitta", 2819:"Melt Cosmetics", 2820:"Members Only", 2821:"Meme", 2822:"Memorex", 2823:"Men's Warehouse", 2824:"Meow Mix", 2825:"Mephisto", 2826:"Mercer", 2827:"Mercer Street Studio", 2828:"Merle Norman", 2829:"Merle Norman Cosmetics", 2830:"Mermaid Maternity", 2831:"Merona", 2832:"Merrell", 2833:"Metal Mulisha", 2834:"Metaphor", 2835:"Method", 2836:"Metrokane", 2837:"Mezco", 2838:"Miami Ink", 2839:"Michael Antonio", 2840:"Michael Aram", 2841:"Michael Brandon", 2842:"Michael Kors", 2843:"Michael Simon", 2844:"Michael Stars", 2845:"Michaelangelo", 2846:"Miche", 2847:"Micro Innovations", 2848:"Micros", 2849:"Microsoft", 2850:"Middleton Doll Company", 2851:"Midland", 2852:"Midtronics", 2853:"Midway", 2854:"Midwest", 2855:"Mighty Fine", 2856:"Mika Yoga Wear", 2857:"Mikasa", 2858:"Mikimoto", 2859:"Milani", 2860:"Milani Cosmetics", 2861:"Milano", 2862:"Miley Cyrus", 2863:"Millau", 2864:"Millenium", 2865:"Miller", 2866:"Miller's Forge", 2867:"Milly", 2868:"Mimi", 2869:"Mimi Chica", 2870:"Mimi Maternity", 2871:"Minecraft", 2872:"Mineral Fusion", 2873:"Mini Boden", 2874:"Mini Melissa", 2875:"Minnetonka", 2876:"Minnie Mouse", 2877:"Minnie Rose", 2878:"Miraclesuit®", 2879:"Mirro", 2880:"Misfits", 2881:"Mishka", 2882:"Miskeen", 2883:"Miss Chievous", 2884:"Miss Dorby", 2885:"Miss Elaine", 2886:"Miss Me", 2887:"Missguided", 2888:"Mission", 2889:"Missoni", 2890:"Mitchell & Ness", 2891:"Mitchum", 2892:"Mitre", 2893:"Mitsubishi", 2894:"Mix Nouveau", 2895:"Mixit", 2896:"Mizuno", 2897:"Moa Moa", 2898:"Mobi Technologies", 2899:"Mobil™", 2900:"Moda International", 2901:"Modcloth", 2902:"Modena", 2903:"Modern Amusement", 2904:"Modern Culture", 2905:"Modern Soul", 2906:"Mojo Moxy", 2907:"Mommy's Helper", 2908:"Monarchy", 2909:"Moncler", 2910:"Monet", 2911:"Mongoose", 2912:"Monistat", 2913:"Monopoly", 2914:"Monoprice", 2915:"Monoreno", 2916:"Monster", 2917:"Monster Cable", 2918:"Monteau", 2919:"Monterey Club", 2920:"Moon Boot", 2921:"Moon Collection", 2922:"Moose Mountain", 2923:"Moose Toys", 2924:"Mopar", 2925:"Mopas", 2926:"Morgan", 2927:"Morgan & Co.", 2928:"Morgan Taylor", 2929:"Mori Lee", 2930:"Morphe Cosmetics", 2931:"Moschino", 2932:"Mossimo", 2933:"Mossimo Supply Co.", 2934:"Mossy Oak", 2935:"Motel", 2936:"Motel Rocks", 2937:"Moth", 2938:"Motherhood", 2939:"Motherhood Maternity", 2940:"Motor Trend", 2941:"Motorcycle Stuff", 2942:"Motorola", 2943:"Motorola Consumer Radios", 2944:"Motto", 2945:"Moulinette Soeurs", 2946:"Mountain Buggy", 2947:"Mountain Hardwear", 2948:"Mountain Lake", 2949:"Movado", 2950:"Moving Comfort", 2951:"Mr. Coffee", 2952:"Mr. Tea", 2953:"Mud Pie", 2954:"Mudd", 2955:"Mueller", 2956:"Muk Luks", 2957:"Mulberry", 2958:"Multipet", 2959:"Munchkin", 2960:"Murad", 2961:"Mural", 2962:"Murano", 2963:"Muse", 2964:"Music Little People", 2965:"Musician's Gear", 2966:"Mustard Seed", 2967:"Mustela", 2968:"My Baby Sam", 2969:"My Brest Friend", 2970:"My Little Pony", 2971:"My Michelle", 2972:"Mycra Pac Designer Wear", 2973:"Mylec", 2974:"Mystic", 2975:"Myths", 2976:"N.Y.L.A.", 2977:"NASCAR", 2978:"NBA", 2979:"NBN Gear", 2980:"NCAA", 2981:"NEC Technologies", 2982:"NECA", 2983:"NEXXUS", 2984:"NFL", 2985:"NHL", 2986:"NIC+ZOE", 2987:"NIX", 2988:"NO-AD", 2989:"NUDESTIX", 2990:"NVIDIA", 2991:"NY Collection", 2992:"NY Jeans", 2993:"NYC", 2994:"NYDJ", 2995:"NYGARD", 2996:"NYX", 2997:"NYX Cosmetics", 2998:"Nabisco Variety", 2999:"Nair", 3000:"Naked & Famous", 3001:"Naked Zebra", 3002:"Nambe", 3003:"Nana", 3004:"Nancy Ganz", 3005:"Nanette Lepore", 3006:"NanoBlock", 3007:"Naomi & Nicole", 3008:"Narciso Rodriguez", 3009:"Nars", 3010:"Nasty Gal", 3011:"Natasha Denona", 3012:"Nathan", 3013:"National Geographic", 3014:"Native", 3015:"Natori", 3016:"Natural Expressions", 3017:"Natural Fitness", 3018:"Naturalizer", 3019:"Nature's Variety Instinct", 3020:"Nautica", 3021:"NeatNik", 3022:"Necessary Clothing", 3023:"Necessary Objects", 3024:"Neewer", 3025:"Neff", 3026:"Neff Headwear", 3027:"Neighborhood", 3028:"Neil Allyn", 3029:"Neiman Marcus", 3030:"Neosporin", 3031:"Nerf", 3032:"Nespresso", 3033:"Nest", 3034:"Netgear", 3035:"Neutrogena", 3036:"New Arrivals", 3037:"New Balance", 3038:"New Bright", 3039:"New Directions", 3040:"New Era", 3041:"New Look", 3042:"New York & Company", 3043:"New York Color", 3044:"Newport News", 3045:"Next", 3046:"Next Level", 3047:"Next Level Apparel", 3048:"Nfinity", 3049:"Nick & Nora", 3050:"Nick Jr.", 3051:"Nickelodeon", 3052:"Nicklaus", 3053:"Nicola", 3054:"Nicole Lee", 3055:"Nicole Miller", 3056:"Nicole by Nicole Miller", 3057:"Night Moves", 3058:"Nike", 3059:"Nike Golf", 3060:"Nikon", 3061:"Nina", 3062:"Nina Leonard", 3063:"Nina Ricci", 3064:"Nine & Co.", 3065:"Nine West", 3066:"Ninety", 3067:"Ninja", 3068:"Nintendo", 3069:"Niteline", 3070:"Nivea", 3071:"Nixon", 3072:"No Boundaries", 3073:"No Fear", 3074:"No7 Make-up", 3075:"NoJo", 3076:"Nobo", 3077:"Nokia", 3078:"Nollie", 3079:"Nook", 3080:"Nor Cal", 3081:"Nordstrom", 3082:"Nordstrom Baby", 3083:"Norelco", 3084:"Norma Kamali", 3085:"Norpro", 3086:"North End", 3087:"North States", 3088:"NorthCrest", 3089:"Northwest Territory", 3090:"Norton McNaughton", 3091:"Nosefrida", 3092:"Nostalgia", 3093:"Nostalgia Electrics", 3094:"Not Rated", 3095:"Not Your Mother's", 3096:"Notations", 3097:"Nothing But Love", 3098:"Notify", 3099:"Novella Royale", 3100:"Noxzema", 3101:"NuRoo", 3102:"Nubian Heritage", 3103:"Nuby", 3104:"Nudie Jeans", 3105:"Nuk", 3106:"Nunn Bush", 3107:"Nursery Fresh", 3108:"Nurture by Lamaze", 3109:"Nutrabella", 3110:"Nutri-Vet", 3111:"Nutro Max", 3112:"Nyko", 3113:"Nylabone", 3114:"O'Neill", 3115:"O.B.", 3116:"OGIO®", 3117:"OGX", 3118:"OLOOK", 3119:"OPI", 3120:"OULUOQI", 3121:"OXO", 3122:"OXO Tot", 3123:"Oakley", 3124:"Oasis", 3125:"Obagi", 3126:"Obermeyer", 3127:"Obey", 3128:"Obey Clothing", 3129:"Ocean Current", 3130:"Ocean Drive", 3131:"Ocean Pacific", 3132:"Ocra", 3133:"October Love", 3134:"Odille", 3135:"Odyssey", 3136:"Off-White", 3137:"Office Depot", 3138:"Oh Baby by Motherhood", 3139:"Oilily", 3140:"Oki", 3141:"Okie Dokie", 3142:"Olay", 3143:"Old Glory", 3144:"Old Navy", 3145:"Old Soles", 3146:"Old Spice", 3147:"Old Varsity Brand", 3148:"Ole Henricksen", 3149:"Oleg Cassini", 3150:"Olga", 3151:"Olian", 3152:"Olive & Oak", 3153:"Oliver Peoples", 3154:"Olivia + Joy", 3155:"Olivia Moon", 3156:"Olsenboye", 3157:"OluKai", 3158:"Olympus", 3159:"Omega", 3160:"Omega Paw", 3161:"Omron", 3162:"On The Fly", 3163:"On the Byas", 3164:"On-Stage Stands", 3165:"One Clothing", 3166:"One Industries", 3167:"One Step Up", 3168:"One Teaspoon", 3169:"One World", 3170:"Oneida", 3171:"Onitsuka Tiger", 3172:"Onkyo", 3173:"Only Hearts", 3174:"Only Mine", 3175:"Onque Casuals", 3176:"Onyx Nite", 3177:"Onzie", 3178:"Op", 3179:"Opening Ceremony", 3180:"Opti-Free", 3181:"Optimum", 3182:"Oral B", 3183:"Orbit Baby", 3184:"Oregon Scientific", 3185:"Organic Wear", 3186:"Original Penguin", 3187:"Origins", 3188:"Oris", 3189:"Orla Kiely", 3190:"Orly", 3191:"Orvis", 3192:"Oscar de la Renta", 3193:"Osh Kosh", 3194:"Osh Kosh B'gosh", 3195:"Oshkosh", 3196:"Osiris", 3197:"Osprey", 3198:"Oster", 3199:"OtterBox", 3200:"Ouai", 3201:"Ouidad", 3202:"OutBrook", 3203:"Outdoor Life", 3204:"Outdoor Research", 3205:"Outset Media", 3206:"Outward Hound", 3207:"Oxbow", 3208:"Oxford", 3209:"Oxford Golf", 3210:"P&N", 3211:"P'Kolino", 3212:"PAIGE", 3213:"PANDORA", 3214:"PASS PORT", 3215:"PAUL & JOE", 3216:"PBS", 3217:"PCA Skin", 3218:"PEZ", 3219:"PGA TOUR", 3220:"PIKOLINOS", 3221:"PING", 3222:"PINK", 3223:"PJ Salvage", 3224:"PNY", 3225:"POND'S", 3226:"PONY", 3227:"PROPPER", 3228:"PRPS", 3229:"PUMA", 3230:"PUR", 3231:"Pacific Sunwear", 3232:"Pacific Trail", 3233:"Pacific World Corp", 3234:"Paco", 3235:"Paco Rabanne", 3236:"Paige Denim", 3237:"Pal Zileri", 3238:"Palladium Boots", 3239:"Palm Beach", 3240:"Palm Grove", 3241:"Palmer's", 3242:"Pampered Chef", 3243:"Pampers", 3244:"Pampers Baby Dry", 3245:"Pampers Baby Fresh", 3246:"Pampers Cruisers", 3247:"Pampers Easy Ups", 3248:"Pampers Sensitive", 3249:"Pampers Splashers", 3250:"Pampers Swaddlers", 3251:"Panache", 3252:"Panasonic", 3253:"Panhandle Slim", 3254:"Pantene", 3255:"Papaya", 3256:"Paper Crane", 3257:"Paper Denim & Cloth", 3258:"Paper Magic", 3259:"Paper Mate", 3260:"Papillon", 3261:"Paprika", 3262:"Paradise Collection", 3263:"Paradise Found", 3264:"Paradise Kids", 3265:"Paradox", 3266:"Parallel", 3267:"Paramount", 3268:"Paris Blues", 3269:"Parish", 3270:"Park Designs", 3271:"Park Lane", 3272:"Parker", 3273:"Partners", 3274:"PartyLite", 3275:"Passport", 3276:"Pastry Shoe Collection", 3277:"Patagonia, Inc.", 3278:"Patek Philippe", 3279:"Patra", 3280:"Patricia Nash", 3281:"Patrizia Pepe", 3282:"Paul Frank", 3283:"Paul Fredrick", 3284:"Paul Jones", 3285:"Paul Mitchell", 3286:"Paula Deen", 3287:"Peaches", 3288:"Peanut Shell", 3289:"Peanuts", 3290:"Pearhead", 3291:"Pearl Izumi", 3292:"Pebble Beach", 3293:"Peck & Peck", 3294:"Pedialyte", 3295:"Pediasure", 3296:"Pedigree", 3297:"Peg Perego", 3298:"Pelican", 3299:"Pelle", 3300:"Pendleton", 3301:"Pendleton Woolen Mills", 3302:"Penguin", 3303:"Penn Fishing", 3304:"People's Liberation", 3305:"Pepe Jeans", 3306:"Peppa Pig", 3307:"Peppe Peluso", 3308:"Per Se", 3309:"Perception Sport", 3310:"Perceptions", 3311:"Perfect Equipment®", 3312:"Perfection", 3313:"Performance Tool", 3314:"Peripera", 3315:"Perler", 3316:"Perlina", 3317:"Permatex®", 3318:"Perricone MD", 3319:"Perry Ellis", 3320:"Persol", 3321:"Peruvian Connection", 3322:"Pet Gear", 3323:"Pet Life", 3324:"PetRageous Designs", 3325:"PetSafe", 3326:"PetSmart", 3327:"Peter Millar", 3328:"Peter Nygard", 3329:"Peter Thomas Roth", 3330:"Petit Tresor", 3331:"Petite Sophisticate", 3332:"Petmate", 3333:"Pets First", 3334:"Petunia Pickle Bottom", 3335:"Peugeot", 3336:"Pfaltzgraff", 3337:"Phat Farm", 3338:"Phat Farm/Baby Phat", 3339:"Phil & Teds", 3340:"Philips", 3341:"Phillips", 3342:"Phillips 66", 3343:"Philosophy", 3344:"Physicians Formula", 3345:"Physicians Prefer", 3346:"Piazza Sempione", 3347:"Piedmont", 3348:"Pier One", 3349:"Pierre Cardin", 3350:"Pierre Dumas", 3351:"Pilcro", 3352:"Pillow Pets", 3353:"Pillsbury", 3354:"Pilot", 3355:"Pinc Premium", 3356:"Pineapple Connection", 3357:"Pink Dolphin", 3358:"Pink Fizz", 3359:"Pink Lotus", 3360:"Pink Rose", 3361:"Pink Sugar", 3362:"Pinkblush", 3363:"Pinky", 3364:"Pinnacle Fishing", 3365:"Pinnacle Systems", 3366:"Pins & Needles", 3367:"Pioneer", 3368:"Pioneer Inc.", 3369:"Pioneer Pet", 3370:"Piperlime", 3371:"Pixi", 3372:"PlaSmart", 3373:"Plackers", 3374:"Planet Earth", 3375:"Planet Gold", 3376:"Planet Hollywood", 3377:"Planet Motherhood", 3378:"Planet Pleasures", 3379:"Plantronics", 3380:"PlastiColor®", 3381:"Play Along", 3382:"Play MG", 3383:"Play-Doh", 3384:"PlayStation", 3385:"Playboy", 3386:"Playhut", 3387:"Playmates", 3388:"Playmobil", 3389:"Playskool", 3390:"Playstar", 3391:"Playtex", 3392:"Pleasure Party", 3393:"Pleione", 3394:"Plugg", 3395:"Plus White", 3396:"PoGo! Products", 3397:"Poetry", 3398:"Point Zero", 3399:"Poise", 3400:"Pokemon", 3401:"Pokemon USA", 3402:"Polar", 3403:"Polar Pro Filters", 3404:"Polaroid", 3405:"Poleci", 3406:"Poligrip", 3407:"Polk Audio", 3408:"Polly & Esther", 3409:"Polo Jeans Co.", 3410:"Polo Ralph Lauren", 3411:"Polycom", 3412:"Poof!", 3413:"Pop Beauty", 3414:"PopCap", 3415:"PopSockets", 3416:"Popatu", 3417:"Popular", 3418:"Porsche Design", 3419:"Port & Company", 3420:"Port Authority", 3421:"Positive Attitude", 3422:"Post-it", 3423:"Pottery Barn", 3424:"Pottery Barn Kids", 3425:"Power A", 3426:"Power Wheels", 3427:"Powerbolt", 3428:"Prabal Gurung", 3429:"Prabal Gurung for Target", 3430:"Prada", 3431:"Prairie Dog Antlers", 3432:"Pravana", 3433:"Pre-Seed", 3434:"Precious Moments", 3435:"Precision", 3436:"Preggers", 3437:"Premier Designs", 3438:"Prescriptives", 3439:"Pressman Toy", 3440:"Prestige", 3441:"Presto", 3442:"Preston & York", 3443:"Pretty Angel", 3444:"Pretty Polly", 3445:"Prevail", 3446:"Prevue Pet Products", 3447:"Primal Wear", 3448:"Primark", 3449:"Primitive", 3450:"Primo Emporio", 3451:"Primula", 3452:"Prince", 3453:"Prince LionHeart", 3454:"Pringle of Scotland", 3455:"Prinz", 3456:"Privilege", 3457:"Pro", 3458:"Pro Edge", 3459:"Pro Keds", 3460:"Pro Line", 3461:"Pro Player", 3462:"ProClub", 3463:"ProTec", 3464:"Proactiv", 3465:"Proctor Silex", 3466:"Proenza Schouler", 3467:"Profile by Gottex", 3468:"Profound Aesthetic", 3469:"Progressive International", 3470:"Pronto Uomo", 3471:"Propet", 3472:"Protege", 3473:"Psycho Bunny", 3474:"Publish", 3475:"Puffs", 3476:"Puj", 3477:"Pull&Bear", 3478:"Pulsar", 3479:"Puma SE", 3480:"Punch Studio", 3481:"Puppia", 3482:"Pur Minerals", 3483:"Pur-lisse", 3484:"Pura", 3485:"Pure Boxing", 3486:"Pure Color", 3487:"Pure Energy", 3488:"Pure Life", 3489:"Pureology", 3490:"Purina", 3491:"Purina Beneful", 3492:"Purina One", 3493:"Puritan", 3494:"PylePro", 3495:"Pyrex", 3496:"QALO", 3497:"Quacker Factory", 3498:"Quantum", 3499:"Quantum Mechanix", 3500:"Quartet", 3501:"Quay Australia", 3502:"Que", 3503:"Quicksilver", 3504:"Quiksilver", 3505:"Quiksilver Waterman Collection", 3506:"Quinny", 3507:"Qupid", 3508:"R & M Richards", 3509:"R&K Originals", 3510:"RACHEL PALLY", 3511:"RAW", 3512:"RBK", 3513:"RCA", 3514:"RED Valentino", 3515:"REI", 3516:"REN", 3517:"REPORT", 3518:"RIM", 3519:"RJ Story", 3520:"RLX Ralph Lauren", 3521:"ROCKY", 3522:"ROMWE", 3523:"ROOK", 3524:"RVCA", 3525:"RXB", 3526:"Rachael Ray", 3527:"Rachel Pally White Label", 3528:"Rachel Roy", 3529:"Rachel Zoe", 3530:"Radio Flyer", 3531:"Rae Dunn", 3532:"Rafaella", 3533:"Rage Broadheads", 3534:"Rags to Raches", 3535:"Rails", 3536:"Rain-X", 3537:"Rainbow", 3538:"Rainbow Shops", 3539:"Raisins", 3540:"Ralph Lauren", 3541:"Ralph Lauren Black Label", 3542:"Ralph Lauren Blue Label", 3543:"Ralph Lauren Collection", 3544:"Ram", 3545:"Ramones", 3546:"Rampage", 3547:"Rand McNally", 3548:"Rare Editions", 3549:"Rasco FR", 3550:"Raskullz", 3551:"Rave", 3552:"Ravebsburger", 3553:"Raven Denim", 3554:"Ravensburger", 3555:"Raw State", 3556:"Rawlings", 3557:"Ray-Ban", 3558:"Razer", 3559:"Razor", 3560:"Reach", 3561:"Reader's Digest", 3562:"Real Techniques", 3563:"Realtree", 3564:"Reba", 3565:"Rebecca Minkoff", 3566:"Rebecca Taylor", 3567:"Rebel Spirit", 3568:"Rebel Yell", 3569:"Rebellious One", 3570:"Recoton", 3571:"Red Ape", 3572:"Red Camel", 3573:"Red Cherry", 3574:"Red Engine", 3575:"Red Haute", 3576:"Red Jacket", 3577:"Red Kap", 3578:"Red Rivet Jeans", 3579:"Red Wing Shoes", 3580:"RedHead", 3581:"Redington", 3582:"Redken", 3583:"Reebok", 3584:"Reed", 3585:"Reeds Jewelers", 3586:"Reef", 3587:"Reference Point", 3588:"Reformation", 3589:"Refuge", 3590:"Regal", 3591:"Regalo", 3592:"Reign", 3593:"Relativity", 3594:"Relic", 3595:"Religion", 3596:"Remain", 3597:"Remington", 3598:"Rene Rofe", 3599:"Renew Life", 3600:"Renu", 3601:"Renuzit", 3602:"Rep-Cal", 3603:"Replay", 3604:"Replogle", 3605:"Report Collection", 3606:"Report Signature", 3607:"Request", 3608:"Requirements", 3609:"Resistol", 3610:"Restoration Hardware", 3611:"Restricted", 3612:"Retro Chic", 3613:"Retrofit", 3614:"Reunion", 3615:"Reusch", 3616:"Revell", 3617:"Revere", 3618:"Reverie", 3619:"RevitaLens", 3620:"Revlon", 3621:"Revlon Colorsilk", 3622:"Revlon Group", 3623:"Revo", 3624:"Revolt", 3625:"Revolution", 3626:"Rewash", 3627:"Rewind", 3628:"Reyn Spooner", 3629:"Rhapsody", 3630:"Rhonda Shear", 3631:"Rich & Skinny", 3632:"Richell", 3633:"Rick Owens", 3634:"Riddell", 3635:"Riders", 3636:"Riders by Lee", 3637:"Riff Stars", 3638:"Right Guard", 3639:"Rihanna", 3640:"Rimmel", 3641:"Ring", 3642:"Ring of Fire", 3643:"Rio Grande Games", 3644:"Rioni", 3645:"Rip Curl", 3646:"Ripcurl", 3647:"Ripple Junction", 3648:"Riva Designs", 3649:"Rival", 3650:"River Island", 3651:"Riviera", 3652:"RoC", 3653:"Road Rippers", 3654:"Roaman's", 3655:"Roar", 3656:"Robbie Bee", 3657:"Robeez", 3658:"Robert Friedman", 3659:"Robert Graham", 3660:"Robert Rodriguez", 3661:"Robert Talbott", 3662:"Roberta", 3663:"Roberto Cavalli", 3664:"Robin Piccone", 3665:"Robin's Jeans", 3666:"Rocawear", 3667:"Rocawear Clothing", 3668:"Rock & Republic", 3669:"Rock & Roll Cowgirl", 3670:"Rock Revival", 3671:"Rocket Dog", 3672:"Rockies", 3673:"Rockport", 3674:"Rocksmith", 3675:"Rockstar", 3676:"Rocktape", 3677:"Rocky Brands, Inc.", 3678:"Rodan and Fields", 3679:"Rodd & Gunn", 3680:"Rogue", 3681:"Rogue Status", 3682:"Roku", 3683:"Rolex", 3684:"Roller Derby", 3685:"Rollerblade", 3686:"Romeo & Juliet Couture", 3687:"Romy", 3688:"Ron Jon", 3689:"Ronco", 3690:"Ronni Nicole", 3691:"RoomMates", 3692:"Roots", 3693:"Roper", 3694:"RoseArt", 3695:"Rosetti", 3696:"Rosewill", 3697:"Rossignol", 3698:"Rothco", 3699:"Rothschild", 3700:"Roulette", 3701:"Roundtree & Yorke", 3702:"Route", 3703:"Route 66", 3704:"Rowan", 3705:"Rowdy Gentleman", 3706:"Roxy", 3707:"Royal", 3708:"Royal Apparel", 3709:"Royal Robbins", 3710:"Royce", 3711:"Rubbermaid", 3712:"Rubbish", 3713:"Rubies Costume Company", 3714:"Ruby Rd.", 3715:"Ruby Rox", 3716:"Rue21", 3717:"Ruehl No. 925", 3718:"Ruff Hewn", 3719:"Rugged Ridge", 3720:"Rumparooz", 3721:"Rusk", 3722:"Russ", 3723:"Russell Athletic", 3724:"Rustler", 3725:"Rustlers", 3726:"Rusty", 3727:"Ruth", 3728:"Ryka", 3729:"SAS Safety Corp.", 3730:"SCHUTZ", 3731:"SEPHORA COLLECTION", 3732:"SIGVARIS", 3733:"SJB Active", 3734:"SJP by Sarah Jessica Parker", 3735:"SK-II", 3736:"SKECHERS", 3737:"SKLZ", 3738:"SL Fashions", 3739:"SO", 3740:"SPANX", 3741:"SPY", 3742:"STX", 3743:"SUGOI", 3744:"SWAK", 3745:"Sabo Skirt", 3746:"Sacred Threads", 3747:"Saddlebred", 3748:"Sadie & Scout", 3749:"Safety 1st", 3750:"Safety st", 3751:"Saffron Rouge", 3752:"Sag Harbor", 3753:"Saga", 3754:"Saint Laurent", 3755:"Saint Tropez West", 3756:"Saks Fifth Avenue", 3757:"Sally Hansen", 3758:"Salomon", 3759:"Salt Lake Clothing", 3760:"Salt Water Sandals by Hoy", 3761:"Salt Works", 3762:"Salton", 3763:"Salvage", 3764:"Salvatore Ferragamo", 3765:"Sam & Libby", 3766:"Sam Edelman", 3767:"Samsonite", 3768:"Samsung", 3769:"Samsung Galaxy", 3770:"Samuel Dong", 3771:"San Lorenzo", 3772:"SanDisk", 3773:"Sanctuary", 3774:"Sandra Darren", 3775:"Sandra Ingrish", 3776:"Sangean", 3777:"Sangria", 3778:"Sanrio", 3779:"Sans Souci", 3780:"Sansabelt", 3781:"Santa Cruz", 3782:"Sante", 3783:"Sanuk", 3784:"Sanyo", 3785:"Sarah Spencer", 3786:"Sass & Bide", 3787:"Sasson", 3788:"Sassy", 3789:"Saucony", 3790:"Savane", 3791:"Savannah", 3792:"Savile Row", 3793:"Savvy", 3794:"Saxx", 3795:"Say What?", 3796:"Sbicca", 3797:"Scala", 3798:"Scarlett", 3799:"ScentBlocker", 3800:"Scentsy", 3801:"Schick", 3802:"Schleich", 3803:"Schmidt", 3804:"Scholastic", 3805:"Scholastic Children", 3806:"Schumacher Electric", 3807:"Schutt", 3808:"Schwarzkopf", 3809:"Schwinn", 3810:"Schylling", 3811:"Scope Brand Shop", 3812:"Scosche", 3813:"Scotch", 3814:"Scotch & Soda", 3815:"Scott McClintock", 3816:"Scott Paper", 3817:"Scott®", 3818:"Screen Stars", 3819:"Scrub Daddy", 3820:"Scunci", 3821:"Sea Wees", 3822:"Seafolly", 3823:"Seagate", 3824:"Sean Collection", 3825:"Sean John", 3826:"Sean John Fashion", 3827:"Sears", 3828:"Sebago", 3829:"Sebastian", 3830:"Sebastian Professional", 3831:"Secret", 3832:"See by Chloe", 3833:"Sega", 3834:"Seiko", 3835:"Sejour", 3836:"Select", 3837:"Self Esteem", 3838:"SeneGence", 3839:"Sennheiser", 3840:"Sensodyne", 3841:"Sentry", 3842:"Sephora", 3843:"Sephora Favorites", 3844:"Sequin Hearts", 3845:"Seraphine", 3846:"Sere Nade", 3847:"Serenity", 3848:"Seresto", 3849:"Sergeants", 3850:"Serious Skin Care", 3851:"Serta", 3852:"Sesame Street", 3853:"Seven Dials", 3854:"Seven7", 3855:"Seychelles", 3856:"Shabby Chic", 3857:"Shaka Wear", 3858:"Shane Company", 3859:"Shany", 3860:"Sharagano", 3861:"Sharp", 3862:"Sharper Image", 3863:"Sharpie", 3864:"Shaun White", 3865:"She and Sky", 3866:"Shea Moisture", 3867:"Sheba", 3868:"Sheer Cover", 3869:"Sheffield Knives", 3870:"Shell", 3871:"Shenanigans", 3872:"Sherpa", 3873:"Sherri Hill", 3874:"Shift Management", 3875:"Shimano Fishing", 3876:"Shinola", 3877:"Shirley Of Hollywood", 3878:"Shiseido", 3879:"Shock Absorber", 3880:"Shock Doctor", 3881:"Shoe Dazzle", 3882:"Shoshanna", 3883:"Shout", 3884:"Show Me Your MuMu", 3885:"Showtime", 3886:"Shu Uemura", 3887:"Shure", 3888:"Sideout", 3889:"Signature", 3890:"Signorelli", 3891:"Sigrid Olsen", 3892:"Silence + Noise", 3893:"Silhouette", 3894:"Silhouettes", 3895:"Silkies", 3896:"Silkland", 3897:"Silly Bandz", 3898:"Silpada", 3899:"Silver", 3900:"Silver Jeans", 3901:"Silver Jeans Co.", 3902:"Silver Star", 3903:"Silverlit", 3904:"Similac", 3905:"Simms", 3906:"Simple", 3907:"SimpleHuman", 3908:"Simplicity", 3909:"Simply Be", 3910:"Simply Irresistible", 3911:"Simply Southern", 3912:"Simply Vera Vera Wang", 3913:"Sinful", 3914:"Sinful By Affliction", 3915:"Singing Machine", 3916:"Sirius Satellite Radio", 3917:"Sisley", 3918:"Siwy Jeans", 3919:"Skagen", 3920:"Skechers Kids", 3921:"Sketchers", 3922:"Skin Inc.", 3923:"Skin Industries", 3924:"Skin Laundry", 3925:"SkinCeuticals", 3926:"Skintimate", 3927:"Skip Hop", 3928:"Skullcandy", 3929:"Sky", 3930:"Sky Organics", 3931:"SkyCaddie", 3932:"Skywalker Trampolines", 3933:"Slatkin & Co.", 3934:"Slazenger", 3935:"Sleek MakeUP", 3936:"Sleeping on Snow", 3937:"Slime®", 3938:"Slinky Brand", 3939:"Smart Games", 3940:"Smart Gear", 3941:"SmartWool", 3942:"Smartzone", 3943:"Smashbox", 3944:"Smashbox Cosmetics", 3945:"Smead", 3946:"Smet", 3947:"Smith Optics", 3948:"Smithsonian", 3949:"Smoky Mountain", 3950:"Snap", 3951:"Snap-on", 3952:"Snark", 3953:"Snuggie", 3954:"Social Collision", 3955:"Sock It to Me", 3956:"Soda", 3957:"Sof Sole", 3958:"Soffe", 3959:"Sofft", 3960:"Soft Joie", 3961:"Soft Surroundings", 3962:"Softcup", 3963:"Softsoap", 3964:"Soho Apparel", 3965:"Sol de Janeiro", 3966:"Solar", 3967:"Sole Society", 3968:"Solo", 3969:"Soludos", 3970:"Solvit", 3971:"Soma", 3972:"Somedays Lovin", 3973:"Sonia Kashuk", 3974:"Sonia Rykiel", 3975:"Sonix", 3976:"Sonoma", 3977:"Sony", 3978:"Sony Electronics", 3979:"Sony Pictures", 3980:"Sophia", 3981:"Soprano", 3982:"Sorel", 3983:"Sorrelli", 3984:"Soulmates", 3985:"South Pole", 3986:"Southern Enterprises", 3987:"Southern Living", 3988:"Southern Marsh", 3989:"Southpole", 3990:"Sovereign Code", 3991:"Soybu", 3992:"Spalding", 3993:"Sparkle & Fade", 3994:"Sparks", 3995:"Speck", 3996:"Spectra", 3997:"SpeeCo", 3998:"Speechless", 3999:"Speed Stick", 4000:"Speedo", 4001:"Spell & The Gypsy Collective", 4002:"Spencer's", 4003:"Spense", 4004:"Spenser Jeremy", 4005:"Sperry", 4006:"Sperry Top-Sider", 4007:"Sperrys", 4008:"Spider-Man", 4009:"Spiderman", 4010:"Spiegel", 4011:"Spigen", 4012:"Spin Master", 4013:"Spin Masters", 4014:"Spitfire", 4015:"Splash", 4016:"Splat", 4017:"Splendid", 4018:"Split", 4019:"Spool 72", 4020:"Sporn", 4021:"Sport Savvy", 4022:"Sport-Tek", 4023:"SportBrain", 4024:"Sprout", 4025:"Spy Tec", 4026:"Spyder", 4027:"Square", 4028:"Square Enix", 4029:"Squeeze", 4030:"Srixon", 4031:"St. Croix", 4032:"St. Ives", 4033:"St. John", 4034:"St. John's Bay", 4035:"St. Tropez Tanning Essentials", 4036:"Stacy Adams", 4037:"Stafford", 4038:"Stampd", 4039:"Stamped", 4040:"Stance", 4041:"Stanley", 4042:"Star City", 4043:"Star Wars", 4044:"Starbucks", 4045:"Staring at Stars", 4046:"Starmark", 4047:"Startee", 4048:"Starter", 4049:"Stats", 4050:"Stayfree", 4051:"Steadfast Brand", 4052:"Stefano Ricci", 4053:"Steiff", 4054:"Stella & Dot", 4055:"Stella McCartney", 4056:"Step", 4057:"Stephen Webster", 4058:"Sterling Industries", 4059:"Sterling Publishing", 4060:"Stetson", 4061:"Steve & Barry's", 4062:"Steve Madden", 4063:"Steven by Steve Madden", 4064:"Stila", 4065:"Stila Cosmetics", 4066:"Stitch's", 4067:"Stitches", 4068:"Stokke", 4069:"Stone Cold Fox", 4070:"Stone Fox Swim", 4071:"Stonebridge", 4072:"Stoosh", 4073:"Stop Staring", 4074:"Storkcraft", 4075:"Stormtech", 4076:"Stranded", 4077:"Streamlight®", 4078:"Streetwise", 4079:"StriVectin", 4080:"Stride Rite", 4081:"Structure", 4082:"Stuart Weitzman", 4083:"Studio", 4084:"Studio Ease", 4085:"Studio I", 4086:"Studio M", 4087:"Studio Works", 4088:"Studio Y", 4089:"Stuhrling Original", 4090:"Stussy", 4091:"Style & Co", 4092:"Style&co.", 4093:"Styli-Style", 4094:"Stüssy", 4095:"Suave", 4096:"Sue Devitt", 4097:"Sue Wong", 4098:"Sugarpill", 4099:"Suit Studio", 4100:"Sullen Clothing", 4101:"Sultra", 4102:"Sumersault", 4103:"Summer Breeze", 4104:"Summer Infant", 4105:"Summer's Eve", 4106:"Sun Mountain", 4107:"Sun River", 4108:"Sunbeam", 4109:"Sundance", 4110:"Sunday Riley", 4111:"Sundog", 4112:"Sunny Leigh", 4113:"Super Stroke", 4114:"Super Sunglasses", 4115:"Superdry", 4116:"Superga", 4117:"Superior", 4118:"Supra", 4119:"Supreme", 4120:"Sure", 4121:"Susan Bristol", 4122:"Susan Graver", 4123:"Susan Lawrence", 4124:"Susana Monaco", 4125:"Sutton Studio", 4126:"Suzi Chin", 4127:"Swaddle Designs", 4128:"Swarovski", 4129:"Swedish Hasbeens", 4130:"Sweet Heart Rose", 4131:"Sweet Jojo Designs", 4132:"Sweet Pea", 4133:"Swell", 4134:"Swiffer", 4135:"Swimways", 4136:"Swiss Legend", 4137:"Swix", 4138:"Sylvania", 4139:"Syroco", 4140:"T Tahari", 4141:"T by Alexander Wang", 4142:"T&J Designs", 4143:"T-Party", 4144:"T-fal", 4145:"T.J.Maxx", 4146:"TAVIK swimwear", 4147:"TDK", 4148:"TEHAMA", 4149:"TENA", 4150:"TETON Sports", 4151:"TIGI", 4152:"TIGLIO", 4153:"TOCCA", 4154:"TOMS", 4155:"TOMY", 4156:"TP-Link", 4157:"TRESemme", 4158:"TRU-SPEC", 4159:"TYR", 4160:"Taboo", 4161:"Tacori", 4162:"Tadashi Shoji", 4163:"Tag", 4164:"Tag Heuer", 4165:"Tag Jeans", 4166:"Taggies", 4167:"Tahari", 4168:"Tail", 4169:"TailorByrd", 4170:"Takara", 4171:"Takeout", 4172:"Talbots", 4173:"Tampax", 4174:"Tangents", 4175:"Tangerine", 4176:"Tangle Teezer", 4177:"Tanner Mark", 4178:"Tano", 4179:"TapouT", 4180:"Tara Toys", 4181:"Target", 4182:"Targus", 4183:"Tarina Tarantino", 4184:"Tart", 4185:"Tarte", 4186:"Taryn Rose", 4187:"Tassimo", 4188:"Tasso Elba", 4189:"Tatcha", 4190:"Taverniti", 4191:"Taylor", 4192:"Taylor Dresses", 4193:"TaylorMade", 4194:"Tbags Los Angeles", 4195:"Tea Collection", 4196:"Team Apparel", 4197:"Tech  Kids", 4198:"Technics", 4199:"Techno Source", 4200:"Technomarine", 4201:"Tecnica", 4202:"Ted Baker", 4203:"Ted Baker London", 4204:"Tee Shop", 4205:"TeeFury", 4206:"Teenage Mutant Ninja Turtles", 4207:"Teeze Me", 4208:"Tek Gear", 4209:"Tek Nek", 4210:"Telebrands", 4211:"Teleflora", 4212:"Ten Sixty Sherman", 4213:"Tennessee River", 4214:"Terani Couture", 4215:"Terry Lewis", 4216:"Tervis", 4217:"Tess", 4218:"Testament", 4219:"Tetra", 4220:"Teva", 4221:"Tevolio", 4222:"Texas Instruments", 4223:"Thames & Kosmos", 4224:"The Balm", 4225:"The Body Shop", 4226:"The Bridge Direct", 4227:"The Children's Place", 4228:"The Doctor's", 4229:"The Estée Edit", 4230:"The First Years", 4231:"The Freshnes", 4232:"The Honest Company", 4233:"The Hundreds", 4234:"The Instrument Store", 4235:"The Learning Journey", 4236:"The Limited", 4237:"The Magic Bullet", 4238:"The Maya Group", 4239:"The Mountain", 4240:"The Noco Company", 4241:"The North Face", 4242:"The Northwest Company", 4243:"The Original Muck Boot Co.", 4244:"The Peanut Shell", 4245:"The Sak", 4246:"The Timberland Company", 4247:"The Wet Brush", 4248:"The Whistler Group", 4249:"The Wish Factory", 4250:"TheLees", 4251:"Theory", 4252:"Therapy", 4253:"Thermos", 4254:"Thierry Mugler", 4255:"ThinkFun", 4256:"Thinkway", 4257:"Thom Browne", 4258:"Thomas & Friends", 4259:"Thor", 4260:"Thrasher", 4261:"Thrasher Magazine", 4262:"Threadless", 4263:"Threads  Thought", 4264:"Threads 4 Thought", 4265:"Three Dots", 4266:"Three Hearts", 4267:"Three Seasons Maternity", 4268:"Thrive", 4269:"Thule", 4270:"Thundershirt", 4271:"Tiana b", 4272:"Tianello", 4273:"Tiara", 4274:"Tiara International", 4275:"Tiare Hawaii", 4276:"Tibi", 4277:"Tick Twister", 4278:"Tickled Pink Tees", 4279:"Tide", 4280:"Tidy Cats", 4281:"Tieks", 4282:"Tiffany & Co.", 4283:"Tiffany Designs", 4284:"Tiger", 4285:"Tigerlily", 4286:"Tignanello", 4287:"Tildon", 4288:"Tile", 4289:"Tilly's", 4290:"Tilt", 4291:"Timber Creek", 4292:"Timberland", 4293:"Timex", 4294:"Timing", 4295:"Tinker Bell", 4296:"Tinley Road", 4297:"Tinseltown", 4298:"Tiny Love", 4299:"Tissot", 4300:"Titleist", 4301:"Tivoli", 4302:"To the Max", 4303:"Toastmaster", 4304:"Tobi", 4305:"Tod's", 4306:"Together", 4307:"Tolani", 4308:"Tolly Tots", 4309:"Tom Ford", 4310:"Tom James", 4311:"Tom's of Maine", 4312:"TomTom", 4313:"Tommee Tippee", 4314:"Tommy Bahama", 4315:"Tommy Hilfiger", 4316:"Tommy John", 4317:"Tone", 4318:"Tonka", 4319:"Tonner Doll Company", 4320:"Tony & Tina", 4321:"Tony Bowls", 4322:"Tony Hawk", 4323:"Tony Lama", 4324:"Tony Moly", 4325:"Too Faced", 4326:"Too Faced x Kat Von D", 4327:"Too Fast", 4328:"Tools of the Trade", 4329:"Tootsie", 4330:"Top Fin", 4331:"Top Flite", 4332:"Top Paw", 4333:"Top of the World", 4334:"TopShop", 4335:"Topman", 4336:"Topps", 4337:"Topshop Petite", 4338:"Torn by Ronny Kobo", 4339:"Torrid", 4340:"Tory Burch", 4341:"Toshiba", 4342:"Tous", 4343:"Towncraft", 4344:"Toy Monster", 4345:"Toynami", 4346:"Toys R Us", 4347:"Toys R Us Plush", 4348:"Toysmith", 4349:"Tractr", 4350:"Tracy Evans", 4351:"Tracy Reese", 4352:"Travis Mathew", 4353:"Treasure & Bond", 4354:"Trench", 4355:"Trend Lab", 4356:"Tresics", 4357:"Tretorn", 4358:"Trevco", 4359:"Tria", 4360:"Triangl", 4361:"Tribal", 4362:"Tribest", 4363:"Tricots St. Raphael", 4364:"Trina Turk", 4365:"Triple 5 Soul", 4366:"Triple Five Soul", 4367:"Tripp", 4368:"Tripp NYC", 4369:"Trish McEvoy", 4370:"Triumph", 4371:"Trixxi", 4372:"Trouve", 4373:"True Grit", 4374:"True Light", 4375:"True Religion", 4376:"True Religion Brand Jeans", 4377:"True Rock", 4378:"Trukfi", 4379:"Truly Scrumptious", 4380:"Trunk Ltd", 4381:"Tularosa", 4382:"Tulle", 4383:"Tultex", 4384:"Tumi", 4385:"Tupperware", 4386:"Turnbull & Asser", 4387:"Turnbury", 4388:"Turtle Beach", 4389:"Tweeds", 4390:"Tweezerman", 4391:"Twelfth Street by Cynthia Vincent", 4392:"Twenty One", 4393:"Twenty X", 4394:"Twiggy", 4395:"Twilight Gypsy Collective", 4396:"Twisted Heart", 4397:"Twisted X", 4398:"Two In The Shirt", 4399:"Two by Vince Camuto", 4400:"Tylenol", 4401:"Tyler Rose", 4402:"Tyra Beauty", 4403:"Tyte Jeans", 4404:"U.S. Polo Assn.", 4405:"UFC", 4406:"UGG", 4407:"UGG Australia", 4408:"UNIF", 4409:"UNIONBAY", 4410:"UPPAbaby", 4411:"US POLO ASSN", 4412:"USA Hockey", 4413:"USA Leather", 4414:"USAF", 4415:"USAopoly", 4416:"Ubbi", 4417:"Ubisoft", 4418:"Ugg boots", 4419:"Uglydoll", 4420:"Ujena", 4421:"Ulla Popken", 4422:"Ulta", 4423:"Ulta Salon, Cosmetics & Fragrance, Inc.", 4424:"Ultima II", 4425:"Ultimate Ears", 4426:"UltimateTV RCA", 4427:"Ultra Flirt", 4428:"Ultra Pro", 4429:"Umbra", 4430:"Umbro", 4431:"Umo Lorenzo", 4432:"Uncle Milton", 4433:"Uncommon Threads", 4434:"Undefeated", 4435:"Under Armour", 4436:"Undercover Mama", 4437:"Underscore", 4438:"Uniden", 4439:"Unik", 4440:"Uniqlo", 4441:"Unique Creations", 4442:"Unisa", 4443:"United Colors Of Benetton", 4444:"United States Sweaters", 4445:"Universal Baby", 4446:"Universal Studios", 4447:"Univsersal Studios", 4448:"Unlisted", 4449:"Unwired Technologies", 4450:"Up Vibe", 4451:"Update International", 4452:"Upspring", 4453:"Urban Behavior", 4454:"Urban Decay", 4455:"Urban Expressions", 4456:"Urban Outfitters", 4457:"Urban Pipeline", 4458:"Urban Trends", 4459:"Urban Up", 4460:"VANS", 4461:"VIGOSS", 4462:"VISSLA", 4463:"VTech", 4464:"Va Va Voom", 4465:"Valco Baby", 4466:"Valentino", 4467:"Valentino Garavani", 4468:"Valerie Stevens", 4469:"Valve", 4470:"Van Cleef & Arpels", 4471:"Van Heusen", 4472:"Vaneli", 4473:"Vanilla Star", 4474:"Vanilla Sugar", 4475:"Vanity", 4476:"Vantage", 4477:"Vaseline", 4478:"Vasque", 4479:"Veet", 4480:"Velocity", 4481:"Velvet", 4482:"Velvet Torch", 4483:"Velvet by Graham & Spencer", 4484:"Venezia", 4485:"Venus", 4486:"Vera Bradley", 4487:"Vera Pelle", 4488:"Vera Wang", 4489:"Veronica Beard", 4490:"Versace", 4491:"Versace Collection", 4492:"Versace Jeans", 4493:"Vertigo", 4494:"Vertigo Paris", 4495:"ViX", 4496:"Via Spiga", 4497:"Vibram", 4498:"Vic Firth", 4499:"Vichy", 4500:"Vickerman", 4501:"Vicks", 4502:"Victor Costa", 4503:"Victoria Beckham", 4504:"Victoria Jones", 4505:"Victoria's Secret", 4506:"Victorinox Swiss Army®", 4507:"Victorious", 4508:"Vidal Sassoon", 4509:"ViewSonic", 4510:"Viktor & Rolf", 4511:"Vilebrequin", 4512:"Villager", 4513:"Vince", 4514:"Vince Camuto", 4515:"Vincent Longo", 4516:"Vintage", 4517:"Vintage Havana", 4518:"Vintage Red", 4519:"Violet & Claire", 4520:"Viper", 4521:"Virgin Only", 4522:"Virgo", 4523:"Viseart", 4524:"Visvim", 4525:"Vitakraft", 4526:"Vital Baby", 4527:"Vitamin A", 4528:"Vitamins Baby", 4529:"Vitamix", 4530:"Vittorio St. Angelo", 4531:"Vivid Imaginations", 4532:"Vivienne Vivienne Tam", 4533:"Vivienne Westwood", 4534:"Viviscal", 4535:"Vixen", 4536:"Viz Media", 4537:"Vocal", 4538:"Volatile", 4539:"Volcom", 4540:"Vollrath", 4541:"Von Dutch", 4542:"Voodoo Vixen", 4543:"WB Games", 4544:"WD-40®", 4545:"WD.NY", 4546:"WILLOW", 4547:"WOW couture", 4548:"WWE", 4549:"Wacoal", 4550:"Wacom", 4551:"Wagner Lighting", 4552:"Wahl", 4553:"Wall Pops", 4554:"Wallflower", 4555:"Walls", 4556:"Walt Disney Studios Entertainment", 4557:"Walt Disney Studios Home Entertainment", 4558:"Walter Hagen", 4559:"Wanted", 4560:"Warby Parker", 4561:"Ware", 4562:"Warehouse", 4563:"Waring", 4564:"Warner Bros.", 4565:"Warner Brothers", 4566:"Warner Home Video", 4567:"Warrior", 4568:"Waterford", 4569:"Waterpik", 4570:"Waverly", 4571:"We The Free", 4572:"WeSC", 4573:"WearFirst", 4574:"Weather Tamer", 4575:"Weatherproof", 4576:"Weaver Leather", 4577:"Weavers", 4578:"Wedgwood", 4579:"Weight Watchers", 4580:"Wellness", 4581:"Wembley", 4582:"Wendy Bellissimo", 4583:"West Bend", 4584:"West Coast Choppers", 4585:"West Elm", 4586:"Westbound", 4587:"Western Chief", 4588:"Western Digital", 4589:"Western Express", 4590:"Weston", 4591:"Wet Seal", 4592:"Wet n Wild", 4593:"Wham-O", 4594:"Whirlpool", 4595:"Whish", 4596:"Whiskas", 4597:"Whisker City", 4598:"White + Warren", 4599:"White Diamond", 4600:"White House Black Market", 4601:"White Mark", 4602:"White Mountain", 4603:"White Mountaineering", 4604:"White Stag", 4605:"Wigwam", 4606:"Wild Diva", 4607:"Wild Republic", 4608:"Wildfox", 4609:"Wildfox Couture", 4610:"Wildkin", 4611:"Willi Smith", 4612:"William Rast", 4613:"Williams Sonoma", 4614:"Willow & Clay", 4615:"Willow Tree", 4616:"Wilson", 4617:"Wilsons Leather", 4618:"Wilton", 4619:"Winchester", 4620:"Windsor", 4621:"Winner's Circle", 4622:"Winterland", 4623:"Wish", 4624:"Wit & Wisdom", 4625:"Wizards of the Coast", 4626:"Wolfgang Puck", 4627:"Wolford", 4628:"Wolverine", 4629:"Woman Within", 4630:"Woman's Touch Apparel", 4631:"Wonderworld", 4632:"WoodWick", 4633:"Woolrich", 4634:"Woot", 4635:"Workman", 4636:"World's Best", 4637:"Worth", 4638:"Worthington", 4639:"WowWee", 4640:"Wrangler", 4641:"Wrapper", 4642:"WubbaNub", 4643:"Wusthof", 4644:"X", 4645:"XCVI", 4646:"XEN-TAN", 4647:"XM", 4648:"XOXO", 4649:"XXI", 4650:"Xbox", 4651:"Xelement", 4652:"Xenith", 4653:"Xerox", 4654:"Xersion", 4655:"Xhilaration", 4656:"Xscape", 4657:"Xtreme Couture", 4658:"Y- by Yohji Yamamoto", 4659:"YBike", 4660:"YMI", 4661:"YMI Jeans", 4662:"YRU", 4663:"YSL Yves Saint Laurent", 4664:"Ya Los Angeles", 4665:"Yada", 4666:"Yakima", 4667:"Yamaha", 4668:"Yankee Candle", 4669:"Yankee Candle Company", 4670:"Yeezy", 4671:"Yelete", 4672:"Yellow Box", 4673:"Yes To", 4674:"Yeti", 4675:"Yigal Azrouel", 4676:"Yoana Baraschi", 4677:"Yohji Yamamoto", 4678:"Yoki", 4679:"Yosi Samra", 4680:"Young & Reckless", 4681:"Young Fabulous & Broke", 4682:"Youngblood", 4683:"Youngland", 4684:"Younique", 4685:"Younique products", 4686:"Youth To The People", 4687:"Yumi Kim", 4688:"Yummie Tummie", 4689:"Yummie by Heather Thomson", 4690:"Yves Saint Laurent", 4691:"Yvolution", 4692:"Z Cavaricci", 4693:"Z-Man Games", 4694:"ZARA", 4695:"ZCO", 4696:"ZENA", 4697:"Zac Posen", 4698:"Zadig & Voltaire", 4699:"Zaggora", 4700:"Zak Designs", 4701:"Zales", 4702:"Zana Di", 4703:"Zanella", 4704:"Zanerobe", 4705:"Zara Terez", 4706:"Zco.", 4707:"Zella", 4708:"Zen Group", 4709:"Zenana Outfitters", 4710:"ZeroXposur", 4711:"Zigi Soho", 4712:"Zilla", 4713:"Zimmerman", 4714:"Zion Rootswear", 4715:"Zobo", 4716:"Zoe Ltd", 4717:"Zojirushi", 4718:"Zoli", 4719:"Zoo Med", 4720:"Zoo York", 4721:"Zoom", 4722:"Zoot", 4723:"Zum Zum", 4724:"Zumba", 4725:"Zune", 4726:"ZyXEL", 4727:"a'gaci", 4728:"a.n.a", 4729:"a.testoni", 4730:"abercrombie kids", 4731:"aden & anais", 4732:"adidas NEO", 4733:"adidas Originals", 4734:"adidas by Stella McCartney", 4735:"aerie", 4736:"amika", 4737:"amy coe", 4738:"anon", 4739:"b. makowsky", 4740:"b.o.c.", 4741:"b.tempt'd", 4742:"bareMinerals", 4743:"beautyblender®", 4744:"bernie mev.", 4745:"bp", 4746:"bumGenius", 4747:"butter LONDON", 4748:"crossroads", 4749:"dELiA*s", 4750:"deBeer", 4751:"deborah lippmann", 4752:"dreamGEAR", 4753:"dreambaby", 4754:"dressbarn", 4755:"e.l.f.", 4756:"elite HOCKEY", 4757:"eos", 4758:"eshakti", 4759:"ethika", 4760:"etnies", 4761:"frogg toggs", 4762:"gDiapers", 4763:"goDog", 4764:"gorjana", 4765:"grass", 4766:"hm-moden", 4767:"iHome", 4768:"iRobot", 4769:"iXCC", 4770:"iris & ivy", 4771:"kate spade new york", 4772:"kathy ireland", 4773:"kiara", 4774:"l.e.i.", 4775:"lei", 4776:"lillebaby", 4777:"lonna & lilly", 4778:"love on a hanger", 4779:"lululemon athletica", 4780:"maje", 4781:"mamaRoo", 4782:"mandee", 4783:"marc ecko enterprises", 4784:"me too", 4785:"metrostyle", 4786:"miss chic", 4787:"moms", 4788:"monster high", 4789:"naughty monkey", 4790:"nununu", 4791:"office supply brands", 4792:"papi", 4793:"patagonia", 4794:"pet brands", 4795:"plenty by Tracy Reese", 4796:"prAna", 4797:"rag & bone", 4798:"riot society", 4799:"rue", 4800:"sandro", 4801:"seeyou", 4802:"theBalm", 4803:"timi & leslie", 4804:"tm!", 4805:"tokidoki", 4806:"totes ISOTONER", 4807:"triangl swimwear", 4808:"vineyard vines", 4809:"wallis"}


a = datetime.now()
for i in range(len(brand_name_dict)):
	df['brand_name'] = np.where(df['brand_name'] == brand_name_dict[i + 1] , i + 1, df['brand_name'])
print ('i m here3')
b = datetime.now()
print((b - a).total_seconds())

df['brand_name'] = df['brand_name'].astype(dtype=float)
df['category_name'] = df['category_name'].astype(dtype=float)

msk = np.random.rand(len(df)) < 0.8
df_train = df[msk]
df_eval = df[~msk]

print ('i m here4')

# In[2]:

X_train = pd.DataFrame()
X_train['item_condition_id'] = df_train['item_condition_id']
X_train['category_name'] = df_train['category_name']
X_train['brand_name'] = df_train['brand_name']
X_train['shipping'] = df_train['shipping']
Y_train = df_train['price']

X_eval = pd.DataFrame()
X_eval['item_condition_id'] = df_eval['item_condition_id']
X_eval['category_name'] = df_eval['category_name']
X_eval['brand_name'] = df_eval['brand_name']
X_eval['shipping'] = df_eval['shipping']
Y_eval = df_eval['price']


# In[ ]:

params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l1'},
    'num_leaves': 31,
    'learning_rate': 0.1,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.9,
    'bagging_freq': 5,
    'verbose': 0
}

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_eval = lgb.Dataset(X_eval, Y_eval, reference=lgb_train)

gbm = lgb.train(params,
			lgb_train,
			num_boost_round=700,
			valid_sets=lgb_eval,
			early_stopping_rounds=10,categorical_feature=[0, 1, 2, 3])
print ('i m here5')

df = pd.read_csv('../input/test.tsv', delimiter='\t',low_memory=True, header = 0)

df['category_name'].fillna(0, inplace=True)
df['brand_name'].fillna(0, inplace=True)
print ('i m here66')
a = datetime.now()
for i in range(len(category_name_dict)):
	df['category_name'] = np.where(df['category_name'] == category_name_dict[i + 1] , i + 1, df['category_name'])
print ('i m here666')
b = datetime.now()
(b - a).total_seconds()
print ('i m here6')
a = datetime.now()
for i in range(len(brand_name_dict)):
	df['brand_name'] = np.where(df['brand_name'] == brand_name_dict[i + 1] , i + 1, df['brand_name'])

b = datetime.now()
(b - a).total_seconds()
print ('i m here8')

def is_string(x):
	try:
		float(x)		
	except:
		#print 'i m here'
		return 0
	return x


df['category_name'] = df.category_name.apply(lambda x: is_string(x))
df['brand_name'] = df.brand_name.apply(lambda x: is_string(x))

df['brand_name'] = df['brand_name'].astype(dtype=float)
df['category_name'] = df['category_name'].astype(dtype=float)
print ('i m here9')

print ('im here 1123')
X_test = pd.DataFrame()
X_test['item_condition_id'] = df['item_condition_id']
X_test['category_name'] = df['category_name']
X_test['brand_name'] = df['brand_name']
X_test['shipping'] = df['shipping']
test_id = df['test_id'].tolist()
print ('i m here 1234')
Y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print ('i m here 111')
Y_pred = [y_pred if y_pred > 0 else 0 for y_pred in Y_pred ]
print ('i m here 112')
with open("sample_submission.csv", 'w') as f:
	f.write('test_id,price\n')
	for i in range(len(Y_pred)):
		f.write('%d,%f\n' % (test_id[i], Y_pred[i]))

print ('complete done')